In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
from random import seed

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import  MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Imputer

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM, BatchNormalization
from keras import layers
from keras import optimizers
from keras import callbacks
from keras import models

Using TensorFlow backend.


In [2]:
#Generate Dataset
#Airfoil Self-Noise
df_airfoil=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Airfoil_self_noise.csv')

#Automobile
df_automobile=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Automobile_data.txt')

#Autompg
df_autompg=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Auto-mpg.csv')

#ChallengerUSA
df_challenger=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Challenger USA Space Shuttle O-Ring.csv')

#Combine Cycle Power
df_combine=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Combine Cycle Power Plant.csv')

#Computer Hardware
df_computer=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Computer Hardware.csv')

#Concrete Compressive Strength
df_concrete_compressive_strength=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Compressive Strength.csv')

#Concrete Slump Test
df_concrete1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete3=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')

#Energy Efficiency
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')

#Fertility Diagnosis
df_fertility=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\fertility_Diagnosis.csv')

#Forest Fires
df_forest=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\forestfires.csv')

#Housing
df_housing=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Housing.csv')

#Istanbul Stock Exchange
df_istanbul=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Istanbul Stock Exchange.csv')

#WDBC
df_wdbc=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\wdbc.csv')

In [3]:
#remove Nan columns of df_energy1
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy1.drop(df_energy1.columns[11],inplace=True, axis = 1)
df_energy1.drop(df_energy1.columns[10],inplace=True, axis = 1)

#remove Nan columns of df_energy2
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2.drop(df_energy2.columns[11],inplace=True, axis = 1)
df_energy2.drop(df_energy2.columns[10],inplace=True, axis = 1)

#remove question marks
#df_automobile
imp= Imputer(missing_values="NaN", strategy="mean")
df_automobile=df_automobile.replace("?", np.NaN)
df_automobile['num-of-doors'] = df_automobile['num-of-doors'].fillna("four")
for i in df_automobile.columns:
    if df_automobile[i].isnull().sum()>0:
        df_automobile[i]=imp.fit_transform(df_automobile[[i]])
#df_automobile = df_automobile[(df_automobile.astype(str) != '?').all(axis=1)]
#df_automobile.reset_index(drop=True, inplace=True)
df_automobile = df_automobile.drop(['engine-location'], axis=1)
#df_autompg
df_autompg = df_autompg[(df_autompg.astype(str) != '?').all(axis=1)]
df_autompg.reset_index(drop=True, inplace=True)

#convert string to number
le=LabelEncoder()
#df_automobile
string_automobile=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=string_automobile.apply(le.fit_transform)
encoder=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']].astype(np.int64)
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=encoder
a=df_automobile[['normalized-losses','horsepower','price']].astype(np.int64)
b=df_automobile[['bore','stroke','peak-rpm']].astype(float)
df_automobile[['normalized-losses','horsepower','price']]=a
df_automobile[['bore','stroke','peak-rpm']]=b
#df_computer
string_computer=df_computer[['Vendor','Model']]
df_computer[['Vendor','Model']]=string_computer.apply(le.fit_transform)
df_computer[['Vendor','Model']]=df_computer[['Vendor','Model']].astype(np.int64)

#dropna spesific column & special preprocessing
#df_forest
df_forest=df_forest[(df_forest['area'].astype(int) != 0)] 
df_forest.reset_index(drop=True, inplace=True)
num_features=10
dimension=(num_features+1)*9+10
population_size=(2*dimension+1)*2
df_forest=df_forest[:population_size]

C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
def preprocessing_data1(df,num_features):
    dimension=(num_features+1)*9+10
    population_size=(2*dimension+1)*2
    df=df.dropna()
    df=df[:population_size]
    df
    return df

In [5]:
def feature_selection(df):
    cor=df.corr()
    plt.figure(figsize=(100,50))
    sns.set(font_scale=3)
    sns.heatmap(cor, annot=True, cmap=plt.cm.Blues)
    plt.show()
    return cor

In [6]:
def train_test_split1(x, y):
    x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.15, random_state=42)
    return x_train, x_test, y_train, y_test

In [7]:
def preprocessing_data2(df):
    train_size = 0.85*len(df)
    train_size = int(train_size)
    df_train = df[:train_size]
    df_test = df[train_size:]
    print(len(df_test))
    print(len(df_train))
    validation_size=len(df_test)/len(df_train)
    print(validation_size)
    return df_train, df_test, validation_size 

In [8]:
#df_airfoil=preprocessing_data1(df_airfoil,5)
#df_automobile=preprocessing_data1(df_automobile,22)
#df_autompg=preprocessing_data1(df_autompg,7)
#df_challenger=preprocessing_data1(df_challenger,3)
#df_combine=preprocessing_data1(df_combine,4)
#df_computer=preprocessing_data1(df_computer,9)
#df_concrete_compressive_strength=preprocessing_data1(df_concrete_compressive_strength,8)
#df_concrete1=preprocessing_data1(df_concrete1,7)
#df_concrete2=preprocessing_data1(df_concrete2,7)
#df_concrete3=preprocessing_data1(df_concrete3,7)
#df_energy1=preprocessing_data1(df_energy1,8)
#df_energy2=preprocessing_data1(df_energy2,8)

#df_fertility=preprocessing_data1(df_fertility,9)
for i in df_fertility:
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'N'] = 1
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'O'] = 2
df_fertility['Diagnosis']=df_fertility['Diagnosis'].astype(np.int64)

#df_forest=preprocessing_data1(df_forest,10)
#df_housing=preprocessing_data1(df_housing,13)

#df_istanbul=preprocessing_data1(df_istanbul,7)
new_header = df_istanbul.iloc[0]
df_istanbul = df_istanbul[1:]
df_istanbul.columns = new_header
df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']]=df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']].astype(np.float64)


#df_wdbc=preprocessing_data1(df_wdbc,10)
df_wdbc.drop(df_wdbc.columns[0], axis=1, inplace = True)
df_wdbc.columns=['id','diagnosis','radius_mean','texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean','concave points_mean','symmetry_mean','fractal_dimension_mean','radius_se','texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se','concave points_se','symmetry_se','fractal_dimension_se','radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst','concave points_worst','symmetry_worst','fractal_dimension_worst']
for i in df_wdbc:
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'M'] = 1
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'B'] = 2
df_wdbc['diagnosis']=df_wdbc['diagnosis'].astype(np.object)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a 

In [9]:
print(df_airfoil.head())
print(df_autompg.head())
print(df_automobile.head())
print(df_challenger.head())

   Frequency  Angle of Attack  Chord length  Free-stream velocity  \
0        800              0.0        0.3048                  71.3   
1       1000              0.0        0.3048                  71.3   
2       1250              0.0        0.3048                  71.3   
3       1600              0.0        0.3048                  71.3   
4       2000              0.0        0.3048                  71.3   

   Suction side displacement thickness  Scaled sound pressure level  
0                             0.002663                      126.201  
1                             0.002663                      125.201  
2                             0.002663                      125.951  
3                             0.002663                      127.591  
4                             0.002663                      127.461  
    Mpg  Cylinders  Displacement Horsepower  Weight  Acceleration  Model Year  \
0  18.0          8         307.0        130    3504          12.0          70   
1  

In [10]:
#Train and Test 17 datasets
#Airfoil
x_airfoil = df_airfoil.drop(['Scaled sound pressure level'], axis = 1)
y_airfoil = df_airfoil['Scaled sound pressure level']
#Wdbc
x_wdbc = df_wdbc[['radius_mean','perimeter_mean','area_mean','concavity_mean','concave points_mean','radius_worst','perimeter_worst','area_worst','concave points_worst']]
y_wdbc = df_wdbc['diagnosis']
labelencoder_Y = LabelEncoder()
y_wdbc = labelencoder_Y.fit_transform(y_wdbc)
#Automobile
x_automobile = df_automobile.drop(['price','make','body-style'], axis = 1)
y_automobile = df_automobile['price']
y_automobile = np.reshape(y_automobile.values, (-1,1))
#Autompg
x_autompg = df_autompg.drop(['Mpg','Car Name'], axis = 1)
y_autompg = df_autompg['Mpg']
#Challenger
x_challenger = df_challenger.drop(['Number of O-rings','Temporal Order of Flight'], axis = 1)
y_challenger = df_challenger['Number of O-rings']
#Combine
x_combine = df_combine.drop(['PE'], axis = 1)
y_combine = df_combine['PE']
y_combine = np.reshape(y_combine.values, (-1,1))
#Computer
x_computer = df_computer.drop(['ERP'], axis = 1)
y_computer = df_computer['ERP']
y_computer = np.reshape(y_computer.values, (-1,1))
#Concrete_compressive_strength
x_concrete_compressive_strength = df_concrete_compressive_strength.drop(['CCS'], axis = 1)
y_concrete_compressive_strength = df_concrete_compressive_strength['CCS']
y_concrete_compressive_strength = np.reshape(y_concrete_compressive_strength.values, (-1,1))
#Concrete1
x_concrete1 = df_concrete1.drop(['No','Compressive Strength (28-day)(Mpa)'], axis = 1)
y_concrete1 = df_concrete1['Compressive Strength (28-day)(Mpa)']
#Concrete2
x_concrete2 = df_concrete2.drop(['No','SLUMP(cm)'], axis = 1)
y_concrete2 = df_concrete2['SLUMP(cm)']
y_concrete2 = np.reshape(y_concrete2.values, (-1,1))
#Concrete3
x_concrete3 = df_concrete3.drop(['No','FLOW(cm)'], axis = 1)
y_concrete3 = df_concrete3['FLOW(cm)']
y_concrete3 = np.reshape(y_concrete3.values, (-1,1))
#Energy1
x_energy1 = df_energy1.drop(['Y1'], axis = 1)
y_energy1 = df_energy1['Y1']
#Energy2
x_energy2 = df_energy2.drop(['Y2'], axis = 1)
y_energy2 = df_energy2['Y2']
#Fertility
x_fertility = df_fertility.drop(['Diagnosis'], axis = 1)
y_fertility = df_fertility['Diagnosis']
#Forest
x_forest = df_forest.drop(['month','day','area'], axis = 1)
y_forest = df_forest['area']
y_forest = np.reshape(y_forest.values, (-1,1))
#Housing
x_housing = df_housing.drop(['MEDV'], axis = 1)
y_housing = df_housing['MEDV']
#Istanbul
x_istanbul = df_istanbul.drop(['date','ISE1','ISE2'], axis = 1)
y_istanbul = df_istanbul[['ISE1','ISE2']]
y_istanbul['mean'] = y_istanbul.mean(axis=1)
y_istanbul = y_istanbul['mean']

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
x_train_airfoil, x_test_airfoil, y_train_airfoil, y_test_airfoil = train_test_split1(x_airfoil, y_airfoil)
x_train_automobile, x_test_automobile, y_train_automobile, y_test_automobile = train_test_split1(x_automobile, y_automobile)
x_train_autompg, x_test_autompg, y_train_autompg, y_test_autompg = train_test_split1(x_autompg, y_autompg)
x_train_challenger, x_test_challenger, y_train_challenger, y_test_challenger = train_test_split1(x_challenger, y_challenger)
x_train_combine, x_test_combine, y_train_combine, y_test_combine = train_test_split1(x_combine, y_combine)
x_train_computer, x_test_computer, y_train_computer, y_test_computer = train_test_split1(x_computer, y_computer)
x_train_concrete_compressive_strength, x_test_concrete_compressive_strength, y_train_concrete_compressive_strength, y_test_concrete_compressive_strength = train_test_split1(x_concrete_compressive_strength, y_concrete_compressive_strength)
x_train_concrete1, x_test_concrete1, y_train_concrete1, y_test_concrete1 = train_test_split1(x_concrete1, y_concrete1)
x_train_concrete2, x_test_concrete2, y_train_concrete2, y_test_concrete2 = train_test_split1(x_concrete2, y_concrete2)
x_train_concrete3, x_test_concrete3, y_train_concrete3, y_test_concrete3 = train_test_split1(x_concrete3, y_concrete3)
x_train_energy1, x_test_energy1, y_train_energy1, y_test_energy1 = train_test_split1(x_energy1, y_energy1)
x_train_energy2, x_test_energy2, y_train_energy2, y_test_energy2 = train_test_split1(x_energy2, y_energy2)
x_train_fertility, x_test_fertility, y_train_fertility, y_test_fertility = train_test_split1(x_fertility, y_fertility)
x_train_forest, x_test_forest, y_train_forest, y_test_forest = train_test_split1(x_forest, y_forest)
x_train_housing, x_test_housing, y_train_housing, y_test_housing = train_test_split1(x_housing, y_housing)
x_train_istanbul, x_test_istanbul, y_train_istanbul, y_test_istanbul = train_test_split1(x_istanbul, y_istanbul)
x_train_wdbc, x_test_wdbc, y_train_wdbc, y_test_wdbc = train_test_split1(x_wdbc, y_wdbc)

In [12]:
def train_val_split(x_train, y_train):
    X_train, X_val, Y_train, Y_val= train_test_split(x_train, y_train, test_size=0.176, random_state=42)
    return X_train, X_val, Y_train, Y_val

In [13]:
def standard_scaler(X_train, X_val, x_test):
    sc=StandardScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=sc.fit(X_train)
    #X_scale_val=sc.fit(X_val)
    #x_scale_test=sc.fit(x_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_train.transform(X_val)
    x_test=X_scale_train.transform(x_test)
    return X_train, X_val, x_test, X_scale_train

In [14]:
def min_max_scaler_automobile(X_train, X_val, x_test, Y_train, Y_val, y_test):
    minmax_x=MinMaxScaler()
    minmax_y=MinMaxScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=minmax_x.fit(X_train)
    #X_scale_val=min_max.fit(X_val)
    #x_scale_test=min_max.fit(x_test)
    Y_scale_train=minmax_y.fit(Y_train)
    #Y_scale_val=min_max.fit(Y_val)
    #y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_train.transform(X_val)
    x_test=X_scale_train.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    Y_val=Y_scale_train.transform(Y_val)
    y_test=Y_scale_train.transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test, X_scale_train, Y_scale_train

In [15]:
def min_max_scaler(X_train, X_val, x_test, Y_train, Y_val, y_test):
    min_max=MinMaxScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=min_max.fit(X_train)
    X_scale_val=min_max.fit(X_val)
    x_scale_test=min_max.fit(x_test)
    Y_scale_train=min_max.fit(Y_train)
    Y_scale_val=min_max.fit(Y_val)
    y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_val.transform(X_val)
    x_test=x_scale_test.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    Y_val=Y_scale_train.transform(Y_val)
    y_test=y_scale_test.transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test, min_max

In [16]:
def normalization(X_train, X_val, x_test, Y_train, Y_val, y_test):
    normalizer=Normalizer()
    X_train=normalizer.fit_transform(X_train)
    X_val=normalizer.fit_transform(X_val)
    x_test=normalizer.fit_transform(x_test)
    Y_train=normalizer.fit_transform(Y_train)
    Y_val=normalizer.fit_transform(Y_val)
    y_test=normalizer.fit_transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test

In [17]:
#Optimizer list
adam0 = optimizers.Adam(lr=0.1)
adam1 = optimizers.Adam(lr=0.001)
adam2 = optimizers.Adam(lr=0.01)
adam3 = optimizers.Adam(lr=0.0001)
adam4 = optimizers.Adam(lr=0.00001)
sgd = optimizers.SGD(lr=0.00001, decay=1e-4, momentum=0.9, nesterov=True)
RMSprop= optimizers.RMSprop(lr=0.01,rho=0.9)
adagrad = optimizers.Adagrad(lr=0.01)
adadelta = optimizers.Adadelta(lr=1.0)

In [18]:
def NN_model_structure_regression_1(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 5))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [19]:
def NN_model_structure_regression_2(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [20]:
def NN_model_structure_regression_3(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 15))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [21]:
def NN_model_structure_regression_4(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 10, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 5))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [22]:
def NN_model_structure_regression_5(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 10, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [23]:
def NN_model_structure_regression_6(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 10, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 15))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [24]:
X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil = train_val_split(x_train_airfoil, y_train_airfoil)
X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile = train_val_split(x_train_automobile, y_train_automobile)
X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg = train_val_split(x_train_autompg, y_train_autompg)
X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger = train_val_split(x_train_challenger, y_train_challenger)
X_train_combine, X_val_combine, Y_train_combine, Y_val_combine = train_val_split(x_train_combine, y_train_combine)
X_train_computer, X_val_computer, Y_train_computer, Y_val_computer = train_val_split(x_train_computer, y_train_computer)
X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength = train_val_split(x_train_concrete_compressive_strength, y_train_concrete_compressive_strength)
X_train_concrete1, X_val_concrete1, Y_train_concrete1, Y_val_concrete1 = train_val_split(x_train_concrete1, y_train_concrete1)
X_train_concrete2, X_val_concrete2, Y_train_concrete2, Y_val_concrete2 = train_val_split(x_train_concrete2, y_train_concrete2)
X_train_concrete3, X_val_concrete3, Y_train_concrete3, Y_val_concrete3 = train_val_split(x_train_concrete3, y_train_concrete3)
X_train_energy1, X_val_energy1, Y_train_energy1, Y_val_energy1 = train_val_split(x_train_energy1, y_train_energy1)
X_train_energy2, X_val_energy2, Y_train_energy2, Y_val_energy2 = train_val_split(x_train_energy2, y_train_energy2)
X_train_fertility, X_val_fertility, Y_train_fertility, Y_val_fertility = train_val_split(x_train_fertility, y_train_fertility)
X_train_forest, X_val_forest, Y_train_forest, Y_val_forest = train_val_split(x_train_forest, y_train_forest)
X_train_housing, X_val_housing, Y_train_housing, Y_val_housing = train_val_split(x_train_housing, y_train_housing)
X_train_istanbul, X_val_istanbul, Y_train_istanbul, Y_val_istanbul = train_val_split(x_train_istanbul, y_train_istanbul)
X_train_wdbc, X_val_wdbc, Y_train_wdbc, Y_val_wdbc = train_val_split(x_train_wdbc, y_train_wdbc)

In [25]:
X_train_airfoil, X_val_airfoil, x_test_airfoil, X_scale_train_airfoil=standard_scaler(X_train_airfoil, X_val_airfoil, x_test_airfoil)
#X_train_automobile, X_val_automobile, x_test_automobile=standard_scaler(X_train_automobile, X_val_automobile, x_test_automobile)
#X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile = normalization(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile, X_scale_train_automobile, Y_scale_train_automobile=min_max_scaler_automobile(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_autompg, X_val_autompg, x_test_autompg, X_scale_train_autompg=standard_scaler(X_train_autompg, X_val_autompg, x_test_autompg)
X_train_challenger, X_val_challenger, x_test_challenger, X_scale_train_challenger=standard_scaler(X_train_challenger, X_val_challenger, x_test_challenger)
X_train_combine, X_val_combine, x_test_combine, Y_train_combine, Y_val_combine, y_test_combine, min_max_combine=min_max_scaler(X_train_combine, X_val_combine, x_test_combine, Y_train_combine, Y_val_combine, y_test_combine)
X_train_computer, X_val_computer, x_test_computer, Y_train_computer, Y_val_computer, y_test_computer, min_max_computer=min_max_scaler(X_train_computer, X_val_computer, x_test_computer, Y_train_computer, Y_val_computer, y_test_computer)
X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, x_test_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength, y_test_concrete_compressive_strength, min_max_concrete_compressive_strength=min_max_scaler(X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, x_test_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength, y_test_concrete_compressive_strength)
X_train_concrete1, X_val_concrete1, x_test_concrete1, X_scale_train_concrete1=standard_scaler(X_train_concrete1, X_val_concrete1, x_test_concrete1)
X_train_concrete2, X_val_concrete2, x_test_concrete2, Y_train_concrete2, Y_val_concrete2, y_test_concrete2, min_max_concrete2=min_max_scaler(X_train_concrete2, X_val_concrete2, x_test_concrete2, Y_train_concrete2, Y_val_concrete2, y_test_concrete2)
X_train_concrete3, X_val_concrete3, x_test_concrete3, Y_train_concrete3, Y_val_concrete3, y_test_concrete3, min_max_concrete3=min_max_scaler(X_train_concrete3, X_val_concrete3, x_test_concrete3, Y_train_concrete3, Y_val_concrete3, y_test_concrete3)
X_train_energy1, X_val_energy1, x_test_energy1, X_scale_train_energy1=standard_scaler(X_train_energy1, X_val_energy1, x_test_energy1)
X_train_energy2, X_val_energy2, x_test_energy2, X_scale_train_energy2=standard_scaler(X_train_energy2, X_val_energy2, x_test_energy2)
X_train_fertility, X_val_fertility, x_test_fertility, X_scale_train_fertility=standard_scaler(X_train_fertility, X_val_fertility, x_test_fertility)
X_train_forest, X_val_forest, x_test_forest, Y_train_forest, Y_val_forest, y_test_forest, min_max_forest=min_max_scaler(X_train_forest, X_val_forest, x_test_forest, Y_train_forest, Y_val_forest, y_test_forest)
X_train_housing, X_val_housing, x_test_housing, X_scale_train_housing=standard_scaler(X_train_housing, X_val_housing, x_test_housing)
X_train_istanbul, X_val_istanbul, x_test_istanbul, X_scale_train_istanbul=standard_scaler(X_train_istanbul, X_val_istanbul, x_test_istanbul)

In [26]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_1(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_2 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6         
Total params: 66
Trainable params: 66
Non-trainable params: 0
_________________________________________________________________

Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 419us/step - lo

1052/1052 [==============================] - 0s 93us/step - loss: 22.1229 - val_loss: 25.5136
Epoch 66/1000
1052/1052 [==============================] - 0s 95us/step - loss: 21.9177 - val_loss: 25.8949
Epoch 67/1000
1052/1052 [==============================] - 0s 101us/step - loss: 22.1790 - val_loss: 26.2936
Epoch 68/1000
1052/1052 [==============================] - 0s 94us/step - loss: 22.2154 - val_loss: 25.6436
Epoch 69/1000
1052/1052 [==============================] - 0s 96us/step - loss: 22.2645 - val_loss: 25.4360
Epoch 70/1000
1052/1052 [==============================] - 0s 92us/step - loss: 22.2862 - val_loss: 26.0401
Epoch 71/1000
1052/1052 [==============================] - 0s 116us/step - loss: 21.9746 - val_loss: 27.5421
Epoch 72/1000
1052/1052 [==============================] - 0s 85us/step - loss: 22.1346 - val_loss: 26.1844
Epoch 73/1000
1052/1052 [==============================] - 0s 94us/step - loss: 22.2426 - val_loss: 25.2986
Epoch 74/1000
1052/1052 [===============

1052/1052 [==============================] - 0s 99us/step - loss: 14.8509 - val_loss: 17.3820
Epoch 216/1000
1052/1052 [==============================] - 0s 89us/step - loss: 15.2031 - val_loss: 16.9816
Epoch 217/1000
1052/1052 [==============================] - 0s 87us/step - loss: 14.8433 - val_loss: 17.1706
Epoch 218/1000
1052/1052 [==============================] - 0s 88us/step - loss: 14.6605 - val_loss: 16.9637
Epoch 219/1000
1052/1052 [==============================] - 0s 91us/step - loss: 14.7953 - val_loss: 17.3168
Epoch 220/1000
1052/1052 [==============================] - 0s 87us/step - loss: 14.2728 - val_loss: 16.8309
Epoch 221/1000
1052/1052 [==============================] - 0s 84us/step - loss: 15.0836 - val_loss: 18.4755
Epoch 222/1000
1052/1052 [==============================] - 0s 87us/step - loss: 14.5885 - val_loss: 16.9238
Epoch 223/1000
1052/1052 [==============================] - 0s 82us/step - loss: 14.6299 - val_loss: 17.2659
Epoch 224/1000
1052/1052 [========

1052/1052 [==============================] - 0s 103us/step - loss: 9.4712 - val_loss: 11.1712
Epoch 366/1000
1052/1052 [==============================] - 0s 100us/step - loss: 9.4239 - val_loss: 12.0570
Epoch 367/1000
1052/1052 [==============================] - 0s 84us/step - loss: 9.7076 - val_loss: 12.0299
Epoch 368/1000
1052/1052 [==============================] - 0s 84us/step - loss: 9.5063 - val_loss: 13.1685
Epoch 369/1000
1052/1052 [==============================] - 0s 93us/step - loss: 9.5736 - val_loss: 11.5382
Epoch 370/1000
1052/1052 [==============================] - 0s 98us/step - loss: 9.2235 - val_loss: 11.2146
Epoch 371/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.9830 - val_loss: 11.7677
Epoch 372/1000
1052/1052 [==============================] - 0s 99us/step - loss: 9.2131 - val_loss: 11.5110
Epoch 373/1000
1052/1052 [==============================] - 0s 84us/step - loss: 9.2530 - val_loss: 11.0122
Epoch 374/1000
1052/1052 [===============

Epoch 441/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.8401 - val_loss: 11.1459
Epoch 442/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.3719 - val_loss: 10.9473
Epoch 443/1000
1052/1052 [==============================] - 0s 93us/step - loss: 9.6838 - val_loss: 11.0747
Epoch 444/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.1661 - val_loss: 10.9504
Epoch 445/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.2901 - val_loss: 11.0711
Epoch 446/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.0314 - val_loss: 11.4100
Epoch 447/1000
1052/1052 [==============================] - 0s 84us/step - loss: 9.1307 - val_loss: 12.6444
Epoch 448/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.9892 - val_loss: 10.8070
Epoch 449/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.9567 - val_loss: 12.1715
Epoch 450/1000
1052/1052 [==

...

Epoch 745/1000
1052/1052 [==============================] - 0s 82us/step - loss: 8.9118 - val_loss: 11.2383
Epoch 746/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.7231 - val_loss: 10.9058
Epoch 747/1000
1052/1052 [==============================] - 0s 82us/step - loss: 8.7342 - val_loss: 11.1041
Epoch 748/1000
1052/1052 [==============================] - 0s 81us/step - loss: 8.7420 - val_loss: 10.2886
Epoch 749/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.9901 - val_loss: 10.1696
Epoch 750/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.7856 - val_loss: 10.8525
Epoch 751/1000
1052/1052 [==============================] - 0s 81us/step - loss: 8.7915 - val_loss: 10.3766
Epoch 752/1000
1052/1052 [==============================] - 0s 76us/step - loss: 8.7411 - val_loss: 10.3195
Epoch 753/1000
1052/1052 [==============================] - 0s 81us/step - loss: 8.7028 - val_loss: 10.4013
Epoch 754/1000
1052/1052 [==

Epoch 821/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.7051 - val_loss: 10.4814
Epoch 822/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.9483 - val_loss: 10.0869
Epoch 823/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.6410 - val_loss: 10.2196
Epoch 824/1000
1052/1052 [==============================] - 0s 83us/step - loss: 8.7992 - val_loss: 10.3507
Epoch 825/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.9446 - val_loss: 10.4689
Epoch 826/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.7712 - val_loss: 10.8250
Epoch 827/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.6661 - val_loss: 10.8679
Epoch 828/1000
1052/1052 [==============================] - 0s 84us/step - loss: 9.2552 - val_loss: 10.4111
Epoch 829/1000
1052/1052 [==============================] - 0s 79us/step - loss: 8.6238 - val_loss: 10.3968
Epoch 830/1000
1052/1052 [==

Epoch 897/1000
1052/1052 [==============================] - 0s 82us/step - loss: 8.7561 - val_loss: 10.3538
Epoch 898/1000
1052/1052 [==============================] - 0s 84us/step - loss: 9.2805 - val_loss: 10.8510
Epoch 899/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.7006 - val_loss: 10.1282
Epoch 900/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.5204 - val_loss: 11.0204
Epoch 901/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.5293 - val_loss: 10.8083
Epoch 902/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.1649 - val_loss: 10.3947
Epoch 903/1000
1052/1052 [==============================] - 0s 83us/step - loss: 8.5897 - val_loss: 10.8829
Epoch 904/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.6897 - val_loss: 10.7956
Epoch 905/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.0491 - val_loss: 11.0319
Epoch 906/1000
1052/1052 [==

Epoch 973/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.5619 - val_loss: 9.8434
Epoch 974/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.6203 - val_loss: 10.2479
Epoch 975/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.8904 - val_loss: 10.0805
Epoch 976/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.6399 - val_loss: 10.1060
Epoch 977/1000
1052/1052 [==============================] - 0s 83us/step - loss: 9.0197 - val_loss: 10.9173
Epoch 978/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.8023 - val_loss: 10.0754
Epoch 979/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.9734 - val_loss: 10.4716
Epoch 980/1000
1052/1052 [==============================] - 0s 83us/step - loss: 8.6428 - val_loss: 10.2365
Epoch 981/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.6078 - val_loss: 10.4844
Epoch 982/1000
1052/1052 [===

[array([[-4.0897183 ,  0.20294963,  1.4910512 ,  1.1518444 , -3.9177892 ],
        [ 0.36034897,  0.26312467, -0.33617806,  0.11392819,  0.07633582],
        [-0.656961  ,  0.37469736, -0.25279558,  0.3490771 ,  0.07739425],
        [ 0.146966  , -0.09817263, -0.09108508, -0.1260312 ,  0.08331141],
        [-0.42905676,  0.21014701,  0.16703925,  0.9846548 , -2.1844993 ]],
       dtype=float32),
 array([-5.0597167 ,  0.23230127,  1.2123454 ,  2.913998  , -5.3612175 ],
       dtype=float32),
 array([[ 2.842912  ,  2.2142937 ,  3.2297137 , -3.3318956 ,  2.2851353 ],
        [ 0.6016961 ,  1.8716263 ,  1.8844707 , -0.60427463,  1.9031094 ],
        [ 0.73900616,  1.1941285 ,  1.0457095 , -1.613533  ,  0.8235355 ],
        [-2.309707  , -2.833071  , -2.4372993 ,  2.7908041 , -2.1663594 ],
        [ 2.413546  ,  2.870484  ,  2.255688  , -2.582084  ,  2.9827347 ]],
       dtype=float32),
 array([-2.3812776, -2.3147817, -2.2919624,  2.3475919, -2.2427897],
       dtype=float32),
 array([[-2.5

In [27]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure1_2nd.h5')

In [28]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_2(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_3 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_4 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 195us/step - l

1052/1052 [==============================] - 0s 88us/step - loss: 21.8269 - val_loss: 26.0124
Epoch 68/1000
1052/1052 [==============================] - 0s 100us/step - loss: 21.8379 - val_loss: 25.7805
Epoch 69/1000
1052/1052 [==============================] - 0s 90us/step - loss: 22.1476 - val_loss: 25.4008
Epoch 70/1000
1052/1052 [==============================] - 0s 84us/step - loss: 21.9497 - val_loss: 25.9883
Epoch 71/1000
1052/1052 [==============================] - 0s 84us/step - loss: 21.6565 - val_loss: 25.8593
Epoch 72/1000
1052/1052 [==============================] - 0s 82us/step - loss: 21.7302 - val_loss: 25.8332
Epoch 73/1000
1052/1052 [==============================] - 0s 83us/step - loss: 22.2131 - val_loss: 26.6225
Epoch 74/1000
1052/1052 [==============================] - 0s 83us/step - loss: 22.0987 - val_loss: 26.4060
Epoch 75/1000
1052/1052 [==============================] - 0s 86us/step - loss: 22.2957 - val_loss: 25.6400
Epoch 76/1000
1052/1052 [================

1052/1052 [==============================] - 0s 85us/step - loss: 12.0751 - val_loss: 15.5505
Epoch 218/1000
1052/1052 [==============================] - 0s 85us/step - loss: 11.3729 - val_loss: 14.2240
Epoch 219/1000
1052/1052 [==============================] - 0s 82us/step - loss: 11.5252 - val_loss: 14.5372
Epoch 220/1000
1052/1052 [==============================] - 0s 90us/step - loss: 11.6619 - val_loss: 18.3387
Epoch 221/1000
1052/1052 [==============================] - 0s 87us/step - loss: 11.6703 - val_loss: 13.7289
Epoch 222/1000
1052/1052 [==============================] - 0s 89us/step - loss: 11.3221 - val_loss: 13.2236
Epoch 223/1000
1052/1052 [==============================] - 0s 88us/step - loss: 11.4247 - val_loss: 13.9834
Epoch 224/1000
1052/1052 [==============================] - 0s 82us/step - loss: 11.5356 - val_loss: 13.6898
Epoch 225/1000
1052/1052 [==============================] - 0s 84us/step - loss: 11.4418 - val_loss: 14.4441
Epoch 226/1000
1052/1052 [========

Epoch 368/1000
1052/1052 [==============================] - 0s 85us/step - loss: 9.2842 - val_loss: 10.8414
Epoch 369/1000
1052/1052 [==============================] - 0s 99us/step - loss: 10.0038 - val_loss: 11.7986
Epoch 370/1000
1052/1052 [==============================] - 0s 98us/step - loss: 9.3805 - val_loss: 11.3898
Epoch 371/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.2109 - val_loss: 11.6621
Epoch 372/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.8163 - val_loss: 10.9957
Epoch 373/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.7730 - val_loss: 14.0300
Epoch 374/1000
1052/1052 [==============================] - 0s 100us/step - loss: 9.3655 - val_loss: 11.5196
Epoch 375/1000
1052/1052 [==============================] - 0s 96us/step - loss: 9.2083 - val_loss: 12.7512
Epoch 376/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.2124 - val_loss: 11.1187
Epoch 377/1000
1052/1052 [

Epoch 519/1000
1052/1052 [==============================] - 0s 85us/step - loss: 9.1962 - val_loss: 11.4973
Epoch 520/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.5557 - val_loss: 10.9005
Epoch 521/1000
1052/1052 [==============================] - 0s 97us/step - loss: 8.7361 - val_loss: 10.5004
Epoch 522/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.9053 - val_loss: 11.5839
Epoch 523/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.5465 - val_loss: 10.9866
Epoch 524/1000
1052/1052 [==============================] - 0s 83us/step - loss: 9.2099 - val_loss: 10.4983
Epoch 525/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.4344 - val_loss: 10.6123
Epoch 526/1000
1052/1052 [==============================] - 0s 93us/step - loss: 9.6271 - val_loss: 14.2579
Epoch 527/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.2974 - val_loss: 11.5645
Epoch 528/1000
1052/1052 [==

...

Epoch 747/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.5250 - val_loss: 11.0209
Epoch 748/1000
1052/1052 [==============================] - 0s 81us/step - loss: 8.7594 - val_loss: 11.0841
Epoch 749/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.8023 - val_loss: 10.7035
Epoch 750/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.7333 - val_loss: 11.4654
Epoch 751/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.1514 - val_loss: 10.5225
Epoch 752/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.9234 - val_loss: 10.6048
Epoch 753/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.3074 - val_loss: 13.2049
Epoch 754/1000
1052/1052 [==============================] - 0s 84us/step - loss: 9.7303 - val_loss: 10.5584
Epoch 755/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.8465 - val_loss: 11.2586
Epoch 756/1000
1052/1052 [==

Epoch 823/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.1908 - val_loss: 11.8977
Epoch 824/1000
1052/1052 [==============================] - 0s 85us/step - loss: 9.5823 - val_loss: 10.8613
Epoch 825/1000
1052/1052 [==============================] - 0s 84us/step - loss: 9.2744 - val_loss: 10.1276
Epoch 826/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.7701 - val_loss: 9.9519
Epoch 827/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.6978 - val_loss: 10.2053
Epoch 828/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.7077 - val_loss: 10.3473
Epoch 829/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.2314 - val_loss: 11.2218
Epoch 830/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.9654 - val_loss: 10.7237
Epoch 831/1000
1052/1052 [==============================] - 0s 83us/step - loss: 8.6111 - val_loss: 11.1061
Epoch 832/1000
1052/1052 [===

Epoch 899/1000
1052/1052 [==============================] - 0s 80us/step - loss: 8.6730 - val_loss: 9.7149
Epoch 900/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.6039 - val_loss: 10.2477
Epoch 901/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.8815 - val_loss: 10.0389
Epoch 902/1000
1052/1052 [==============================] - 0s 83us/step - loss: 8.5184 - val_loss: 10.6394
Epoch 903/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.7649 - val_loss: 10.2736
Epoch 904/1000
1052/1052 [==============================] - 0s 83us/step - loss: 8.9266 - val_loss: 10.5641
Epoch 905/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.9230 - val_loss: 10.9645
Epoch 906/1000
1052/1052 [==============================] - 0s 84us/step - loss: 9.0474 - val_loss: 11.0230
Epoch 907/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.2330 - val_loss: 11.6175
Epoch 908/1000
1052/1052 [===

1052/1052 [==============================] - 0s 86us/step - loss: 8.5234 - val_loss: 10.8772
Epoch 976/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.8712 - val_loss: 11.7992
Epoch 977/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.3013 - val_loss: 10.0362
Epoch 978/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.1607 - val_loss: 10.3723
Epoch 979/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.6772 - val_loss: 10.4805
Epoch 980/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.6524 - val_loss: 11.3664
Epoch 981/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.7131 - val_loss: 10.2985
Epoch 982/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.6030 - val_loss: 9.9162
Epoch 983/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.0983 - val_loss: 10.1898
Epoch 984/1000
1052/1052 [==================

[array([[-4.1505246 ,  0.15476568,  4.1660204 ,  1.3145167 , -1.4364762 ],
        [ 0.3391187 ,  0.25200796, -0.06585328,  0.115622  ,  0.3018943 ],
        [-0.707991  ,  0.36330968, -0.10965822,  0.35833016,  0.2306936 ],
        [ 0.17200775, -0.0794817 , -0.08613301, -0.14420195,  0.05515834],
        [-0.40839565,  0.2017554 ,  2.2008257 ,  0.968707  , -0.16303156]],
       dtype=float32),
 array([-5.1657286 ,  0.29125994,  5.534727  ,  2.6142447 , -1.1628367 ],
       dtype=float32),
 array([[-1.0954572 ,  1.8518851 , -1.8723832 , -2.4609966 ,  1.891905  ,
         -1.953476  ,  2.224686  ,  2.453822  ,  1.60024   ,  1.8109215 ],
        [-1.3287536 ,  1.0080909 , -1.3975655 , -1.2597661 ,  0.62724495,
         -1.3384815 ,  0.7788654 ,  0.647432  ,  0.6210419 ,  1.3702668 ],
        [ 0.8276924 , -1.39233   ,  2.1468153 ,  1.6171973 , -1.8247293 ,
          1.935787  , -1.4345741 , -2.1272771 , -2.1461022 , -1.7483677 ],
        [ 0.50650924, -1.1015131 ,  1.5892018 ,  1.095573

In [29]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure2_2nd.h5')

In [30]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_3(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_5 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 15)                90        
_________________________________________________________________
activation_6 (Activation)    (None, 15)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 16        
Total params: 136
Trainable params: 136
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 212us/step - l

1052/1052 [==============================] - 0s 99us/step - loss: 15.1999 - val_loss: 19.2733
Epoch 68/1000
1052/1052 [==============================] - 0s 106us/step - loss: 15.4067 - val_loss: 19.0220
Epoch 69/1000
1052/1052 [==============================] - 0s 95us/step - loss: 15.4535 - val_loss: 18.4699
Epoch 70/1000
1052/1052 [==============================] - 0s 95us/step - loss: 15.3069 - val_loss: 18.5459
Epoch 71/1000
1052/1052 [==============================] - 0s 90us/step - loss: 15.0496 - val_loss: 19.2589
Epoch 72/1000
1052/1052 [==============================] - 0s 98us/step - loss: 15.1918 - val_loss: 19.6996
Epoch 73/1000
1052/1052 [==============================] - 0s 98us/step - loss: 15.1390 - val_loss: 18.7794
Epoch 74/1000
1052/1052 [==============================] - 0s 90us/step - loss: 15.0921 - val_loss: 17.9556
Epoch 75/1000
1052/1052 [==============================] - 0s 89us/step - loss: 15.0470 - val_loss: 19.3717
Epoch 76/1000
1052/1052 [================

1052/1052 [==============================] - 0s 87us/step - loss: 8.9995 - val_loss: 11.7478
Epoch 218/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.4595 - val_loss: 11.5144
Epoch 219/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.8039 - val_loss: 10.8337
Epoch 220/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.2392 - val_loss: 11.4192
Epoch 221/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.3212 - val_loss: 10.6455
Epoch 222/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.8058 - val_loss: 11.6072
Epoch 223/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.8115 - val_loss: 11.9838
Epoch 224/1000
1052/1052 [==============================] - 0s 93us/step - loss: 9.2250 - val_loss: 10.8308
Epoch 225/1000
1052/1052 [==============================] - 0s 109us/step - loss: 8.7295 - val_loss: 11.5613
Epoch 226/1000
1052/1052 [================

Epoch 368/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.8485 - val_loss: 9.5505
Epoch 369/1000
1052/1052 [==============================] - 0s 97us/step - loss: 8.6949 - val_loss: 11.0238
Epoch 370/1000
1052/1052 [==============================] - 0s 104us/step - loss: 9.1732 - val_loss: 10.6767
Epoch 371/1000
1052/1052 [==============================] - 0s 101us/step - loss: 8.7308 - val_loss: 10.0783
Epoch 372/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.4544 - val_loss: 10.6555
Epoch 373/1000
1052/1052 [==============================] - 0s 96us/step - loss: 8.4745 - val_loss: 11.4818
Epoch 374/1000
1052/1052 [==============================] - 0s 95us/step - loss: 8.6548 - val_loss: 12.1804
Epoch 375/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.4994 - val_loss: 10.5669
Epoch 376/1000
1052/1052 [==============================] - 0s 97us/step - loss: 8.6503 - val_loss: 10.6844
Epoch 377/1000
1052/1052 [=

Epoch 444/1000
1052/1052 [==============================] - 0s 95us/step - loss: 8.1629 - val_loss: 9.4447
Epoch 445/1000
1052/1052 [==============================] - 0s 98us/step - loss: 8.1337 - val_loss: 10.6800
Epoch 446/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.6559 - val_loss: 9.7492
Epoch 447/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.5649 - val_loss: 10.5463
Epoch 448/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.3545 - val_loss: 10.1091
Epoch 449/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.7244 - val_loss: 9.6219
Epoch 450/1000
1052/1052 [==============================] - 0s 96us/step - loss: 8.5358 - val_loss: 9.3921
Epoch 451/1000
1052/1052 [==============================] - 0s 96us/step - loss: 8.5051 - val_loss: 9.7637
Epoch 452/1000
1052/1052 [==============================] - 0s 101us/step - loss: 8.4037 - val_loss: 10.0208
Epoch 453/1000
1052/1052 [======

Epoch 520/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.0971 - val_loss: 10.4467
Epoch 521/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.9850 - val_loss: 10.6194
Epoch 522/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.6496 - val_loss: 10.2920
Epoch 523/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.7724 - val_loss: 10.7903
Epoch 524/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.0135 - val_loss: 9.7904
Epoch 525/1000
1052/1052 [==============================] - 0s 83us/step - loss: 8.3362 - val_loss: 9.6644
Epoch 526/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.1666 - val_loss: 9.2770
Epoch 527/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.7649 - val_loss: 11.6987
Epoch 528/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.6591 - val_loss: 9.3214
Epoch 529/1000
1052/1052 [======

1052/1052 [==============================] - 0s 87us/step - loss: 8.1404 - val_loss: 8.8220
Epoch 673/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.4090 - val_loss: 9.2097
Epoch 674/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.0614 - val_loss: 8.8011
Epoch 675/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.3261 - val_loss: 9.3333
Epoch 676/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.0478 - val_loss: 9.0199
Epoch 677/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.2980 - val_loss: 9.1635
Epoch 678/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.9266 - val_loss: 8.8138
Epoch 679/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.1140 - val_loss: 8.9283
Epoch 680/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.0922 - val_loss: 9.2325
Epoch 681/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 83us/step - loss: 8.2721 - val_loss: 9.0789
Epoch 825/1000
1052/1052 [==============================] - 0s 88us/step - loss: 7.8888 - val_loss: 9.0883
Epoch 826/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.1332 - val_loss: 9.0228
Epoch 827/1000
1052/1052 [==============================] - 0s 88us/step - loss: 7.8449 - val_loss: 9.4141
Epoch 828/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.1363 - val_loss: 8.9675
Epoch 829/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.0437 - val_loss: 9.9646
Epoch 830/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.8997 - val_loss: 9.7236
Epoch 831/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.1689 - val_loss: 9.0821
Epoch 832/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.1240 - val_loss: 11.4596
Epoch 833/1000
1052/1052 [=========================

1052/1052 [==============================] - 0s 83us/step - loss: 7.9079 - val_loss: 8.6204
Epoch 977/1000
1052/1052 [==============================] - 0s 83us/step - loss: 8.1793 - val_loss: 9.8702
Epoch 978/1000
1052/1052 [==============================] - 0s 89us/step - loss: 7.8835 - val_loss: 8.8679
Epoch 979/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.8280 - val_loss: 8.5605
Epoch 980/1000
1052/1052 [==============================] - 0s 83us/step - loss: 8.0170 - val_loss: 9.7143
Epoch 981/1000
1052/1052 [==============================] - 0s 89us/step - loss: 7.8678 - val_loss: 9.0436
Epoch 982/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.9869 - val_loss: 9.7396
Epoch 983/1000
1052/1052 [==============================] - 0s 93us/step - loss: 7.8722 - val_loss: 8.9448
Epoch 984/1000
1052/1052 [==============================] - 0s 89us/step - loss: 7.9049 - val_loss: 8.8083
Epoch 985/1000
1052/1052 [==========================

[array([[ 0.85346943, -2.1789339 ,  0.29538095, -3.530977  ,  3.3994591 ],
        [ 1.9697311 ,  0.4923424 ,  0.1959119 ,  0.1482339 , -0.0514848 ],
        [ 2.4308016 ,  0.36377305,  0.42316145, -0.8372641 , -0.00640358],
        [-0.41652805,  0.01712474, -0.18393423,  0.14243858, -0.05126983],
        [-0.89154863, -0.29412702,  0.35875502, -0.27712104,  2.2884166 ]],
       dtype=float32),
 array([ 1.1832584, -1.3751085, -0.6532213, -4.5993385,  4.91756  ],
       dtype=float32),
 array([[-0.2402981 ,  0.352329  , -0.30730477, -0.19603644,  0.53829145,
          0.13814636,  0.6548321 ,  0.629318  , -0.05803755, -0.40725568,
         -0.5868875 , -0.7142429 ,  0.2383918 ,  0.17176844,  0.10203674],
        [ 0.0682101 , -0.7753987 ,  0.07264737, -0.772795  , -0.8039052 ,
         -0.08553825, -0.5229733 , -0.3350849 ,  0.12042087,  0.48839143,
          0.8573923 ,  0.7019924 ,  0.07577219,  0.44396344,  0.66432583],
        [ 0.9489872 ,  0.6535794 ,  0.54525787,  0.6971635 ,  0

In [31]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure3_2nd.h5')

In [32]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_4(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_7 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 55        
_________________________________________________________________
activation_8 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 6         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 224us/step - l

1052/1052 [==============================] - 0s 84us/step - loss: 13.5054 - val_loss: 16.8570
Epoch 68/1000
1052/1052 [==============================] - 0s 90us/step - loss: 13.3930 - val_loss: 17.8513
Epoch 69/1000
1052/1052 [==============================] - 0s 88us/step - loss: 13.9452 - val_loss: 16.6790
Epoch 70/1000
1052/1052 [==============================] - 0s 87us/step - loss: 13.3012 - val_loss: 16.5872
Epoch 71/1000
1052/1052 [==============================] - 0s 86us/step - loss: 13.1164 - val_loss: 16.6363
Epoch 72/1000
1052/1052 [==============================] - 0s 88us/step - loss: 13.0655 - val_loss: 16.6278
Epoch 73/1000
1052/1052 [==============================] - 0s 85us/step - loss: 12.9508 - val_loss: 17.0794
Epoch 74/1000
1052/1052 [==============================] - 0s 87us/step - loss: 13.2706 - val_loss: 16.4417
Epoch 75/1000
1052/1052 [==============================] - 0s 86us/step - loss: 12.8294 - val_loss: 17.5029
Epoch 76/1000
1052/1052 [=================

Epoch 218/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.4365 - val_loss: 10.0573
Epoch 219/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.1404 - val_loss: 11.2669
Epoch 220/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.8227 - val_loss: 9.1628
Epoch 221/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.3187 - val_loss: 9.5693
Epoch 222/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.7310 - val_loss: 11.1434
Epoch 223/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.6465 - val_loss: 10.1012
Epoch 224/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.1574 - val_loss: 10.0884
Epoch 225/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.4035 - val_loss: 10.2130
Epoch 226/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.3878 - val_loss: 9.6406
Epoch 227/1000
1052/1052 [=====

1052/1052 [==============================] - 0s 85us/step - loss: 8.3102 - val_loss: 8.5600
Epoch 371/1000
1052/1052 [==============================] - 0s 88us/step - loss: 7.6140 - val_loss: 9.0143
Epoch 372/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.0569 - val_loss: 9.2112
Epoch 373/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.9059 - val_loss: 9.1115
Epoch 374/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.8310 - val_loss: 8.9175
Epoch 375/1000
1052/1052 [==============================] - 0s 88us/step - loss: 7.7812 - val_loss: 8.7262
Epoch 376/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.1701 - val_loss: 8.4176
Epoch 377/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.0569 - val_loss: 8.8111
Epoch 378/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.7295 - val_loss: 8.9212
Epoch 379/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 91us/step - loss: 8.0182 - val_loss: 10.3083
Epoch 523/1000
1052/1052 [==============================] - 0s 88us/step - loss: 7.9381 - val_loss: 8.6356
Epoch 524/1000
1052/1052 [==============================] - 0s 88us/step - loss: 7.5751 - val_loss: 9.2195
Epoch 525/1000
1052/1052 [==============================] - 0s 90us/step - loss: 7.7897 - val_loss: 8.8694
Epoch 526/1000
1052/1052 [==============================] - 0s 84us/step - loss: 7.7531 - val_loss: 9.1554
Epoch 527/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.6447 - val_loss: 8.7868
Epoch 528/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.5233 - val_loss: 8.6801
Epoch 529/1000
1052/1052 [==============================] - 0s 82us/step - loss: 7.8532 - val_loss: 8.5991
Epoch 530/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.6578 - val_loss: 8.2822
Epoch 531/1000
1052/1052 [=========================

1052/1052 [==============================] - 0s 88us/step - loss: 7.4965 - val_loss: 9.1613
Epoch 675/1000
1052/1052 [==============================] - 0s 83us/step - loss: 7.7794 - val_loss: 8.4109
Epoch 676/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.4787 - val_loss: 8.1821
Epoch 677/1000
1052/1052 [==============================] - 0s 84us/step - loss: 7.5652 - val_loss: 7.8994
Epoch 678/1000
1052/1052 [==============================] - 0s 88us/step - loss: 7.2800 - val_loss: 8.2816
Epoch 679/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.1966 - val_loss: 8.3514
Epoch 680/1000
1052/1052 [==============================] - 0s 88us/step - loss: 7.3769 - val_loss: 8.4968
Epoch 681/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.5687 - val_loss: 8.2313
Epoch 682/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.4443 - val_loss: 8.8426
Epoch 683/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 91us/step - loss: 7.0096 - val_loss: 8.0488
Epoch 827/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.0498 - val_loss: 8.0574
Epoch 828/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.1615 - val_loss: 8.2156
Epoch 829/1000
1052/1052 [==============================] - 0s 88us/step - loss: 7.6470 - val_loss: 8.1431
Epoch 830/1000
1052/1052 [==============================] - 0s 88us/step - loss: 7.3522 - val_loss: 8.1068
Epoch 831/1000
1052/1052 [==============================] - 0s 106us/step - loss: 7.7921 - val_loss: 10.3639
Epoch 832/1000
1052/1052 [==============================] - 0s 95us/step - loss: 7.6157 - val_loss: 8.4087
Epoch 833/1000
1052/1052 [==============================] - 0s 89us/step - loss: 7.3105 - val_loss: 7.6420
Epoch 834/1000
1052/1052 [==============================] - 0s 88us/step - loss: 7.1729 - val_loss: 9.6059
Epoch 835/1000
1052/1052 [========================

1052/1052 [==============================] - 0s 89us/step - loss: 7.4464 - val_loss: 9.5374
Epoch 979/1000
1052/1052 [==============================] - 0s 87us/step - loss: 6.9621 - val_loss: 8.1684
Epoch 980/1000
1052/1052 [==============================] - 0s 84us/step - loss: 7.8263 - val_loss: 7.6371
Epoch 981/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.3486 - val_loss: 8.3846
Epoch 982/1000
1052/1052 [==============================] - 0s 83us/step - loss: 7.2951 - val_loss: 8.2246
Epoch 983/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.0111 - val_loss: 7.5610
Epoch 984/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.1851 - val_loss: 8.9556
Epoch 985/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.3241 - val_loss: 9.0312
Epoch 986/1000
1052/1052 [==============================] - 0s 84us/step - loss: 7.3886 - val_loss: 8.2903
Epoch 987/1000
1052/1052 [==========================

[array([[-2.95092314e-01,  5.07176638e-01,  4.88057089e+00,
          3.17043990e-01,  5.64508557e-01, -5.20645916e-01,
         -2.37642139e-01, -5.21843863e+00,  1.69647074e+00,
          4.30521393e+00],
        [-5.23071826e-01,  1.20774055e+00, -2.28234336e-01,
          6.53233171e-01,  8.08367431e-01, -3.99232656e-01,
         -1.86323315e-01, -1.98227808e-01, -1.56775042e-01,
         -4.23849560e-02],
        [-7.56681502e-01,  1.82341349e+00,  1.34337139e+00,
          9.32926595e-01,  1.59977758e+00, -6.90866172e-01,
         -4.01334971e-01,  6.25746250e-02, -6.56992868e-02,
          1.29846871e+00],
        [ 1.08904406e-01,  3.92923951e-01, -1.86798647e-01,
         -2.51446175e-03, -2.28729650e-01,  2.99601853e-01,
          9.71929610e-01,  6.57775328e-02, -7.93439448e-02,
         -2.46725023e-01],
        [-3.85078758e-01,  6.34336650e-01,  2.53758520e-01,
          4.28221017e-01,  9.46744323e-01, -5.07243752e-01,
         -4.08247232e-01, -1.92404592e+00,  1.236561

In [33]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure4_2nd.h5')

In [34]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_5(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_9 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 10)                110       
_________________________________________________________________
activation_10 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 11        
Total params: 181
Trainable params: 181
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 242us/step - l

1052/1052 [==============================] - 0s 90us/step - loss: 14.6882 - val_loss: 17.1268
Epoch 68/1000
1052/1052 [==============================] - 0s 86us/step - loss: 14.1954 - val_loss: 19.8190
Epoch 69/1000
1052/1052 [==============================] - 0s 86us/step - loss: 14.5990 - val_loss: 18.4403
Epoch 70/1000
1052/1052 [==============================] - 0s 87us/step - loss: 14.2316 - val_loss: 17.4480
Epoch 71/1000
1052/1052 [==============================] - 0s 79us/step - loss: 14.7540 - val_loss: 17.2663
Epoch 72/1000
1052/1052 [==============================] - 0s 85us/step - loss: 14.1956 - val_loss: 19.0268
Epoch 73/1000
1052/1052 [==============================] - 0s 84us/step - loss: 14.8561 - val_loss: 17.5094
Epoch 74/1000
1052/1052 [==============================] - 0s 87us/step - loss: 14.3251 - val_loss: 17.2204
Epoch 75/1000
1052/1052 [==============================] - 0s 88us/step - loss: 14.1247 - val_loss: 17.1757
Epoch 76/1000
1052/1052 [=================

Epoch 218/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.6467 - val_loss: 10.4682
Epoch 219/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.1566 - val_loss: 10.5164
Epoch 220/1000
1052/1052 [==============================] - 0s 81us/step - loss: 9.1741 - val_loss: 13.7055
Epoch 221/1000
1052/1052 [==============================] - 0s 82us/step - loss: 8.9927 - val_loss: 10.7527
Epoch 222/1000
1052/1052 [==============================] - 0s 82us/step - loss: 8.8033 - val_loss: 11.0874
Epoch 223/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.9085 - val_loss: 10.3363
Epoch 224/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.7654 - val_loss: 12.3299
Epoch 225/1000
1052/1052 [==============================] - 0s 85us/step - loss: 9.7186 - val_loss: 12.0738
Epoch 226/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.8344 - val_loss: 10.1442
Epoch 227/1000
1052/1052 [==

Epoch 294/1000
1052/1052 [==============================] - 0s 97us/step - loss: 8.3164 - val_loss: 9.6853
Epoch 295/1000
1052/1052 [==============================] - 0s 95us/step - loss: 8.2527 - val_loss: 10.0372
Epoch 296/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.3704 - val_loss: 9.4903
Epoch 297/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.9432 - val_loss: 12.1776
Epoch 298/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.3324 - val_loss: 10.4272
Epoch 299/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.5301 - val_loss: 10.2670
Epoch 300/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.2911 - val_loss: 9.5914
Epoch 301/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.9872 - val_loss: 11.5682
Epoch 302/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.7659 - val_loss: 9.7922
Epoch 303/1000
1052/1052 [======

1052/1052 [==============================] - 0s 89us/step - loss: 8.2936 - val_loss: 9.1227
Epoch 447/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.3172 - val_loss: 10.9174
Epoch 448/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.6328 - val_loss: 14.5892
Epoch 449/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.7161 - val_loss: 9.7737
Epoch 450/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.0020 - val_loss: 10.5626
Epoch 451/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.3113 - val_loss: 10.1466
Epoch 452/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.1490 - val_loss: 10.3467
Epoch 453/1000
1052/1052 [==============================] - 0s 100us/step - loss: 8.3741 - val_loss: 11.0351
Epoch 454/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.6147 - val_loss: 9.5385
Epoch 455/1000
1052/1052 [===================

1052/1052 [==============================] - 0s 84us/step - loss: 8.5513 - val_loss: 10.5764
Epoch 599/1000
1052/1052 [==============================] - 0s 83us/step - loss: 8.1577 - val_loss: 9.1485
Epoch 600/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.0674 - val_loss: 9.6820
Epoch 601/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.1540 - val_loss: 8.9895
Epoch 602/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.0255 - val_loss: 10.2112
Epoch 603/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.1054 - val_loss: 9.9564
Epoch 604/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.9688 - val_loss: 9.1547
Epoch 605/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.9262 - val_loss: 9.0148
Epoch 606/1000
1052/1052 [==============================] - 0s 79us/step - loss: 8.1737 - val_loss: 9.3401
Epoch 607/1000
1052/1052 [========================

1052/1052 [==============================] - 0s 97us/step - loss: 8.5238 - val_loss: 9.4986
Epoch 751/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.7196 - val_loss: 10.5923
Epoch 752/1000
1052/1052 [==============================] - 0s 95us/step - loss: 8.5413 - val_loss: 10.6458
Epoch 753/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.2284 - val_loss: 9.0992
Epoch 754/1000
1052/1052 [==============================] - 0s 101us/step - loss: 8.2463 - val_loss: 8.9824
Epoch 755/1000
1052/1052 [==============================] - 0s 89us/step - loss: 7.9751 - val_loss: 9.0927
Epoch 756/1000
1052/1052 [==============================] - 0s 97us/step - loss: 8.0558 - val_loss: 8.8992
Epoch 757/1000
1052/1052 [==============================] - 0s 105us/step - loss: 7.8054 - val_loss: 9.6242
Epoch 758/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.1007 - val_loss: 8.9457
Epoch 759/1000
1052/1052 [======================

1052/1052 [==============================] - 0s 84us/step - loss: 7.8623 - val_loss: 8.7199
Epoch 903/1000
1052/1052 [==============================] - 0s 80us/step - loss: 7.9739 - val_loss: 8.7850
Epoch 904/1000
1052/1052 [==============================] - 0s 84us/step - loss: 7.9414 - val_loss: 9.4185
Epoch 905/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.6936 - val_loss: 8.3736
Epoch 906/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.8867 - val_loss: 8.8520
Epoch 907/1000
1052/1052 [==============================] - 0s 84us/step - loss: 7.9377 - val_loss: 8.6912
Epoch 908/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.9525 - val_loss: 11.7110
Epoch 909/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.7786 - val_loss: 9.2586
Epoch 910/1000
1052/1052 [==============================] - 0s 80us/step - loss: 7.8068 - val_loss: 8.4412
Epoch 911/1000
1052/1052 [=========================

[array([[ 0.16869591, -4.5830703 ,  0.21853624,  0.34685877,  2.134072  ,
         -0.52742636,  4.362717  ,  0.20580088,  0.24192305, -0.25082925],
        [ 0.14573134,  0.13558641,  0.0393726 ,  0.15022649, -0.24018942,
         -1.3695341 ,  0.21921332,  0.03771839,  0.04177385, -0.04437679],
        [ 0.28876925, -1.1104798 ,  0.3465955 ,  0.45129636, -0.23567395,
         -1.6067374 ,  0.36886227,  0.34542382,  0.339256  , -0.35149327],
        [ 0.34279472,  0.19039941, -0.9646872 , -0.17773917, -0.07163884,
          0.24728714, -0.14898492, -0.96232986, -0.93833876,  0.96948516],
        [ 0.20692174, -0.28288946,  0.44806746,  0.49227995,  0.2068768 ,
          0.50140893,  3.5404427 ,  0.4430999 ,  0.44466478, -0.4614763 ]],
       dtype=float32),
 array([-3.872446  , -5.503177  , -4.111775  , -0.6098754 ,  1.1344056 ,
        -0.97056836,  5.187788  , -4.2110896 , -4.176836  ,  3.9304414 ],
       dtype=float32),
 array([[ 1.3923874e+00, -1.1029625e+00,  1.2223166e+00,  1.4

In [35]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure5_2nd.h5')

In [36]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_6(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_11 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 15)                165       
_________________________________________________________________
activation_12 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 16        
Total params: 241
Trainable params: 241
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 255us/step - l

1052/1052 [==============================] - 0s 91us/step - loss: 12.8849 - val_loss: 14.6735
Epoch 68/1000
1052/1052 [==============================] - 0s 95us/step - loss: 13.4410 - val_loss: 14.8840
Epoch 69/1000
1052/1052 [==============================] - 0s 98us/step - loss: 12.8463 - val_loss: 14.6931
Epoch 70/1000
1052/1052 [==============================] - 0s 89us/step - loss: 13.5069 - val_loss: 14.3315
Epoch 71/1000
1052/1052 [==============================] - 0s 85us/step - loss: 12.3790 - val_loss: 14.0956
Epoch 72/1000
1052/1052 [==============================] - 0s 89us/step - loss: 12.5050 - val_loss: 14.2997
Epoch 73/1000
1052/1052 [==============================] - 0s 88us/step - loss: 12.6325 - val_loss: 13.8672
Epoch 74/1000
1052/1052 [==============================] - 0s 88us/step - loss: 12.3889 - val_loss: 15.8649
Epoch 75/1000
1052/1052 [==============================] - 0s 92us/step - loss: 12.4547 - val_loss: 13.8593
Epoch 76/1000
1052/1052 [=================

Epoch 143/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.2733 - val_loss: 10.1605
Epoch 144/1000
1052/1052 [==============================] - 0s 98us/step - loss: 8.4915 - val_loss: 11.6016
Epoch 145/1000
1052/1052 [==============================] - 0s 98us/step - loss: 8.8348 - val_loss: 9.9954
Epoch 146/1000
1052/1052 [==============================] - 0s 96us/step - loss: 8.4765 - val_loss: 10.3518
Epoch 147/1000
1052/1052 [==============================] - 0s 103us/step - loss: 8.3667 - val_loss: 9.2169
Epoch 148/1000
1052/1052 [==============================] - 0s 100us/step - loss: 8.5000 - val_loss: 9.0866
Epoch 149/1000
1052/1052 [==============================] - 0s 99us/step - loss: 8.1656 - val_loss: 9.3191
Epoch 150/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.5301 - val_loss: 10.6372
Epoch 151/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.4282 - val_loss: 9.4615
Epoch 152/1000
1052/1052 [=====

1052/1052 [==============================] - 0s 87us/step - loss: 7.6856 - val_loss: 9.9444
Epoch 296/1000
1052/1052 [==============================] - 0s 89us/step - loss: 7.7588 - val_loss: 8.6988
Epoch 297/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.6054 - val_loss: 8.8464
Epoch 298/1000
1052/1052 [==============================] - 0s 84us/step - loss: 7.0572 - val_loss: 8.2975
Epoch 299/1000
1052/1052 [==============================] - 0s 93us/step - loss: 7.2954 - val_loss: 9.9817
Epoch 300/1000
1052/1052 [==============================] - 0s 90us/step - loss: 7.2068 - val_loss: 8.4897
Epoch 301/1000
1052/1052 [==============================] - 0s 94us/step - loss: 7.1954 - val_loss: 9.0634
Epoch 302/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.4489 - val_loss: 8.8754
Epoch 303/1000
1052/1052 [==============================] - 0s 91us/step - loss: 7.0611 - val_loss: 9.6904
Epoch 304/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 82us/step - loss: 6.8295 - val_loss: 7.8712
Epoch 448/1000
1052/1052 [==============================] - 0s 82us/step - loss: 7.1870 - val_loss: 8.4880
Epoch 449/1000
1052/1052 [==============================] - 0s 96us/step - loss: 7.1340 - val_loss: 8.7921
Epoch 450/1000
1052/1052 [==============================] - 0s 100us/step - loss: 7.1282 - val_loss: 9.6255
Epoch 451/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.5050 - val_loss: 9.5300
Epoch 452/1000
1052/1052 [==============================] - 0s 89us/step - loss: 7.6649 - val_loss: 8.2846
Epoch 453/1000
1052/1052 [==============================] - 0s 88us/step - loss: 7.2491 - val_loss: 8.3517
Epoch 454/1000
1052/1052 [==============================] - 0s 94us/step - loss: 7.1095 - val_loss: 8.8948
Epoch 455/1000
1052/1052 [==============================] - ETA: 0s - loss: 7.755 - 0s 96us/step - loss: 7.4384 - val_loss: 7.9820
Epoch 456/1000
1052/1052 [=

1052/1052 [==============================] - 0s 88us/step - loss: 7.1763 - val_loss: 8.0852
Epoch 600/1000
1052/1052 [==============================] - 0s 89us/step - loss: 7.0127 - val_loss: 8.3425
Epoch 601/1000
1052/1052 [==============================] - 0s 89us/step - loss: 6.9239 - val_loss: 7.6967
Epoch 602/1000
1052/1052 [==============================] - 0s 89us/step - loss: 7.0073 - val_loss: 7.8207
Epoch 603/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.1629 - val_loss: 10.2555
Epoch 604/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.3465 - val_loss: 10.4141
Epoch 605/1000
1052/1052 [==============================] - 0s 86us/step - loss: 6.9241 - val_loss: 7.9673
Epoch 606/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.2750 - val_loss: 8.5936
Epoch 607/1000
1052/1052 [==============================] - 0s 88us/step - loss: 7.1834 - val_loss: 9.1962
Epoch 608/1000
1052/1052 [========================

1052/1052 [==============================] - 0s 88us/step - loss: 7.0505 - val_loss: 7.5105
Epoch 752/1000
1052/1052 [==============================] - 0s 90us/step - loss: 6.9068 - val_loss: 8.1477
Epoch 753/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.2761 - val_loss: 7.4967
Epoch 754/1000
1052/1052 [==============================] - 0s 90us/step - loss: 7.5195 - val_loss: 8.7371
Epoch 755/1000
1052/1052 [==============================] - 0s 88us/step - loss: 7.1637 - val_loss: 8.0743
Epoch 756/1000
1052/1052 [==============================] - 0s 87us/step - loss: 6.8398 - val_loss: 8.0194
Epoch 757/1000
1052/1052 [==============================] - 0s 84us/step - loss: 6.9569 - val_loss: 8.7062
Epoch 758/1000
1052/1052 [==============================] - 0s 89us/step - loss: 7.4001 - val_loss: 7.5629
Epoch 759/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.0471 - val_loss: 8.0261
Epoch 760/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 88us/step - loss: 7.2597 - val_loss: 7.3529
Epoch 904/1000
1052/1052 [==============================] - 0s 88us/step - loss: 6.8960 - val_loss: 7.6019
Epoch 905/1000
1052/1052 [==============================] - 0s 89us/step - loss: 7.1663 - val_loss: 8.5923
Epoch 906/1000
1052/1052 [==============================] - 0s 91us/step - loss: 6.5846 - val_loss: 9.5824
Epoch 907/1000
1052/1052 [==============================] - 0s 88us/step - loss: 6.9346 - val_loss: 7.3088
Epoch 908/1000
1052/1052 [==============================] - 0s 86us/step - loss: 6.7606 - val_loss: 7.5741
Epoch 909/1000
1052/1052 [==============================] - 0s 84us/step - loss: 7.5758 - val_loss: 8.1231
Epoch 910/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.3403 - val_loss: 7.1867
Epoch 911/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.0490 - val_loss: 8.2262
Epoch 912/1000
1052/1052 [==========================

[array([[-2.47793466e-01,  3.61094522e+00,  4.24492687e-01,
          1.47435784e-01,  4.46640682e+00, -2.56397223e+00,
          1.48210302e-01,  1.47676364e-01, -2.77113914e-01,
          1.87571704e+00],
        [-1.97419614e-01,  3.02350134e-01,  6.73184514e-01,
          1.74133524e-01, -1.42767847e-01, -1.53159845e+00,
          1.74932569e-01,  1.74360529e-01, -6.45039916e-01,
          1.08035171e+00],
        [-3.64782602e-01,  1.09964681e+00,  1.48487186e+00,
          2.38617003e-01,  1.15158808e+00, -3.05185378e-01,
          2.39781246e-01,  2.38938406e-01, -1.05844760e+00,
          1.06740512e-01],
        [ 2.82195628e-01, -1.90865025e-01, -1.32741570e-01,
         -3.57131008e-03, -1.96244135e-01,  1.80269733e-01,
         -3.07226880e-03, -3.39731830e-03, -5.19601166e-01,
         -1.88727871e-01],
        [-2.14215666e-01,  3.42611015e-01, -8.18934850e-03,
          1.09354384e-01,  3.76127660e-01, -2.43394017e+00,
          1.10053815e-01,  1.09613836e-01, -3.773584

In [37]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure6_2nd.h5')

In [38]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_1(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_13 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_20 (Dense)             (None, 5)                 30        
_________________________________________________________________
activation_14 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 6         
Total params: 76
Trainable params: 76
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 778us/step - loss: 5

274/274 [==============================] - 0s 98us/step - loss: 7.1227 - val_loss: 9.3638
Epoch 70/1000
274/274 [==============================] - 0s 102us/step - loss: 7.0851 - val_loss: 9.4406
Epoch 71/1000
274/274 [==============================] - 0s 98us/step - loss: 7.1238 - val_loss: 9.4670
Epoch 72/1000
274/274 [==============================] - 0s 100us/step - loss: 7.0666 - val_loss: 9.7162
Epoch 73/1000
274/274 [==============================] - 0s 98us/step - loss: 7.1370 - val_loss: 9.5185
Epoch 74/1000
274/274 [==============================] - 0s 104us/step - loss: 7.0704 - val_loss: 9.5101
Epoch 75/1000
274/274 [==============================] - 0s 101us/step - loss: 7.0139 - val_loss: 9.6685
Epoch 76/1000
274/274 [==============================] - 0s 107us/step - loss: 7.0194 - val_loss: 9.3185
Epoch 77/1000
274/274 [==============================] - 0s 104us/step - loss: 7.2111 - val_loss: 9.5054
Epoch 78/1000
274/274 [==============================] - 0s 100us/step -

Epoch 225/1000
274/274 [==============================] - 0s 105us/step - loss: 6.7172 - val_loss: 9.5818
Epoch 226/1000
274/274 [==============================] - 0s 100us/step - loss: 6.6865 - val_loss: 9.5145
Epoch 227/1000
274/274 [==============================] - 0s 102us/step - loss: 6.7090 - val_loss: 9.3930
Epoch 228/1000
274/274 [==============================] - 0s 98us/step - loss: 6.7215 - val_loss: 9.2417
Epoch 229/1000
274/274 [==============================] - 0s 98us/step - loss: 6.7632 - val_loss: 9.5293
Epoch 230/1000
274/274 [==============================] - 0s 96us/step - loss: 6.7293 - val_loss: 9.5376
Epoch 231/1000
274/274 [==============================] - 0s 97us/step - loss: 6.7980 - val_loss: 9.5078
Epoch 232/1000
274/274 [==============================] - 0s 100us/step - loss: 6.7091 - val_loss: 9.7736
Epoch 233/1000
274/274 [==============================] - 0s 93us/step - loss: 6.8123 - val_loss: 9.3798
Epoch 234/1000
274/274 [===========================

Epoch 380/1000
274/274 [==============================] - 0s 102us/step - loss: 6.4627 - val_loss: 9.3690
Epoch 381/1000
274/274 [==============================] - 0s 100us/step - loss: 6.4931 - val_loss: 9.7361
Epoch 382/1000
274/274 [==============================] - 0s 100us/step - loss: 6.5118 - val_loss: 9.6017
Epoch 383/1000
274/274 [==============================] - 0s 97us/step - loss: 6.5057 - val_loss: 9.6406
Epoch 384/1000
274/274 [==============================] - 0s 102us/step - loss: 6.4482 - val_loss: 9.3137
Epoch 385/1000
274/274 [==============================] - 0s 96us/step - loss: 6.5615 - val_loss: 9.4749
Epoch 386/1000
274/274 [==============================] - 0s 99us/step - loss: 6.4765 - val_loss: 9.4938
Epoch 387/1000
274/274 [==============================] - 0s 103us/step - loss: 6.4750 - val_loss: 9.5404
Epoch 388/1000
274/274 [==============================] - 0s 97us/step - loss: 6.5045 - val_loss: 9.2448
Epoch 389/1000
274/274 [==========================

Epoch 458/1000
274/274 [==============================] - 0s 94us/step - loss: 6.5985 - val_loss: 9.5492
Epoch 459/1000
274/274 [==============================] - 0s 99us/step - loss: 6.6675 - val_loss: 9.2638
Epoch 460/1000
274/274 [==============================] - 0s 95us/step - loss: 6.4392 - val_loss: 9.3610
Epoch 461/1000
274/274 [==============================] - 0s 95us/step - loss: 6.5028 - val_loss: 9.3586
Epoch 462/1000
274/274 [==============================] - 0s 94us/step - loss: 6.4326 - val_loss: 9.5125
Epoch 463/1000
274/274 [==============================] - 0s 98us/step - loss: 6.4812 - val_loss: 9.2284
Epoch 464/1000
274/274 [==============================] - 0s 105us/step - loss: 6.3471 - val_loss: 9.7232
Epoch 465/1000
274/274 [==============================] - 0s 102us/step - loss: 6.3831 - val_loss: 9.3782
Epoch 466/1000
274/274 [==============================] - 0s 101us/step - loss: 6.4748 - val_loss: 9.4011
Epoch 467/1000
274/274 [============================

Epoch 613/1000
274/274 [==============================] - 0s 99us/step - loss: 6.2901 - val_loss: 9.3021
Epoch 614/1000
274/274 [==============================] - 0s 100us/step - loss: 6.2321 - val_loss: 9.4562
Epoch 615/1000
274/274 [==============================] - 0s 96us/step - loss: 6.4187 - val_loss: 9.5610
Epoch 616/1000
274/274 [==============================] - 0s 98us/step - loss: 6.3680 - val_loss: 9.5269
Epoch 617/1000
274/274 [==============================] - 0s 101us/step - loss: 6.3151 - val_loss: 9.3429
Epoch 618/1000
274/274 [==============================] - 0s 98us/step - loss: 6.3055 - val_loss: 9.5596
Epoch 619/1000
274/274 [==============================] - 0s 98us/step - loss: 6.2323 - val_loss: 9.3556
Epoch 620/1000
274/274 [==============================] - 0s 98us/step - loss: 6.3227 - val_loss: 9.6393
Epoch 621/1000
274/274 [==============================] - 0s 98us/step - loss: 6.3200 - val_loss: 9.3893
Epoch 622/1000
274/274 [=============================

Epoch 691/1000
274/274 [==============================] - 0s 91us/step - loss: 6.4563 - val_loss: 9.5090
Epoch 692/1000
274/274 [==============================] - 0s 98us/step - loss: 6.3241 - val_loss: 9.8035
Epoch 693/1000
274/274 [==============================] - 0s 102us/step - loss: 6.2443 - val_loss: 9.5423
Epoch 694/1000
274/274 [==============================] - 0s 98us/step - loss: 6.2987 - val_loss: 9.3935
Epoch 695/1000
274/274 [==============================] - 0s 95us/step - loss: 6.3576 - val_loss: 9.8774
Epoch 696/1000
274/274 [==============================] - 0s 99us/step - loss: 6.3651 - val_loss: 9.3323
Epoch 697/1000
274/274 [==============================] - 0s 99us/step - loss: 6.2691 - val_loss: 9.6725
Epoch 698/1000
274/274 [==============================] - 0s 101us/step - loss: 6.2820 - val_loss: 9.9629
Epoch 699/1000
274/274 [==============================] - 0s 99us/step - loss: 6.2915 - val_loss: 9.3756
Epoch 700/1000
274/274 [=============================

Epoch 846/1000
274/274 [==============================] - 0s 105us/step - loss: 6.1267 - val_loss: 9.7445
Epoch 847/1000
274/274 [==============================] - 0s 98us/step - loss: 6.0790 - val_loss: 9.7196
Epoch 848/1000
274/274 [==============================] - 0s 109us/step - loss: 6.1939 - val_loss: 9.5502
Epoch 849/1000
274/274 [==============================] - 0s 113us/step - loss: 6.2554 - val_loss: 9.9799
Epoch 850/1000
274/274 [==============================] - 0s 98us/step - loss: 6.1509 - val_loss: 9.7210
Epoch 851/1000
274/274 [==============================] - 0s 97us/step - loss: 6.1447 - val_loss: 9.6797
Epoch 852/1000
274/274 [==============================] - 0s 109us/step - loss: 6.0593 - val_loss: 9.6463
Epoch 853/1000
274/274 [==============================] - 0s 109us/step - loss: 6.1015 - val_loss: 9.8201
Epoch 854/1000
274/274 [==============================] - 0s 109us/step - loss: 6.1343 - val_loss: 9.5965
Epoch 855/1000
274/274 [=========================

274/274 [==============================] - 0s 110us/step - loss: 6.2183 - val_loss: 9.7685
6.663557723417121


[array([[ 8.5977864e-01,  2.8469706e+00, -3.9561072e-01,  1.8970086e-01,
         -2.3424170e+00],
        [-1.2839850e+00,  3.3574734e+00, -1.6425178e+00, -3.9922032e-01,
          1.3754725e-01],
        [-1.7036173e-01, -2.2711823e+00, -3.0983596e+00, -2.2707460e-04,
          5.1260465e-01],
        [ 8.7766141e-01, -2.9744709e-01,  2.1063912e-01,  6.9774294e-01,
          8.1938338e-01],
        [-9.5679188e-01,  1.4381605e-01, -9.2261642e-01, -2.2024387e-01,
          4.6889967e-01],
        [ 6.7043227e-01,  2.5877359e+00, -7.9490113e-01, -1.5886112e-01,
         -2.2414677e-01],
        [ 1.4893905e+00, -2.1596235e-01,  1.3260319e+00,  2.8514113e-02,
          7.5780994e-01]], dtype=float32),
 array([ 2.4181623 ,  2.5415113 , -1.2846136 ,  0.19532198, -3.80093   ],
       dtype=float32),
 array([[ 0.4826093 ,  1.391788  ,  0.7483634 , -0.7809304 ,  1.20772   ],
        [ 0.09191418,  0.8145214 , -0.02129825,  0.31373885,  0.65178853],
        [ 0.2492145 ,  1.4384949 , -0.08068

In [39]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure1_2nd.h5')

In [40]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_2(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_15 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_16 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 916us/step - loss:

274/274 [==============================] - 0s 98us/step - loss: 6.7336 - val_loss: 8.2764
Epoch 147/1000
274/274 [==============================] - 0s 95us/step - loss: 6.4319 - val_loss: 7.9687
Epoch 148/1000
274/274 [==============================] - 0s 95us/step - loss: 6.4623 - val_loss: 8.1737
Epoch 149/1000
274/274 [==============================] - 0s 98us/step - loss: 6.5009 - val_loss: 8.0598
Epoch 150/1000
274/274 [==============================] - 0s 95us/step - loss: 6.6419 - val_loss: 7.8546
Epoch 151/1000
274/274 [==============================] - 0s 91us/step - loss: 6.6105 - val_loss: 8.1596
Epoch 152/1000
274/274 [==============================] - 0s 96us/step - loss: 6.5263 - val_loss: 7.8630
Epoch 153/1000
274/274 [==============================] - 0s 97us/step - loss: 6.5371 - val_loss: 8.3366
Epoch 154/1000
274/274 [==============================] - 0s 98us/step - loss: 6.5865 - val_loss: 8.2561
Epoch 155/1000
274/274 [==============================] - 0s 95us/step

274/274 [==============================] - 0s 98us/step - loss: 6.6168 - val_loss: 8.1658
Epoch 225/1000
274/274 [==============================] - 0s 97us/step - loss: 6.2905 - val_loss: 7.8141
Epoch 226/1000
274/274 [==============================] - 0s 91us/step - loss: 6.1687 - val_loss: 8.1433
Epoch 227/1000
274/274 [==============================] - 0s 95us/step - loss: 6.2355 - val_loss: 8.0984
Epoch 228/1000
274/274 [==============================] - 0s 98us/step - loss: 6.2840 - val_loss: 7.9906
Epoch 229/1000
274/274 [==============================] - 0s 95us/step - loss: 6.1785 - val_loss: 7.8054
Epoch 230/1000
274/274 [==============================] - 0s 91us/step - loss: 6.1865 - val_loss: 7.8583
Epoch 231/1000
274/274 [==============================] - 0s 102us/step - loss: 6.1615 - val_loss: 7.8737
Epoch 232/1000
274/274 [==============================] - 0s 98us/step - loss: 6.2939 - val_loss: 7.7138
Epoch 233/1000
274/274 [==============================] - 0s 102us/st

Epoch 302/1000
274/274 [==============================] - 0s 101us/step - loss: 6.0228 - val_loss: 7.9409
Epoch 303/1000
274/274 [==============================] - 0s 101us/step - loss: 6.2741 - val_loss: 7.8158
Epoch 304/1000
274/274 [==============================] - 0s 99us/step - loss: 6.3226 - val_loss: 7.8789
Epoch 305/1000
274/274 [==============================] - 0s 101us/step - loss: 6.1869 - val_loss: 7.8631
Epoch 306/1000
274/274 [==============================] - 0s 97us/step - loss: 6.0429 - val_loss: 7.8995
Epoch 307/1000
274/274 [==============================] - 0s 107us/step - loss: 6.1077 - val_loss: 8.2990
Epoch 308/1000
274/274 [==============================] - 0s 106us/step - loss: 6.2962 - val_loss: 7.7488
Epoch 309/1000
274/274 [==============================] - 0s 91us/step - loss: 6.5633 - val_loss: 7.8618
Epoch 310/1000
274/274 [==============================] - 0s 95us/step - loss: 6.1728 - val_loss: 7.8445
Epoch 311/1000
274/274 [==========================

Epoch 380/1000
274/274 [==============================] - 0s 102us/step - loss: 6.0499 - val_loss: 8.8429
Epoch 381/1000
274/274 [==============================] - 0s 95us/step - loss: 6.6846 - val_loss: 8.2762
Epoch 382/1000
274/274 [==============================] - 0s 95us/step - loss: 6.2707 - val_loss: 8.6687
Epoch 383/1000
274/274 [==============================] - 0s 94us/step - loss: 6.1605 - val_loss: 8.6825
Epoch 384/1000
274/274 [==============================] - 0s 91us/step - loss: 6.3898 - val_loss: 8.8463
Epoch 385/1000
274/274 [==============================] - 0s 94us/step - loss: 6.3905 - val_loss: 7.9247
Epoch 386/1000
274/274 [==============================] - 0s 100us/step - loss: 6.0417 - val_loss: 8.2732
Epoch 387/1000
274/274 [==============================] - 0s 101us/step - loss: 6.2147 - val_loss: 8.1759
Epoch 388/1000
274/274 [==============================] - 0s 98us/step - loss: 6.1536 - val_loss: 8.3895
Epoch 389/1000
274/274 [============================

...

Epoch 613/1000
274/274 [==============================] - 0s 109us/step - loss: 5.7349 - val_loss: 8.4534
Epoch 614/1000
274/274 [==============================] - 0s 96us/step - loss: 5.7128 - val_loss: 8.7005
Epoch 615/1000
274/274 [==============================] - 0s 106us/step - loss: 5.7223 - val_loss: 8.1128
Epoch 616/1000
274/274 [==============================] - 0s 113us/step - loss: 5.7125 - val_loss: 8.5042
Epoch 617/1000
274/274 [==============================] - 0s 116us/step - loss: 6.0211 - val_loss: 8.9720
Epoch 618/1000
274/274 [==============================] - 0s 102us/step - loss: 5.8627 - val_loss: 8.7070
Epoch 619/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7220 - val_loss: 8.6817
Epoch 620/1000
274/274 [==============================] - 0s 113us/step - loss: 5.7052 - val_loss: 8.7477
Epoch 621/1000
274/274 [==============================] - 0s 138us/step - loss: 5.6648 - val_loss: 8.3895
Epoch 622/1000
274/274 [========================

Epoch 691/1000
274/274 [==============================] - 0s 100us/step - loss: 5.6847 - val_loss: 8.5402
Epoch 692/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6217 - val_loss: 8.3660
Epoch 693/1000
274/274 [==============================] - 0s 91us/step - loss: 5.7292 - val_loss: 8.1995
Epoch 694/1000
274/274 [==============================] - 0s 96us/step - loss: 5.7649 - val_loss: 8.4223
Epoch 695/1000
274/274 [==============================] - 0s 95us/step - loss: 5.8627 - val_loss: 8.5415
Epoch 696/1000
274/274 [==============================] - 0s 92us/step - loss: 5.7337 - val_loss: 8.4919
Epoch 697/1000
274/274 [==============================] - 0s 94us/step - loss: 5.8104 - val_loss: 8.0921
Epoch 698/1000
274/274 [==============================] - 0s 99us/step - loss: 5.6009 - val_loss: 8.3601
Epoch 699/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7984 - val_loss: 9.1142
Epoch 700/1000
274/274 [==============================

Epoch 769/1000
274/274 [==============================] - 0s 106us/step - loss: 5.5826 - val_loss: 9.0343
Epoch 770/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6554 - val_loss: 8.5340
Epoch 771/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6227 - val_loss: 8.7475
Epoch 772/1000
274/274 [==============================] - 0s 102us/step - loss: 5.6468 - val_loss: 8.1728
Epoch 773/1000
274/274 [==============================] - 0s 98us/step - loss: 5.7054 - val_loss: 8.4064
Epoch 774/1000
274/274 [==============================] - 0s 98us/step - loss: 5.8643 - val_loss: 8.4565
Epoch 775/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6043 - val_loss: 8.8369
Epoch 776/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6617 - val_loss: 8.5499
Epoch 777/1000
274/274 [==============================] - 0s 96us/step - loss: 5.5717 - val_loss: 8.3901
Epoch 778/1000
274/274 [=============================

Epoch 847/1000
274/274 [==============================] - 0s 102us/step - loss: 5.5105 - val_loss: 8.5177
Epoch 848/1000
274/274 [==============================] - 0s 97us/step - loss: 5.5142 - val_loss: 8.5313
Epoch 849/1000
274/274 [==============================] - 0s 98us/step - loss: 5.5916 - val_loss: 8.4050
Epoch 850/1000
274/274 [==============================] - 0s 103us/step - loss: 5.4585 - val_loss: 8.5096
Epoch 851/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4951 - val_loss: 8.4994
Epoch 852/1000
274/274 [==============================] - 0s 100us/step - loss: 5.4393 - val_loss: 8.5141
Epoch 853/1000
274/274 [==============================] - 0s 101us/step - loss: 5.4785 - val_loss: 8.7878
Epoch 854/1000
274/274 [==============================] - 0s 91us/step - loss: 5.4951 - val_loss: 8.5963
Epoch 855/1000
274/274 [==============================] - 0s 102us/step - loss: 5.6489 - val_loss: 8.5114
Epoch 856/1000
274/274 [==========================

[array([[ 2.6584237 ,  3.600912  ,  1.0642779 , -1.4646646 , -3.9994962 ],
        [-0.55934453, -3.151859  ,  1.2732308 , -1.1256683 ,  1.5035728 ],
        [-0.49883747, -0.6132283 ,  0.18551446, -0.21356298, -1.1399486 ],
        [-0.2986147 ,  1.943602  , -0.3162826 ,  1.1644708 , -2.184272  ],
        [-0.11221903,  1.2907693 ,  0.7853834 , -1.884835  ,  0.00748462],
        [ 0.31130916, -1.4160132 , -0.46827623, -1.2205663 ,  0.4667793 ],
        [ 0.072436  , -3.4009252 , -0.26400906, -0.47478816, -0.42128587]],
       dtype=float32),
 array([ 1.1896387, -1.6924345, -2.210228 ,  4.021315 , -1.340502 ],
       dtype=float32),
 array([[ 0.5058452 ,  1.0068543 , -0.9291062 ,  1.0465548 , -0.6207959 ,
          1.0926247 ,  0.2773367 , -1.0355852 , -1.0305797 ,  0.06563292],
        [ 0.20539135, -0.41403428,  0.5349409 ,  0.26193297, -0.44147187,
         -0.63234824, -0.1750498 ,  0.06486125,  0.6244549 ,  0.43516093],
        [-0.5177047 , -0.79228765,  0.35194433, -0.14061534, 

In [41]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure2_2nd.h5')

In [42]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_3(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_17 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_26 (Dense)             (None, 15)                90        
_________________________________________________________________
activation_18 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 16        
Total params: 146
Trainable params: 146
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 948us/step - loss:

Epoch 146/1000
274/274 [==============================] - 0s 91us/step - loss: 6.3042 - val_loss: 8.0422
Epoch 147/1000
274/274 [==============================] - 0s 113us/step - loss: 6.3881 - val_loss: 8.3940
Epoch 148/1000
274/274 [==============================] - 0s 102us/step - loss: 6.4413 - val_loss: 7.8944
Epoch 149/1000
274/274 [==============================] - 0s 98us/step - loss: 6.6447 - val_loss: 9.3696
Epoch 150/1000
274/274 [==============================] - 0s 109us/step - loss: 7.0073 - val_loss: 8.2852
Epoch 151/1000
274/274 [==============================] - 0s 113us/step - loss: 6.0863 - val_loss: 8.7905
Epoch 152/1000
274/274 [==============================] - 0s 98us/step - loss: 6.1801 - val_loss: 8.1403
Epoch 153/1000
274/274 [==============================] - 0s 98us/step - loss: 6.6319 - val_loss: 8.5936
Epoch 154/1000
274/274 [==============================] - 0s 95us/step - loss: 6.3525 - val_loss: 8.1524
Epoch 155/1000
274/274 [===========================

Epoch 224/1000
274/274 [==============================] - 0s 98us/step - loss: 5.9509 - val_loss: 8.2830
Epoch 225/1000
274/274 [==============================] - 0s 102us/step - loss: 6.0493 - val_loss: 9.4845
Epoch 226/1000
274/274 [==============================] - 0s 98us/step - loss: 6.1341 - val_loss: 9.1441
Epoch 227/1000
274/274 [==============================] - 0s 102us/step - loss: 6.0621 - val_loss: 8.8939
Epoch 228/1000
274/274 [==============================] - 0s 96us/step - loss: 5.8597 - val_loss: 9.0665
Epoch 229/1000
274/274 [==============================] - 0s 98us/step - loss: 5.7652 - val_loss: 8.4737
Epoch 230/1000
274/274 [==============================] - 0s 98us/step - loss: 5.9208 - val_loss: 8.3633
Epoch 231/1000
274/274 [==============================] - 0s 99us/step - loss: 6.0212 - val_loss: 8.9237
Epoch 232/1000
274/274 [==============================] - 0s 102us/step - loss: 5.8658 - val_loss: 8.5688
Epoch 233/1000
274/274 [============================

Epoch 379/1000
274/274 [==============================] - 0s 106us/step - loss: 5.6828 - val_loss: 9.1192
Epoch 380/1000
274/274 [==============================] - 0s 100us/step - loss: 5.5685 - val_loss: 9.0724
Epoch 381/1000
274/274 [==============================] - 0s 106us/step - loss: 5.7468 - val_loss: 8.8775
Epoch 382/1000
274/274 [==============================] - 0s 99us/step - loss: 5.7016 - val_loss: 9.3940
Epoch 383/1000
274/274 [==============================] - 0s 98us/step - loss: 5.8634 - val_loss: 8.9072
Epoch 384/1000
274/274 [==============================] - 0s 106us/step - loss: 5.5683 - val_loss: 9.0496
Epoch 385/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6736 - val_loss: 9.1286
Epoch 386/1000
274/274 [==============================] - 0s 99us/step - loss: 5.5384 - val_loss: 9.2883
Epoch 387/1000
274/274 [==============================] - 0s 105us/step - loss: 5.5845 - val_loss: 9.1878
Epoch 388/1000
274/274 [==========================

Epoch 534/1000
274/274 [==============================] - 0s 96us/step - loss: 5.5516 - val_loss: 9.5090
Epoch 535/1000
274/274 [==============================] - 0s 96us/step - loss: 5.8314 - val_loss: 9.0904
Epoch 536/1000
274/274 [==============================] - 0s 107us/step - loss: 5.6296 - val_loss: 9.3880
Epoch 537/1000
274/274 [==============================] - 0s 101us/step - loss: 5.7780 - val_loss: 9.9750
Epoch 538/1000
274/274 [==============================] - 0s 102us/step - loss: 5.7543 - val_loss: 9.6646
Epoch 539/1000
274/274 [==============================] - 0s 102us/step - loss: 6.0473 - val_loss: 9.7126
Epoch 540/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7267 - val_loss: 9.3544
Epoch 541/1000
274/274 [==============================] - 0s 91us/step - loss: 5.4839 - val_loss: 9.8189
Epoch 542/1000
274/274 [==============================] - 0s 102us/step - loss: 5.5549 - val_loss: 9.0899
Epoch 543/1000
274/274 [==========================

274/274 [==============================] - 0s 95us/step - loss: 5.5030 - val_loss: 9.2114
Epoch 689/1000
274/274 [==============================] - 0s 98us/step - loss: 5.5578 - val_loss: 9.4644
Epoch 690/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6272 - val_loss: 9.1553
Epoch 691/1000
274/274 [==============================] - 0s 99us/step - loss: 5.5598 - val_loss: 8.9750
Epoch 692/1000
274/274 [==============================] - 0s 98us/step - loss: 5.5245 - val_loss: 9.3381
Epoch 693/1000
274/274 [==============================] - 0s 102us/step - loss: 5.5146 - val_loss: 9.3581
Epoch 694/1000
274/274 [==============================] - 0s 120us/step - loss: 5.6121 - val_loss: 9.4964
Epoch 695/1000
274/274 [==============================] - 0s 116us/step - loss: 5.5516 - val_loss: 9.0642
Epoch 696/1000
274/274 [==============================] - 0s 120us/step - loss: 5.6120 - val_loss: 9.0865
Epoch 697/1000
274/274 [==============================] - 0s 120us

Epoch 766/1000
274/274 [==============================] - 0s 99us/step - loss: 5.4588 - val_loss: 9.0335
Epoch 767/1000
274/274 [==============================] - 0s 102us/step - loss: 5.5421 - val_loss: 9.6007
Epoch 768/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4234 - val_loss: 9.1499
Epoch 769/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6686 - val_loss: 9.5731
Epoch 770/1000
274/274 [==============================] - 0s 102us/step - loss: 5.6311 - val_loss: 9.2685
Epoch 771/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6894 - val_loss: 9.2863
Epoch 772/1000
274/274 [==============================] - 0s 99us/step - loss: 5.5497 - val_loss: 9.4878
Epoch 773/1000
274/274 [==============================] - 0s 98us/step - loss: 5.5220 - val_loss: 9.5342
Epoch 774/1000
274/274 [==============================] - 0s 93us/step - loss: 5.5667 - val_loss: 9.1968
Epoch 775/1000
274/274 [=============================

Epoch 844/1000
274/274 [==============================] - 0s 99us/step - loss: 5.6165 - val_loss: 9.2997
Epoch 845/1000
274/274 [==============================] - 0s 101us/step - loss: 5.4580 - val_loss: 9.1707
Epoch 846/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4764 - val_loss: 9.0675
Epoch 847/1000
274/274 [==============================] - 0s 102us/step - loss: 5.8806 - val_loss: 9.7701
Epoch 848/1000
274/274 [==============================] - 0s 98us/step - loss: 5.9770 - val_loss: 9.1519
Epoch 849/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4763 - val_loss: 9.0274
Epoch 850/1000
274/274 [==============================] - 0s 102us/step - loss: 5.6616 - val_loss: 9.0353
Epoch 851/1000
274/274 [==============================] - 0s 91us/step - loss: 5.7695 - val_loss: 10.0548
Epoch 852/1000
274/274 [==============================] - 0s 96us/step - loss: 5.7629 - val_loss: 9.4538
Epoch 853/1000
274/274 [===========================

Epoch 999/1000
274/274 [==============================] - 0s 98us/step - loss: 5.5458 - val_loss: 9.1686
Epoch 1000/1000
274/274 [==============================] - 0s 97us/step - loss: 5.4293 - val_loss: 9.2524
6.98731241387836


[array([[-0.27501857,  0.00647581, -1.7981249 , -0.25894523, -1.5347608 ],
        [ 0.13419281, -0.7983037 , -1.756959  , -0.17998771,  0.98929536],
        [-0.0946307 ,  0.2640069 ,  0.41315675,  0.54601234, -0.10241376],
        [-0.7360216 ,  1.0029446 ,  1.9121959 , -0.26165012, -1.6212268 ],
        [ 0.01639562, -0.56000984, -0.12175646,  0.77160424,  0.2056142 ],
        [-0.38294467, -1.2857094 ,  3.834698  , -0.0809181 , -1.7040383 ],
        [-0.3256187 , -0.5813387 ,  2.7194846 , -1.7284831 , -0.3302039 ]],
       dtype=float32),
 array([-0.7437551,  1.9427844,  3.4529698, -2.1505299, -1.9501916],
       dtype=float32),
 array([[-0.876621  ,  0.85907936, -0.46372515,  0.40297446, -0.9730287 ,
          0.91716117,  0.13793106,  0.96972895,  0.7497994 , -1.0428926 ,
         -1.0962548 , -0.40737143,  0.9935972 , -1.0462191 ,  0.7164666 ],
        [ 0.23083621, -0.9596442 ,  0.512562  , -0.56063604,  0.3212999 ,
         -0.65442467,  0.51854604, -0.44116563, -0.3355713 ,  

In [43]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure3_2nd.h5')

In [44]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_4(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 10)                80        
_________________________________________________________________
activation_19 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_29 (Dense)             (None, 5)                 55        
_________________________________________________________________
activation_20 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 6         
Total params: 141
Trainable params: 141
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 963us/step - loss

Epoch 69/1000
274/274 [==============================] - 0s 99us/step - loss: 6.6693 - val_loss: 9.8223
Epoch 70/1000
274/274 [==============================] - 0s 101us/step - loss: 6.6120 - val_loss: 9.9789
Epoch 71/1000
274/274 [==============================] - 0s 97us/step - loss: 6.5831 - val_loss: 10.1454
Epoch 72/1000
274/274 [==============================] - 0s 96us/step - loss: 6.6086 - val_loss: 9.9002
Epoch 73/1000
274/274 [==============================] - 0s 98us/step - loss: 6.5332 - val_loss: 9.7211
Epoch 74/1000
274/274 [==============================] - 0s 106us/step - loss: 6.6703 - val_loss: 9.6867
Epoch 75/1000
274/274 [==============================] - 0s 138us/step - loss: 6.6127 - val_loss: 9.9490
Epoch 76/1000
274/274 [==============================] - 0s 116us/step - loss: 6.6256 - val_loss: 10.1970
Epoch 77/1000
274/274 [==============================] - 0s 113us/step - loss: 6.5375 - val_loss: 9.8546
Epoch 78/1000
274/274 [==============================] - 

274/274 [==============================] - 0s 102us/step - loss: 5.4319 - val_loss: 8.6407
Epoch 224/1000
274/274 [==============================] - 0s 106us/step - loss: 5.4240 - val_loss: 8.5830
Epoch 225/1000
274/274 [==============================] - 0s 120us/step - loss: 5.4726 - val_loss: 8.6315
Epoch 226/1000
274/274 [==============================] - 0s 115us/step - loss: 5.4913 - val_loss: 9.0857
Epoch 227/1000
274/274 [==============================] - 0s 109us/step - loss: 5.4123 - val_loss: 8.7764
Epoch 228/1000
274/274 [==============================] - 0s 106us/step - loss: 5.5273 - val_loss: 8.8630
Epoch 229/1000
274/274 [==============================] - 0s 114us/step - loss: 5.9388 - val_loss: 8.7012
Epoch 230/1000
274/274 [==============================] - 0s 109us/step - loss: 6.6022 - val_loss: 9.6993
Epoch 231/1000
274/274 [==============================] - 0s 124us/step - loss: 6.2796 - val_loss: 8.9992
Epoch 232/1000
274/274 [==============================] - 0s 

Epoch 301/1000
274/274 [==============================] - 0s 98us/step - loss: 5.1859 - val_loss: 8.9376
Epoch 302/1000
274/274 [==============================] - 0s 100us/step - loss: 5.6016 - val_loss: 8.9243
Epoch 303/1000
274/274 [==============================] - 0s 97us/step - loss: 5.5169 - val_loss: 9.1779
Epoch 304/1000
274/274 [==============================] - 0s 91us/step - loss: 5.3494 - val_loss: 8.6621
Epoch 305/1000
274/274 [==============================] - 0s 91us/step - loss: 5.3918 - val_loss: 8.6900
Epoch 306/1000
274/274 [==============================] - 0s 91us/step - loss: 5.2350 - val_loss: 8.8340
Epoch 307/1000
274/274 [==============================] - 0s 100us/step - loss: 5.2331 - val_loss: 9.1243
Epoch 308/1000
274/274 [==============================] - 0s 98us/step - loss: 5.1740 - val_loss: 8.8797
Epoch 309/1000
274/274 [==============================] - 0s 102us/step - loss: 5.1826 - val_loss: 8.6074
Epoch 310/1000
274/274 [============================

Epoch 456/1000
274/274 [==============================] - 0s 98us/step - loss: 4.5772 - val_loss: 8.3102
Epoch 457/1000
274/274 [==============================] - 0s 104us/step - loss: 4.5726 - val_loss: 8.1749
Epoch 458/1000
274/274 [==============================] - 0s 97us/step - loss: 4.5530 - val_loss: 8.2962
Epoch 459/1000
274/274 [==============================] - 0s 95us/step - loss: 4.6079 - val_loss: 8.5377
Epoch 460/1000
274/274 [==============================] - 0s 101us/step - loss: 4.6006 - val_loss: 8.2637
Epoch 461/1000
274/274 [==============================] - 0s 101us/step - loss: 4.6644 - val_loss: 8.8640
Epoch 462/1000
274/274 [==============================] - 0s 101us/step - loss: 4.7857 - val_loss: 8.0354
Epoch 463/1000
274/274 [==============================] - 0s 97us/step - loss: 4.5057 - val_loss: 8.0610
Epoch 464/1000
274/274 [==============================] - 0s 96us/step - loss: 4.7606 - val_loss: 8.4042
Epoch 465/1000
274/274 [===========================

Epoch 611/1000
274/274 [==============================] - 0s 95us/step - loss: 4.0133 - val_loss: 7.5551
Epoch 612/1000
274/274 [==============================] - 0s 106us/step - loss: 4.0788 - val_loss: 7.6348
Epoch 613/1000
274/274 [==============================] - 0s 106us/step - loss: 4.1148 - val_loss: 7.4295
Epoch 614/1000
274/274 [==============================] - 0s 109us/step - loss: 4.1005 - val_loss: 7.2918
Epoch 615/1000
274/274 [==============================] - 0s 116us/step - loss: 4.2004 - val_loss: 7.8070
Epoch 616/1000
274/274 [==============================] - 0s 131us/step - loss: 4.1892 - val_loss: 7.2778
Epoch 617/1000
274/274 [==============================] - 0s 120us/step - loss: 4.2518 - val_loss: 7.8173
Epoch 618/1000
274/274 [==============================] - 0s 124us/step - loss: 4.1191 - val_loss: 7.5672
Epoch 619/1000
274/274 [==============================] - 0s 91us/step - loss: 4.1287 - val_loss: 7.6330
Epoch 620/1000
274/274 [========================

274/274 [==============================] - 0s 100us/step - loss: 3.7869 - val_loss: 7.3469
Epoch 766/1000
274/274 [==============================] - 0s 102us/step - loss: 4.0242 - val_loss: 7.4108
Epoch 767/1000
274/274 [==============================] - 0s 106us/step - loss: 3.9241 - val_loss: 7.3580
Epoch 768/1000
274/274 [==============================] - 0s 96us/step - loss: 3.7637 - val_loss: 7.5442
Epoch 769/1000
274/274 [==============================] - 0s 102us/step - loss: 3.7848 - val_loss: 7.5673
Epoch 770/1000
274/274 [==============================] - 0s 118us/step - loss: 3.8265 - val_loss: 7.5101
Epoch 771/1000
274/274 [==============================] - 0s 113us/step - loss: 3.8131 - val_loss: 7.2731
Epoch 772/1000
274/274 [==============================] - 0s 109us/step - loss: 3.9448 - val_loss: 7.3341
Epoch 773/1000
274/274 [==============================] - 0s 106us/step - loss: 3.9795 - val_loss: 7.4957
Epoch 774/1000
274/274 [==============================] - 0s 1

274/274 [==============================] - 0s 98us/step - loss: 3.7454 - val_loss: 7.8913
Epoch 920/1000
274/274 [==============================] - 0s 98us/step - loss: 3.7238 - val_loss: 7.8811
Epoch 921/1000
274/274 [==============================] - 0s 98us/step - loss: 3.6043 - val_loss: 7.6127
Epoch 922/1000
274/274 [==============================] - 0s 95us/step - loss: 3.6456 - val_loss: 7.7524
Epoch 923/1000
274/274 [==============================] - 0s 110us/step - loss: 3.5490 - val_loss: 7.9838
Epoch 924/1000
274/274 [==============================] - 0s 98us/step - loss: 3.6964 - val_loss: 7.9559
Epoch 925/1000
274/274 [==============================] - 0s 102us/step - loss: 3.8322 - val_loss: 8.1030
Epoch 926/1000
274/274 [==============================] - 0s 112us/step - loss: 3.5935 - val_loss: 7.8680
Epoch 927/1000
274/274 [==============================] - 0s 98us/step - loss: 3.7905 - val_loss: 8.2328
Epoch 928/1000
274/274 [==============================] - 0s 113us/

Epoch 997/1000
274/274 [==============================] - 0s 103us/step - loss: 3.7287 - val_loss: 8.1999
Epoch 998/1000
274/274 [==============================] - 0s 102us/step - loss: 3.5821 - val_loss: 8.9222
Epoch 999/1000
274/274 [==============================] - 0s 102us/step - loss: 3.6396 - val_loss: 8.0143
Epoch 1000/1000
274/274 [==============================] - 0s 101us/step - loss: 3.4320 - val_loss: 7.9626
5.531032740059546


[array([[ 0.04545702, -2.511662  ,  3.762047  , -0.17416118,  3.2291272 ,
         -2.6302717 ,  3.9915245 , -7.318204  , -0.82214326, -1.0902684 ],
        [-0.5517557 ,  0.9220406 ,  0.8571945 , -1.1297529 , -0.5887715 ,
         -1.1524109 , -3.4047036 ,  4.364822  ,  0.44232997,  1.7254438 ],
        [-1.1246996 , -0.45825884, -0.7950726 , -0.77377963, -0.882725  ,
          2.3346918 , -0.56510377, -2.187164  ,  1.6658965 , -1.2665586 ],
        [-0.5957374 , -0.25802156, -0.6191043 ,  2.0048451 , -1.3560736 ,
          5.347828  ,  2.4444008 , -2.6487172 , -0.2875823 ,  0.38301754],
        [-1.6832465 ,  0.11433085, -0.20251735, -1.1141636 , -0.33357382,
         -0.29563668, -1.0002967 , -0.51464176, -0.7839633 ,  0.38977957],
        [ 1.0633119 ,  0.13517897,  2.013581  ,  0.60491997, -0.8806438 ,
         -0.24715768, -0.6073789 ,  1.1115314 ,  0.8893951 , -0.84443676],
        [ 0.29118034, -0.50261205, -0.24787265,  1.7554169 ,  1.6885302 ,
          2.2992573 , -0.2716301

In [45]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure4_2nd.h5')

In [46]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_5(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 10)                80        
_________________________________________________________________
activation_21 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_32 (Dense)             (None, 10)                110       
_________________________________________________________________
activation_22 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 11        
Total params: 201
Trainable params: 201
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 1ms/step - loss: 

274/274 [==============================] - 0s 102us/step - loss: 5.8255 - val_loss: 9.5496
Epoch 146/1000
274/274 [==============================] - 0s 93us/step - loss: 5.5659 - val_loss: 9.3731
Epoch 147/1000
274/274 [==============================] - 0s 102us/step - loss: 5.9967 - val_loss: 9.0754
Epoch 148/1000
274/274 [==============================] - 0s 99us/step - loss: 5.8555 - val_loss: 8.8210
Epoch 149/1000
274/274 [==============================] - 0s 100us/step - loss: 6.1195 - val_loss: 8.9670
Epoch 150/1000
274/274 [==============================] - 0s 99us/step - loss: 5.9019 - val_loss: 9.5477
Epoch 151/1000
274/274 [==============================] - 0s 91us/step - loss: 5.8813 - val_loss: 8.9837
Epoch 152/1000
274/274 [==============================] - 0s 102us/step - loss: 6.2581 - val_loss: 8.9939
Epoch 153/1000
274/274 [==============================] - 0s 95us/step - loss: 6.0094 - val_loss: 8.9180
Epoch 154/1000
274/274 [==============================] - 0s 102us

274/274 [==============================] - 0s 106us/step - loss: 5.2704 - val_loss: 8.6838
Epoch 300/1000
274/274 [==============================] - 0s 106us/step - loss: 5.2349 - val_loss: 8.6283
Epoch 301/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2338 - val_loss: 8.5189
Epoch 302/1000
274/274 [==============================] - 0s 98us/step - loss: 5.1907 - val_loss: 8.4384
Epoch 303/1000
274/274 [==============================] - 0s 106us/step - loss: 5.5670 - val_loss: 8.7658
Epoch 304/1000
274/274 [==============================] - 0s 98us/step - loss: 5.8402 - val_loss: 8.9564
Epoch 305/1000
274/274 [==============================] - 0s 91us/step - loss: 5.4961 - val_loss: 8.8410
Epoch 306/1000
274/274 [==============================] - 0s 106us/step - loss: 5.2879 - val_loss: 8.5619
Epoch 307/1000
274/274 [==============================] - 0s 113us/step - loss: 5.1612 - val_loss: 8.6637
Epoch 308/1000
274/274 [==============================] - 0s 116

Epoch 377/1000
274/274 [==============================] - 0s 102us/step - loss: 5.1561 - val_loss: 8.9644
Epoch 378/1000
274/274 [==============================] - 0s 96us/step - loss: 5.2456 - val_loss: 9.2892
Epoch 379/1000
274/274 [==============================] - 0s 103us/step - loss: 4.9201 - val_loss: 8.8100
Epoch 380/1000
274/274 [==============================] - 0s 102us/step - loss: 5.0610 - val_loss: 9.5258
Epoch 381/1000
274/274 [==============================] - 0s 100us/step - loss: 5.2874 - val_loss: 8.6354
Epoch 382/1000
274/274 [==============================] - 0s 98us/step - loss: 4.9103 - val_loss: 9.1938
Epoch 383/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2319 - val_loss: 9.2695
Epoch 384/1000
274/274 [==============================] - 0s 102us/step - loss: 5.7969 - val_loss: 10.7642
Epoch 385/1000
274/274 [==============================] - 0s 106us/step - loss: 5.5361 - val_loss: 8.8056
Epoch 386/1000
274/274 [=======================

Epoch 532/1000
274/274 [==============================] - 0s 95us/step - loss: 4.4399 - val_loss: 8.9371
Epoch 533/1000
274/274 [==============================] - 0s 107us/step - loss: 4.3482 - val_loss: 9.1442
Epoch 534/1000
274/274 [==============================] - 0s 102us/step - loss: 4.4128 - val_loss: 9.0763
Epoch 535/1000
274/274 [==============================] - 0s 102us/step - loss: 4.4010 - val_loss: 8.9970
Epoch 536/1000
274/274 [==============================] - 0s 100us/step - loss: 4.4058 - val_loss: 9.6554
Epoch 537/1000
274/274 [==============================] - 0s 99us/step - loss: 4.4039 - val_loss: 9.0180
Epoch 538/1000
274/274 [==============================] - 0s 113us/step - loss: 4.5730 - val_loss: 9.1350
Epoch 539/1000
274/274 [==============================] - 0s 119us/step - loss: 4.6047 - val_loss: 8.7054
Epoch 540/1000
274/274 [==============================] - 0s 120us/step - loss: 4.3387 - val_loss: 9.0380
Epoch 541/1000
274/274 [========================

...

274/274 [==============================] - 0s 100us/step - loss: 3.9596 - val_loss: 10.5487
Epoch 764/1000
274/274 [==============================] - 0s 97us/step - loss: 3.9825 - val_loss: 10.4311
Epoch 765/1000
274/274 [==============================] - 0s 102us/step - loss: 4.0018 - val_loss: 10.3198
Epoch 766/1000
274/274 [==============================] - 0s 102us/step - loss: 3.9364 - val_loss: 10.6699
Epoch 767/1000
274/274 [==============================] - 0s 100us/step - loss: 4.1773 - val_loss: 10.5603
Epoch 768/1000
274/274 [==============================] - 0s 97us/step - loss: 4.6772 - val_loss: 11.7385
Epoch 769/1000
274/274 [==============================] - 0s 131us/step - loss: 4.3667 - val_loss: 11.1301
Epoch 770/1000
274/274 [==============================] - 0s 109us/step - loss: 4.2853 - val_loss: 11.1078
Epoch 771/1000
274/274 [==============================] - 0s 101us/step - loss: 4.1061 - val_loss: 10.7172
Epoch 772/1000
274/274 [==============================

274/274 [==============================] - 0s 102us/step - loss: 4.1776 - val_loss: 10.9921
Epoch 841/1000
274/274 [==============================] - 0s 109us/step - loss: 4.1452 - val_loss: 10.9293
Epoch 842/1000
274/274 [==============================] - 0s 127us/step - loss: 3.9378 - val_loss: 10.9285
Epoch 843/1000
274/274 [==============================] - 0s 124us/step - loss: 3.9324 - val_loss: 10.8874
Epoch 844/1000
274/274 [==============================] - 0s 113us/step - loss: 4.0852 - val_loss: 11.3629
Epoch 845/1000
274/274 [==============================] - 0s 120us/step - loss: 3.8441 - val_loss: 10.9106
Epoch 846/1000
274/274 [==============================] - 0s 113us/step - loss: 4.0601 - val_loss: 10.7395
Epoch 847/1000
274/274 [==============================] - 0s 113us/step - loss: 4.0264 - val_loss: 10.4928
Epoch 848/1000
274/274 [==============================] - 0s 115us/step - loss: 3.8326 - val_loss: 10.7362
Epoch 849/1000
274/274 [============================

Epoch 917/1000
274/274 [==============================] - 0s 102us/step - loss: 4.0515 - val_loss: 11.3256
Epoch 918/1000
274/274 [==============================] - 0s 94us/step - loss: 4.3801 - val_loss: 11.3889
Epoch 919/1000
274/274 [==============================] - 0s 102us/step - loss: 4.1810 - val_loss: 11.5525
Epoch 920/1000
274/274 [==============================] - 0s 101us/step - loss: 4.0436 - val_loss: 11.1889
Epoch 921/1000
274/274 [==============================] - 0s 99us/step - loss: 3.8888 - val_loss: 11.2211
Epoch 922/1000
274/274 [==============================] - 0s 98us/step - loss: 3.8534 - val_loss: 10.9472
Epoch 923/1000
274/274 [==============================] - 0s 96us/step - loss: 4.0277 - val_loss: 11.1284
Epoch 924/1000
274/274 [==============================] - 0s 95us/step - loss: 3.9411 - val_loss: 11.1117
Epoch 925/1000
274/274 [==============================] - 0s 98us/step - loss: 3.8441 - val_loss: 11.1721
Epoch 926/1000
274/274 [===================

Epoch 994/1000
274/274 [==============================] - 0s 103us/step - loss: 3.8443 - val_loss: 11.3694
Epoch 995/1000
274/274 [==============================] - 0s 98us/step - loss: 3.8624 - val_loss: 11.5985
Epoch 996/1000
274/274 [==============================] - 0s 102us/step - loss: 3.8490 - val_loss: 11.4366
Epoch 997/1000
274/274 [==============================] - 0s 101us/step - loss: 3.8170 - val_loss: 11.3329
Epoch 998/1000
274/274 [==============================] - 0s 98us/step - loss: 3.7863 - val_loss: 11.3269
Epoch 999/1000
274/274 [==============================] - 0s 98us/step - loss: 3.8335 - val_loss: 11.4319
Epoch 1000/1000
274/274 [==============================] - 0s 95us/step - loss: 3.9774 - val_loss: 11.7863
7.638522034984524


[array([[ 2.3485718 ,  3.437333  , -1.0346681 , -1.11541   , -0.39312732,
         -0.4332369 ,  1.2525725 , -2.305369  ,  0.29607502,  1.1767436 ],
        [-2.8701944 , -1.1326944 ,  1.3748207 ,  0.8238783 ,  0.25929376,
         -0.62357056,  2.1771872 ,  1.0054055 , -2.0650096 ,  1.9648839 ],
        [ 0.35068494, -2.829381  , -0.8547602 ,  0.27258775,  2.0103989 ,
          0.55739146,  0.11665559, -0.56137866, -1.226192  , -0.8768354 ],
        [ 2.9578562 , -1.196214  ,  0.64882755, -0.99563617,  1.0634879 ,
          1.1656897 , -0.6445601 , -0.4994563 ,  2.0238454 , -1.9855816 ],
        [-1.48828   , -0.2082753 ,  0.38139126, -0.31505525,  0.1825463 ,
          0.00885664,  0.5097318 , -0.14638576, -0.12460362,  0.19514143],
        [-1.2721285 ,  0.11190529, -0.20485285,  1.4725543 , -3.2043078 ,
         -1.7638916 , -2.9814444 ,  1.3098832 ,  1.773347  ,  0.52337813],
        [ 0.20947824, -0.7044808 , -0.63588274,  1.6007673 ,  1.722647  ,
         -1.0264131 , -0.4418670

In [47]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure5_2nd.h5')

In [48]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_6(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 10)                80        
_________________________________________________________________
activation_23 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_35 (Dense)             (None, 15)                165       
_________________________________________________________________
activation_24 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 16        
Total params: 261
Trainable params: 261
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 1ms/step - loss: 

Epoch 146/1000
274/274 [==============================] - 0s 101us/step - loss: 5.0340 - val_loss: 9.1894
Epoch 147/1000
274/274 [==============================] - 0s 106us/step - loss: 4.9609 - val_loss: 9.2438
Epoch 148/1000
274/274 [==============================] - 0s 98us/step - loss: 4.9536 - val_loss: 10.0652
Epoch 149/1000
274/274 [==============================] - 0s 100us/step - loss: 4.9702 - val_loss: 9.6150
Epoch 150/1000
274/274 [==============================] - 0s 98us/step - loss: 5.0117 - val_loss: 10.3622
Epoch 151/1000
274/274 [==============================] - 0s 95us/step - loss: 5.0213 - val_loss: 9.4448
Epoch 152/1000
274/274 [==============================] - 0s 100us/step - loss: 5.2346 - val_loss: 9.5990
Epoch 153/1000
274/274 [==============================] - 0s 102us/step - loss: 4.9507 - val_loss: 9.3637
Epoch 154/1000
274/274 [==============================] - 0s 97us/step - loss: 4.8615 - val_loss: 9.7265
Epoch 155/1000
274/274 [========================

274/274 [==============================] - 0s 98us/step - loss: 4.6093 - val_loss: 10.2927
Epoch 301/1000
274/274 [==============================] - 0s 102us/step - loss: 4.9127 - val_loss: 9.6568
Epoch 302/1000
274/274 [==============================] - 0s 102us/step - loss: 4.9229 - val_loss: 10.1451
Epoch 303/1000
274/274 [==============================] - 0s 101us/step - loss: 4.6673 - val_loss: 9.6211
Epoch 304/1000
274/274 [==============================] - 0s 101us/step - loss: 4.3005 - val_loss: 10.3397
Epoch 305/1000
274/274 [==============================] - 0s 99us/step - loss: 4.3368 - val_loss: 9.7183
Epoch 306/1000
274/274 [==============================] - 0s 102us/step - loss: 4.2903 - val_loss: 9.7470
Epoch 307/1000
274/274 [==============================] - 0s 102us/step - loss: 4.2156 - val_loss: 10.0457
Epoch 308/1000
274/274 [==============================] - 0s 102us/step - loss: 4.1904 - val_loss: 9.8168
Epoch 309/1000
274/274 [==============================] - 0

Epoch 454/1000
274/274 [==============================] - 0s 113us/step - loss: 3.8758 - val_loss: 10.2558
Epoch 455/1000
274/274 [==============================] - 0s 106us/step - loss: 4.1210 - val_loss: 10.3487
Epoch 456/1000
274/274 [==============================] - 0s 106us/step - loss: 4.0374 - val_loss: 10.9622
Epoch 457/1000
274/274 [==============================] - 0s 106us/step - loss: 4.3973 - val_loss: 9.8555
Epoch 458/1000
274/274 [==============================] - 0s 106us/step - loss: 4.0488 - val_loss: 10.4038
Epoch 459/1000
274/274 [==============================] - 0s 106us/step - loss: 4.1209 - val_loss: 11.2876
Epoch 460/1000
274/274 [==============================] - 0s 102us/step - loss: 3.9448 - val_loss: 10.8651
Epoch 461/1000
274/274 [==============================] - 0s 116us/step - loss: 5.4078 - val_loss: 12.8986
Epoch 462/1000
274/274 [==============================] - 0s 105us/step - loss: 4.4373 - val_loss: 10.6714
Epoch 463/1000
274/274 [==============

Epoch 607/1000
274/274 [==============================] - 0s 99us/step - loss: 4.1432 - val_loss: 12.3640
Epoch 608/1000
274/274 [==============================] - 0s 107us/step - loss: 4.0119 - val_loss: 10.6758
Epoch 609/1000
274/274 [==============================] - 0s 94us/step - loss: 3.6846 - val_loss: 10.9994
Epoch 610/1000
274/274 [==============================] - 0s 95us/step - loss: 3.8513 - val_loss: 11.5600
Epoch 611/1000
274/274 [==============================] - 0s 98us/step - loss: 4.0204 - val_loss: 11.9968
Epoch 612/1000
274/274 [==============================] - 0s 102us/step - loss: 4.2423 - val_loss: 10.6691
Epoch 613/1000
274/274 [==============================] - 0s 97us/step - loss: 4.1389 - val_loss: 10.6382
Epoch 614/1000
274/274 [==============================] - 0s 98us/step - loss: 3.7621 - val_loss: 11.7662
Epoch 615/1000
274/274 [==============================] - 0s 106us/step - loss: 3.7341 - val_loss: 10.6983
Epoch 616/1000
274/274 [===================

Epoch 684/1000
274/274 [==============================] - 0s 96us/step - loss: 3.9320 - val_loss: 10.8083
Epoch 685/1000
274/274 [==============================] - 0s 100us/step - loss: 3.8308 - val_loss: 13.4431
Epoch 686/1000
274/274 [==============================] - 0s 101us/step - loss: 4.4885 - val_loss: 10.8176
Epoch 687/1000
274/274 [==============================] - 0s 106us/step - loss: 4.1774 - val_loss: 11.0359
Epoch 688/1000
274/274 [==============================] - 0s 98us/step - loss: 3.8579 - val_loss: 10.9697
Epoch 689/1000
274/274 [==============================] - 0s 98us/step - loss: 3.7351 - val_loss: 11.1712
Epoch 690/1000
274/274 [==============================] - 0s 98us/step - loss: 3.6363 - val_loss: 11.6623
Epoch 691/1000
274/274 [==============================] - 0s 99us/step - loss: 3.7531 - val_loss: 11.2172
Epoch 692/1000
274/274 [==============================] - 0s 103us/step - loss: 3.8164 - val_loss: 10.8230
Epoch 693/1000
274/274 [==================

274/274 [==============================] - 0s 98us/step - loss: 4.1515 - val_loss: 10.4842
Epoch 838/1000
274/274 [==============================] - 0s 98us/step - loss: 3.8572 - val_loss: 11.2537
Epoch 839/1000
274/274 [==============================] - 0s 101us/step - loss: 3.7744 - val_loss: 12.8360
Epoch 840/1000
274/274 [==============================] - 0s 102us/step - loss: 3.9729 - val_loss: 10.9312
Epoch 841/1000
274/274 [==============================] - 0s 106us/step - loss: 3.5291 - val_loss: 11.5649
Epoch 842/1000
274/274 [==============================] - 0s 109us/step - loss: 3.5626 - val_loss: 11.0101
Epoch 843/1000
274/274 [==============================] - 0s 119us/step - loss: 3.6645 - val_loss: 11.3338
Epoch 844/1000
274/274 [==============================] - 0s 106us/step - loss: 3.6323 - val_loss: 11.2983
Epoch 845/1000
274/274 [==============================] - 0s 113us/step - loss: 3.5609 - val_loss: 11.8530
Epoch 846/1000
274/274 [==============================

Epoch 914/1000
274/274 [==============================] - 0s 120us/step - loss: 3.5228 - val_loss: 11.6186
Epoch 915/1000
274/274 [==============================] - 0s 116us/step - loss: 3.7746 - val_loss: 11.1980
Epoch 916/1000
274/274 [==============================] - 0s 120us/step - loss: 3.9242 - val_loss: 10.6387
Epoch 917/1000
274/274 [==============================] - 0s 109us/step - loss: 3.8344 - val_loss: 14.0475
Epoch 918/1000
274/274 [==============================] - 0s 120us/step - loss: 4.0842 - val_loss: 10.9821
Epoch 919/1000
274/274 [==============================] - 0s 113us/step - loss: 3.6502 - val_loss: 11.0763
Epoch 920/1000
274/274 [==============================] - 0s 109us/step - loss: 3.5723 - val_loss: 11.1523
Epoch 921/1000
274/274 [==============================] - 0s 109us/step - loss: 3.5980 - val_loss: 12.3213
Epoch 922/1000
274/274 [==============================] - 0s 113us/step - loss: 3.5960 - val_loss: 10.6400
Epoch 923/1000
274/274 [=============

Epoch 991/1000
274/274 [==============================] - 0s 102us/step - loss: 4.2650 - val_loss: 11.2564
Epoch 992/1000
274/274 [==============================] - 0s 95us/step - loss: 4.1977 - val_loss: 12.5269
Epoch 993/1000
274/274 [==============================] - 0s 101us/step - loss: 4.4979 - val_loss: 10.8023
Epoch 994/1000
274/274 [==============================] - 0s 101us/step - loss: 4.1876 - val_loss: 10.3173
Epoch 995/1000
274/274 [==============================] - 0s 102us/step - loss: 3.8070 - val_loss: 11.1629
Epoch 996/1000
274/274 [==============================] - 0s 103us/step - loss: 3.4901 - val_loss: 11.1088
Epoch 997/1000
274/274 [==============================] - 0s 106us/step - loss: 3.5817 - val_loss: 10.6880
Epoch 998/1000
274/274 [==============================] - 0s 98us/step - loss: 3.7846 - val_loss: 11.0440
Epoch 999/1000
274/274 [==============================] - 0s 98us/step - loss: 3.5008 - val_loss: 10.8611
Epoch 1000/1000
274/274 [===============

[array([[ 1.1809119 ,  0.05248419, -0.4461234 , -0.48930568, -1.6214685 ,
          1.8095565 ,  1.7396572 , -5.5524507 , -0.23961815,  0.2033678 ],
        [-0.8258639 ,  1.1540966 ,  0.55125743, -0.83661705,  0.14821894,
         -1.2706293 ,  1.3272252 ,  1.2854592 , -1.9795737 ,  2.422064  ],
        [ 0.6645644 ,  0.28592727, -0.6108597 ,  0.80949426,  0.27631927,
          0.5237329 , -0.43020216, -4.579145  , -2.84042   ,  1.2264276 ],
        [ 1.6136824 , -1.1411405 , -0.7480782 ,  0.6712501 ,  0.7271336 ,
          0.12609679, -1.7450751 , -1.4833832 ,  0.50500077, -1.4211518 ],
        [-0.8509135 ,  0.84900045,  0.13964376,  0.8328872 ,  0.07463697,
         -0.13533783,  0.28014556,  0.6807951 , -0.29070285,  0.58345723],
        [-1.5723346 , -0.42963967,  1.6803507 ,  0.50648135, -0.32829776,
         -0.5607675 , -2.9704347 , -0.33758435, -0.22591737,  0.8456348 ],
        [-0.8142687 , -1.2406414 ,  0.12384932, -2.5659118 ,  0.4395675 ,
          0.7164058 , -1.9186012

In [49]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure6_2nd.h5')

In [50]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_1(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_25 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_38 (Dense)             (None, 5)                 30        
_________________________________________________________________
activation_26 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_39 (Dense)             (None, 1)                 6         
Total params: 151
Trainable params: 151
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 117us/step - loss: 0.0068 - val_loss: 0.0066
Epoch 70/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0067 - val_loss: 0.0054
Epoch 71/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0067 - val_loss: 0.0057
Epoch 72/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0066 - val_loss: 0.0053
Epoch 73/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0066 - val_loss: 0.0054
Epoch 74/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0066 - val_loss: 0.0054
Epoch 75/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0064 - val_loss: 0.0055
Epoch 76/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0064 - val_loss: 0.0053
Epoch 77/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0065 - val_loss: 0.0052
Epoch 78/1000
143/143 [==============================] - 0s 119us/ste

143/143 [==============================] - 0s 119us/step - loss: 0.0049 - val_loss: 0.0041
Epoch 224/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0056 - val_loss: 0.0045
Epoch 225/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0050 - val_loss: 0.0040
Epoch 226/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0052 - val_loss: 0.0045
Epoch 227/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0051 - val_loss: 0.0040
Epoch 228/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0053 - val_loss: 0.0046
Epoch 229/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0053 - val_loss: 0.0050
Epoch 230/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0052 - val_loss: 0.0044
Epoch 231/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0054 - val_loss: 0.0051
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 118us/step - loss: 0.0045 - val_loss: 0.0039
Epoch 378/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0045 - val_loss: 0.0037
Epoch 379/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0046 - val_loss: 0.0039
Epoch 380/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0045 - val_loss: 0.0037
Epoch 381/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0045 - val_loss: 0.0040
Epoch 382/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0046 - val_loss: 0.0035
Epoch 383/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0044 - val_loss: 0.0046
Epoch 384/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0048 - val_loss: 0.0037
Epoch 385/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0045 - val_loss: 0.0040
Epoch 386/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 118us/step - loss: 0.0047 - val_loss: 0.0030
Epoch 532/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0042 - val_loss: 0.0038
Epoch 533/1000
143/143 [==============================] - 0s 120us/step - loss: 0.0043 - val_loss: 0.0030
Epoch 534/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0042 - val_loss: 0.0034
Epoch 535/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0041 - val_loss: 0.0032
Epoch 536/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0047 - val_loss: 0.0039
Epoch 537/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0046 - val_loss: 0.0031
Epoch 538/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0043 - val_loss: 0.0036
Epoch 539/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0052 - val_loss: 0.0032
Epoch 540/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 126us/step - loss: 0.0035 - val_loss: 0.0026
Epoch 686/1000
143/143 [==============================] - 0s 146us/step - loss: 0.0035 - val_loss: 0.0025
Epoch 687/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0036 - val_loss: 0.0026
Epoch 688/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0037 - val_loss: 0.0034
Epoch 689/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0037 - val_loss: 0.0025
Epoch 690/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0035 - val_loss: 0.0037
Epoch 691/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0036 - val_loss: 0.0024
Epoch 692/1000
143/143 [==============================] - 0s 132us/step - loss: 0.0037 - val_loss: 0.0036
Epoch 693/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0035 - val_loss: 0.0024
Epoch 694/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0029 - val_loss: 0.0035
Epoch 840/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0028 - val_loss: 0.0023
Epoch 841/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0028 - val_loss: 0.0035
Epoch 842/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0026 - val_loss: 0.0023
Epoch 843/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0029 - val_loss: 0.0039
Epoch 844/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0028 - val_loss: 0.0025
Epoch 845/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0028 - val_loss: 0.0034
Epoch 846/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0033 - val_loss: 0.0042
Epoch 847/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0037 - val_loss: 0.0025
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0024 - val_loss: 0.0031
Epoch 994/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0025
Epoch 995/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0024 - val_loss: 0.0029
Epoch 996/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0023 - val_loss: 0.0029
Epoch 997/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0023 - val_loss: 0.0030
Epoch 998/1000
143/143 [==============================] - 0s 122us/step - loss: 0.0023 - val_loss: 0.0027
Epoch 999/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0024 - val_loss: 0.0045
Epoch 1000/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0029 - val_loss: 0.0028
0.011725426651537418


[array([[-1.0138961 , -0.23617822,  0.6278258 ,  0.22500198,  0.46470216],
        [-0.59554577,  0.3107354 , -0.7839685 ,  0.1192729 ,  0.6189596 ],
        [-0.44013596,  0.31653598, -0.85171247,  0.01691079, -0.2820443 ],
        [-0.5190926 , -0.02201221, -0.11430769,  0.37365848,  1.2335027 ],
        [-0.22187409,  0.03705249, -0.34099478, -0.02803409, -0.09494247],
        [-0.7632476 , -0.02111059,  0.5061534 ,  0.14301753,  0.45890406],
        [-0.3176996 , -0.1022935 , -0.3648605 , -0.7576992 , -0.15210307],
        [-0.1030855 ,  0.01438237,  0.05054303,  0.33922064,  0.20812123],
        [-0.5256499 ,  0.18947607,  0.33318004, -0.11476175, -0.3557342 ],
        [-0.9234985 , -0.4002752 ,  0.13668562, -0.50796765,  0.48396426],
        [-0.45526153,  0.16595559, -0.38524875, -0.93099993, -0.06124977],
        [-0.25857064,  0.29217872, -0.42656755,  0.17636001,  0.47156373],
        [-0.71443075,  0.14756984, -0.33939964, -0.05902457,  0.13912672],
        [-0.77061903,  1.

In [51]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure1_2nd.h5')

In [52]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_2(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_27 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_41 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_28 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_42 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 115us/step - loss: 0.0076 - val_loss: 0.0044
Epoch 70/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0077 - val_loss: 0.0042
Epoch 71/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0070 - val_loss: 0.0044
Epoch 72/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0071 - val_loss: 0.0044
Epoch 73/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0072 - val_loss: 0.0045
Epoch 74/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0070 - val_loss: 0.0045
Epoch 75/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0071 - val_loss: 0.0046
Epoch 76/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0068 - val_loss: 0.0048
Epoch 77/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0070 - val_loss: 0.0044
Epoch 78/1000
143/143 [==============================] - 0s 118us/ste

143/143 [==============================] - 0s 111us/step - loss: 0.0046 - val_loss: 0.0055
Epoch 224/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0050 - val_loss: 0.0069
Epoch 225/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0049 - val_loss: 0.0060
Epoch 226/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0049 - val_loss: 0.0052
Epoch 227/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0045 - val_loss: 0.0059
Epoch 228/1000
143/143 [==============================] - 0s 101us/step - loss: 0.0045 - val_loss: 0.0053
Epoch 229/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0045 - val_loss: 0.0056
Epoch 230/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0051 - val_loss: 0.0066
Epoch 231/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0053 - val_loss: 0.0071
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 126us/step - loss: 0.0028 - val_loss: 0.0066
Epoch 378/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0035 - val_loss: 0.0053
Epoch 379/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0030 - val_loss: 0.0051
Epoch 380/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0034 - val_loss: 0.0081
Epoch 381/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0037 - val_loss: 0.0055
Epoch 382/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0030 - val_loss: 0.0061
Epoch 383/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0030 - val_loss: 0.0056
Epoch 384/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0029 - val_loss: 0.0048
Epoch 385/1000
143/143 [==============================] - 0s 122us/step - loss: 0.0028 - val_loss: 0.0054
Epoch 386/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 117us/step - loss: 0.0029 - val_loss: 0.0051
Epoch 532/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0028 - val_loss: 0.0084
Epoch 533/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0037 - val_loss: 0.0048
Epoch 534/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0028 - val_loss: 0.0042
Epoch 535/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0022 - val_loss: 0.0049
Epoch 536/1000
143/143 [==============================] - 0s 121us/step - loss: 0.0021 - val_loss: 0.0057
Epoch 537/1000
143/143 [==============================] - 0s 121us/step - loss: 0.0022 - val_loss: 0.0052
Epoch 538/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0022 - val_loss: 0.0042
Epoch 539/1000
143/143 [==============================] - 0s 146us/step - loss: 0.0026 - val_loss: 0.0048
Epoch 540/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 115us/step - loss: 0.0019 - val_loss: 0.0044
Epoch 686/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0021 - val_loss: 0.0075
Epoch 687/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0021 - val_loss: 0.0047
Epoch 688/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0019 - val_loss: 0.0044
Epoch 689/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0019 - val_loss: 0.0069
Epoch 690/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0025 - val_loss: 0.0047
Epoch 691/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0026 - val_loss: 0.0046
Epoch 692/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0022 - val_loss: 0.0048
Epoch 693/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0024 - val_loss: 0.0051
Epoch 694/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0062
Epoch 840/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0018 - val_loss: 0.0059
Epoch 841/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0018 - val_loss: 0.0050
Epoch 842/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0017 - val_loss: 0.0048
Epoch 843/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0019 - val_loss: 0.0055
Epoch 844/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0018 - val_loss: 0.0056
Epoch 845/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0017 - val_loss: 0.0056
Epoch 846/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0016 - val_loss: 0.0063
Epoch 847/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0017 - val_loss: 0.0050
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 116us/step - loss: 0.0015 - val_loss: 0.0061
Epoch 994/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0018 - val_loss: 0.0051
Epoch 995/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0015 - val_loss: 0.0045
Epoch 996/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0017 - val_loss: 0.0057
Epoch 997/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0017 - val_loss: 0.0057
Epoch 998/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0018 - val_loss: 0.0049
Epoch 999/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0022 - val_loss: 0.0070
Epoch 1000/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0019 - val_loss: 0.0049
0.015014992095530033


[array([[-0.5736573 ,  0.1265544 , -0.47902927,  0.61840904, -0.50585866],
        [-0.683833  ,  0.8060595 , -0.021943  , -0.3240458 ,  0.05592718],
        [ 0.45083618, -1.0342698 ,  0.706376  , -0.13129845,  0.15516737],
        [-1.6272156 , -0.06322861,  0.3655931 ,  0.0433594 ,  0.0223134 ],
        [-0.00631125, -1.6157868 ,  0.56717384, -0.25213358, -0.28233984],
        [ 0.25117785, -0.553465  , -0.6082802 ,  0.11214534, -1.6101968 ],
        [ 0.3234074 , -0.05128997,  0.25775555,  0.05939157,  0.78603673],
        [ 0.29637182,  0.30213124, -0.07114127, -0.3336933 , -0.15259276],
        [ 0.37869915, -1.1231941 ,  0.29945993,  0.21226929,  0.00956837],
        [-0.33944264,  0.96623266, -0.68134356,  0.03110527, -0.02534997],
        [ 0.15359375,  0.5271991 ,  0.20085715, -1.0992467 ,  0.55244654],
        [-0.58744514,  1.060027  , -0.18913722, -0.07728682, -0.52706   ],
        [-0.00441534,  0.7454373 , -0.0794742 ,  0.06878286, -0.90017486],
        [-0.6685028 , -0.

In [53]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure2_2nd.h5')

In [54]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_3(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_43 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_29 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_44 (Dense)             (None, 15)                90        
_________________________________________________________________
activation_30 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_45 (Dense)             (None, 1)                 16        
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 119us/step - loss: 0.0075 - val_loss: 0.0053
Epoch 70/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0073 - val_loss: 0.0053
Epoch 71/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0073 - val_loss: 0.0058
Epoch 72/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0076 - val_loss: 0.0052
Epoch 73/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0075 - val_loss: 0.0065
Epoch 74/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0077 - val_loss: 0.0054
Epoch 75/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0079 - val_loss: 0.0054
Epoch 76/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0076 - val_loss: 0.0054
Epoch 77/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0073 - val_loss: 0.0054
Epoch 78/1000
143/143 [==============================] - 0s 120us/ste

143/143 [==============================] - 0s 112us/step - loss: 0.0055 - val_loss: 0.0051
Epoch 224/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0055 - val_loss: 0.0051
Epoch 225/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0055 - val_loss: 0.0049
Epoch 226/1000
143/143 [==============================] - 0s 130us/step - loss: 0.0056 - val_loss: 0.0054
Epoch 227/1000
143/143 [==============================] - 0s 121us/step - loss: 0.0054 - val_loss: 0.0053
Epoch 228/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0056 - val_loss: 0.0056
Epoch 229/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0055 - val_loss: 0.0054
Epoch 230/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0055 - val_loss: 0.0053
Epoch 231/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0058 - val_loss: 0.0049
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0042 - val_loss: 0.0046
Epoch 378/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0041 - val_loss: 0.0044
Epoch 379/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0044 - val_loss: 0.0075
Epoch 380/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0058 - val_loss: 0.0053
Epoch 381/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0053 - val_loss: 0.0048
Epoch 382/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0043 - val_loss: 0.0041
Epoch 383/1000
143/143 [==============================] - 0s 147us/step - loss: 0.0041 - val_loss: 0.0042
Epoch 384/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0043 - val_loss: 0.0048
Epoch 385/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0041 - val_loss: 0.0043
Epoch 386/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 114us/step - loss: 0.0023 - val_loss: 0.0047
Epoch 532/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0021 - val_loss: 0.0046
Epoch 533/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0022 - val_loss: 0.0050
Epoch 534/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0022 - val_loss: 0.0045
Epoch 535/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0020 - val_loss: 0.0049
Epoch 536/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0021 - val_loss: 0.0044
Epoch 537/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0020 - val_loss: 0.0066
Epoch 538/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0027 - val_loss: 0.0046
Epoch 539/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0026 - val_loss: 0.0047
Epoch 540/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0018 - val_loss: 0.0039
Epoch 686/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0020 - val_loss: 0.0045
Epoch 687/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0017 - val_loss: 0.0044
Epoch 688/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0018 - val_loss: 0.0042
Epoch 689/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0018 - val_loss: 0.0045
Epoch 690/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0019 - val_loss: 0.0049
Epoch 691/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0018 - val_loss: 0.0047
Epoch 692/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0017 - val_loss: 0.0042
Epoch 693/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0017 - val_loss: 0.0046
Epoch 694/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 111us/step - loss: 0.0016 - val_loss: 0.0042
Epoch 840/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0015 - val_loss: 0.0047
Epoch 841/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0015 - val_loss: 0.0041
Epoch 842/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0017 - val_loss: 0.0040
Epoch 843/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0016 - val_loss: 0.0042
Epoch 844/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0014 - val_loss: 0.0044
Epoch 845/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0014 - val_loss: 0.0045
Epoch 846/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0016 - val_loss: 0.0039
Epoch 847/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0015 - val_loss: 0.0042
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 113us/step - loss: 0.0019 - val_loss: 0.0059
Epoch 994/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0019 - val_loss: 0.0045
Epoch 995/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0018 - val_loss: 0.0043
Epoch 996/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0019 - val_loss: 0.0039
Epoch 997/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0015 - val_loss: 0.0043
Epoch 998/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0016 - val_loss: 0.0045
Epoch 999/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0017 - val_loss: 0.0041
Epoch 1000/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0020 - val_loss: 0.0050
0.009791904129087925


[array([[-0.5255522 , -0.09260228, -0.49330962, -0.9216876 , -0.5651125 ],
        [-0.03541639,  0.40660682,  1.1710927 , -0.17194907, -0.1852446 ],
        [ 0.36717063, -0.9793008 ,  0.3815718 , -0.07199725, -0.2897665 ],
        [ 0.12579791, -0.557407  ,  0.3072762 , -1.109443  ,  0.31677562],
        [ 0.1355897 , -0.25938445, -0.1403504 ,  0.04746757, -0.03559031],
        [-0.7056149 ,  0.806231  ,  0.31346   , -0.34633514, -0.40193793],
        [-0.38638026,  0.17395428,  0.35183176,  0.02083181,  0.02005108],
        [-0.3674216 , -0.37435734, -0.06876706, -0.6989836 ,  0.5061537 ],
        [ 0.17992376, -0.10178285, -0.7727435 ,  0.5489126 ,  0.8463981 ],
        [-0.16879638,  0.19843319, -0.07119418, -0.27431184, -0.1200474 ],
        [ 0.814577  ,  0.14653325,  0.15072307,  0.8478684 ,  0.61418843],
        [ 1.1191216 ,  0.295338  ,  0.6228138 ,  0.4253674 ,  0.19461577],
        [ 0.5305401 ,  0.54548115,  1.1093385 , -0.06105691, -0.4524011 ],
        [ 0.41338006,  2.

In [55]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure3_2nd.h5')

In [56]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_4(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_46 (Dense)             (None, 10)                230       
_________________________________________________________________
activation_31 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_47 (Dense)             (None, 5)                 55        
_________________________________________________________________
activation_32 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_48 (Dense)             (None, 1)                 6         
Total params: 291
Trainable params: 291
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 119us/step - loss: 0.0064 - val_loss: 0.0046
Epoch 70/1000
143/143 [==============================] - 0s 123us/step - loss: 0.0062 - val_loss: 0.0042
Epoch 71/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0066 - val_loss: 0.0044
Epoch 72/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0062 - val_loss: 0.0046
Epoch 73/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0060 - val_loss: 0.0043
Epoch 74/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0059 - val_loss: 0.0045
Epoch 75/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0059 - val_loss: 0.0043
Epoch 76/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0059 - val_loss: 0.0044
Epoch 77/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0060 - val_loss: 0.0041
Epoch 78/1000
143/143 [==============================] - 0s 119us/ste

143/143 [==============================] - 0s 118us/step - loss: 0.0037 - val_loss: 0.0029
Epoch 224/1000
143/143 [==============================] - 0s 120us/step - loss: 0.0037 - val_loss: 0.0031
Epoch 225/1000
143/143 [==============================] - 0s 124us/step - loss: 0.0035 - val_loss: 0.0029
Epoch 226/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0034 - val_loss: 0.0055
Epoch 227/1000
143/143 [==============================] - 0s 146us/step - loss: 0.0032 - val_loss: 0.0033
Epoch 228/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0033 - val_loss: 0.0043
Epoch 229/1000
143/143 [==============================] - 0s 132us/step - loss: 0.0028 - val_loss: 0.0029
Epoch 230/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0028 - val_loss: 0.0030
Epoch 231/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0026 - val_loss: 0.0029
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 98us/step - loss: 0.0020 - val_loss: 0.0019
Epoch 378/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0018 - val_loss: 0.0024
Epoch 379/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0018 - val_loss: 0.0023
Epoch 380/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0018 - val_loss: 0.0019
Epoch 381/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0021 - val_loss: 0.0023
Epoch 382/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0018 - val_loss: 0.0022
Epoch 383/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0018 - val_loss: 0.0037
Epoch 384/1000
143/143 [==============================] - 0s 120us/step - loss: 0.0020 - val_loss: 0.0019
Epoch 385/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0020 - val_loss: 0.0022
Epoch 386/1000
143/143 [==============================] - 0s 11

143/143 [==============================] - 0s 112us/step - loss: 0.0017 - val_loss: 0.0034
Epoch 532/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0018 - val_loss: 0.0018
Epoch 533/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0017 - val_loss: 0.0019
Epoch 534/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0016 - val_loss: 0.0022
Epoch 535/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0018 - val_loss: 0.0033
Epoch 536/1000
143/143 [==============================] - 0s 96us/step - loss: 0.0019 - val_loss: 0.0021
Epoch 537/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0020 - val_loss: 0.0018
Epoch 538/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0018 - val_loss: 0.0017
Epoch 539/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0021 - val_loss: 0.0037
Epoch 540/1000
143/143 [==============================] - 0s 12

143/143 [==============================] - 0s 119us/step - loss: 0.0018 - val_loss: 0.0021
Epoch 686/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0019 - val_loss: 0.0017
Epoch 687/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0021 - val_loss: 0.0025
Epoch 688/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0018 - val_loss: 0.0022
Epoch 689/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0016 - val_loss: 0.0014
Epoch 690/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0016 - val_loss: 0.0028
Epoch 691/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0016 - val_loss: 0.0018
Epoch 692/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0020
Epoch 693/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0014 - val_loss: 0.0021
Epoch 694/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0014 - val_loss: 0.0019
Epoch 840/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0021 - val_loss: 0.0030
Epoch 841/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0016 - val_loss: 0.0031
Epoch 842/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0018 - val_loss: 0.0018
Epoch 843/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0029 - val_loss: 0.0026
Epoch 844/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0027 - val_loss: 0.0049
Epoch 845/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0025 - val_loss: 0.0019
Epoch 846/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0020 - val_loss: 0.0020
Epoch 847/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0019 - val_loss: 0.0029
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0013 - val_loss: 0.0035
Epoch 994/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0012 - val_loss: 0.0020
Epoch 995/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0013 - val_loss: 0.0022
Epoch 996/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0013 - val_loss: 0.0030
Epoch 997/1000
143/143 [==============================] - 0s 124us/step - loss: 0.0012 - val_loss: 0.0022
Epoch 998/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0012 - val_loss: 0.0023
Epoch 999/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0011 - val_loss: 0.0034
Epoch 1000/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0013 - val_loss: 0.0029
0.007169807329773903


[array([[ 0.14152452, -0.18154316, -0.5027929 , -1.0379568 , -0.33447295,
         -0.359882  ,  0.31829327, -0.59946203, -0.73631847, -0.12072644],
        [ 0.09193673,  0.41734576, -0.57767457, -1.3535154 , -0.34632316,
          0.2547017 , -0.30201617, -0.43302813,  1.0867004 ,  0.12813191],
        [ 0.00857275, -0.4601806 , -0.6179865 ,  0.20987892, -0.4366662 ,
         -0.07716192, -0.01325955, -0.0600386 ,  0.57893157, -0.43296665],
        [-0.01223801,  0.45243114, -0.26250425,  1.3320085 , -0.19800974,
         -0.3582311 ,  0.8764556 ,  0.26378852,  0.39583528, -0.25944135],
        [ 0.03666883, -0.07855121, -0.33102754, -0.4455303 , -0.12275294,
          0.27266353, -0.14822869, -0.02552002, -0.02837286,  0.35114753],
        [ 0.38372484,  0.6373168 , -0.655392  ,  0.12057621, -0.8310448 ,
          0.3079887 , -0.9029526 , -1.1841974 ,  0.1852856 , -0.18111914],
        [-0.07108203,  0.33087465, -0.28488982,  0.4797545 , -0.30858007,
          0.17835045,  0.5279162

In [57]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure4_2nd.h5')

In [58]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_5(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_49 (Dense)             (None, 10)                230       
_________________________________________________________________
activation_33 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_50 (Dense)             (None, 10)                110       
_________________________________________________________________
activation_34 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_51 (Dense)             (None, 1)                 11        
Total params: 351
Trainable params: 351
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 3ms/step - loss: 

143/143 [==============================] - 0s 133us/step - loss: 0.0061 - val_loss: 0.0052
Epoch 70/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0061 - val_loss: 0.0054
Epoch 71/1000
143/143 [==============================] - 0s 146us/step - loss: 0.0060 - val_loss: 0.0053
Epoch 72/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0061 - val_loss: 0.0053
Epoch 73/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0059 - val_loss: 0.0055
Epoch 74/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0058 - val_loss: 0.0057
Epoch 75/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0059 - val_loss: 0.0054
Epoch 76/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0059 - val_loss: 0.0052
Epoch 77/1000
143/143 [==============================] - 0s 132us/step - loss: 0.0059 - val_loss: 0.0056
Epoch 78/1000
143/143 [==============================] - 0s 126us/ste

143/143 [==============================] - 0s 116us/step - loss: 0.0033 - val_loss: 0.0053
Epoch 224/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0029 - val_loss: 0.0074
Epoch 225/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0037 - val_loss: 0.0043
Epoch 226/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0032 - val_loss: 0.0044
Epoch 227/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0040 - val_loss: 0.0092
Epoch 228/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0035 - val_loss: 0.0044
Epoch 229/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0031 - val_loss: 0.0041
Epoch 230/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0034 - val_loss: 0.0065
Epoch 231/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0029 - val_loss: 0.0046
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 125us/step - loss: 0.0021 - val_loss: 0.0050
Epoch 378/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0020 - val_loss: 0.0052
Epoch 379/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0020 - val_loss: 0.0045
Epoch 380/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0019 - val_loss: 0.0053
Epoch 381/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0019 - val_loss: 0.0042
Epoch 382/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0023 - val_loss: 0.0049
Epoch 383/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0021 - val_loss: 0.0062
Epoch 384/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0022 - val_loss: 0.0044
Epoch 385/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0020 - val_loss: 0.0040
Epoch 386/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0020 - val_loss: 0.0047
Epoch 532/1000
143/143 [==============================] - 0s 131us/step - loss: 0.0015 - val_loss: 0.0046
Epoch 533/1000
143/143 [==============================] - 0s 132us/step - loss: 0.0016 - val_loss: 0.0042
Epoch 534/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0016 - val_loss: 0.0044
Epoch 535/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0016 - val_loss: 0.0046
Epoch 536/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0016 - val_loss: 0.0045
Epoch 537/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0017 - val_loss: 0.0041
Epoch 538/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0018 - val_loss: 0.0053
Epoch 539/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0018 - val_loss: 0.0061
Epoch 540/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 130us/step - loss: 0.0020 - val_loss: 0.0046
Epoch 686/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0015 - val_loss: 0.0044
Epoch 687/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0014 - val_loss: 0.0041
Epoch 688/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0015 - val_loss: 0.0042
Epoch 689/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0018 - val_loss: 0.0047
Epoch 690/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0016 - val_loss: 0.0047
Epoch 691/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0014 - val_loss: 0.0042
Epoch 692/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0014 - val_loss: 0.0044
Epoch 693/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0017 - val_loss: 0.0055
Epoch 694/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 118us/step - loss: 0.0013 - val_loss: 0.0043
Epoch 840/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0012 - val_loss: 0.0042
Epoch 841/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0015 - val_loss: 0.0047
Epoch 842/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0014 - val_loss: 0.0045
Epoch 843/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0013 - val_loss: 0.0045
Epoch 844/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0066
Epoch 845/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0018 - val_loss: 0.0051
Epoch 846/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0016 - val_loss: 0.0049
Epoch 847/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0020 - val_loss: 0.0045
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 123us/step - loss: 0.0013 - val_loss: 0.0049
Epoch 994/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0013 - val_loss: 0.0039
Epoch 995/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0012 - val_loss: 0.0041
Epoch 996/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0010 - val_loss: 0.0044
Epoch 997/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0014 - val_loss: 0.0051
Epoch 998/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0013 - val_loss: 0.0050
Epoch 999/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0016 - val_loss: 0.0045
Epoch 1000/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0012 - val_loss: 0.0040
0.006033340003341436


[array([[-2.63109416e-01,  2.87394315e-01,  3.36530685e-01,
         -8.23357403e-02, -5.12385607e-01, -8.40016007e-01,
         -2.13714793e-01,  4.58041936e-01, -1.85280755e-01,
         -5.73321342e-01],
        [ 1.58114403e-01, -1.79817379e-01, -2.11871684e-01,
         -1.42843321e-01,  6.22609677e-03,  1.04388809e+00,
         -9.76363122e-01,  4.71190929e-01,  1.05771244e-01,
         -7.43597224e-02],
        [ 9.66557190e-02,  5.15188277e-01,  3.80168408e-01,
          2.27735594e-01,  6.14834189e-01,  5.43149054e-01,
          4.82464641e-01,  6.44063577e-02, -3.87285680e-01,
          3.48457456e-01],
        [-4.35258180e-01,  2.93920785e-01,  2.14602843e-01,
         -1.26953751e-01, -7.18626678e-01,  5.65966487e-01,
         -2.13499808e+00,  3.77359218e-03, -4.30591673e-01,
         -1.43307462e-01],
        [ 1.02253094e-01,  1.74734235e-01, -4.65976119e-01,
          2.96346936e-02, -1.39747381e-01, -1.68274596e-01,
          1.23926312e-01, -1.20168231e-01,  9.254860

In [59]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure5_2nd.h5')

In [60]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_6(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_52 (Dense)             (None, 10)                230       
_________________________________________________________________
activation_35 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_53 (Dense)             (None, 15)                165       
_________________________________________________________________
activation_36 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_54 (Dense)             (None, 1)                 16        
Total params: 411
Trainable params: 411
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 3ms/step - loss: 

143/143 [==============================] - 0s 125us/step - loss: 0.0070 - val_loss: 0.0051
Epoch 70/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0076 - val_loss: 0.0050
Epoch 71/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0072 - val_loss: 0.0066
Epoch 72/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0077 - val_loss: 0.0048
Epoch 73/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0069 - val_loss: 0.0058
Epoch 74/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0071 - val_loss: 0.0046
Epoch 75/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0069 - val_loss: 0.0050
Epoch 76/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0066 - val_loss: 0.0045
Epoch 77/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0065 - val_loss: 0.0047
Epoch 78/1000
143/143 [==============================] - 0s 112us/ste

143/143 [==============================] - 0s 126us/step - loss: 0.0047 - val_loss: 0.0041
Epoch 224/1000
143/143 [==============================] - 0s 123us/step - loss: 0.0036 - val_loss: 0.0049
Epoch 225/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0038 - val_loss: 0.0041
Epoch 226/1000
143/143 [==============================] - 0s 123us/step - loss: 0.0038 - val_loss: 0.0040
Epoch 227/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0038 - val_loss: 0.0036
Epoch 228/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0041 - val_loss: 0.0044
Epoch 229/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0037 - val_loss: 0.0039
Epoch 230/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0040 - val_loss: 0.0040
Epoch 231/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0041 - val_loss: 0.0054
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 126us/step - loss: 0.0023 - val_loss: 0.0050
Epoch 378/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0022 - val_loss: 0.0068
Epoch 379/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0026 - val_loss: 0.0041
Epoch 380/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0020 - val_loss: 0.0036
Epoch 381/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0020 - val_loss: 0.0037
Epoch 382/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0020 - val_loss: 0.0037
Epoch 383/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0023 - val_loss: 0.0045
Epoch 384/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0019 - val_loss: 0.0043
Epoch 385/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0020 - val_loss: 0.0058
Epoch 386/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0016 - val_loss: 0.0045
Epoch 532/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0017 - val_loss: 0.0055
Epoch 533/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0017 - val_loss: 0.0053
Epoch 534/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0020 - val_loss: 0.0061
Epoch 535/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0019 - val_loss: 0.0056
Epoch 536/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0016 - val_loss: 0.0046
Epoch 537/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0017 - val_loss: 0.0044
Epoch 538/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0015 - val_loss: 0.0049
Epoch 539/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0016 - val_loss: 0.0061
Epoch 540/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 125us/step - loss: 0.0015 - val_loss: 0.0042
Epoch 686/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0015 - val_loss: 0.0051
Epoch 687/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0015 - val_loss: 0.0069
Epoch 688/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0018 - val_loss: 0.0068
Epoch 689/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0020 - val_loss: 0.0045
Epoch 690/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0019 - val_loss: 0.0045
Epoch 691/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0018 - val_loss: 0.0058
Epoch 692/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0014 - val_loss: 0.0042
Epoch 693/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0014 - val_loss: 0.0048
Epoch 694/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 133us/step - loss: 0.0014 - val_loss: 0.0040
Epoch 840/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0015 - val_loss: 0.0054
Epoch 841/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0019 - val_loss: 0.0068
Epoch 842/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0018 - val_loss: 0.0056
Epoch 843/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0016 - val_loss: 0.0039
Epoch 844/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0015 - val_loss: 0.0041
Epoch 845/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0014 - val_loss: 0.0044
Epoch 846/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0016 - val_loss: 0.0067
Epoch 847/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0013 - val_loss: 0.0051
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 126us/step - loss: 0.0010 - val_loss: 0.0062
Epoch 994/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0013 - val_loss: 0.0052
Epoch 995/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0011 - val_loss: 0.0044
Epoch 996/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0011 - val_loss: 0.0047
Epoch 997/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0010 - val_loss: 0.0059
Epoch 998/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0013 - val_loss: 0.0063
Epoch 999/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0012 - val_loss: 0.0046
Epoch 1000/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0010 - val_loss: 0.0046
0.007517980877310038


[array([[ 4.12755638e-01,  6.52505178e-03,  4.21901822e-01,
          1.46211982e-01, -3.81319821e-01, -1.37775600e+00,
          1.46873429e-01,  1.14680924e-01,  1.31995782e-01,
          6.77955866e-01],
        [-1.28790671e-02,  2.45136186e-01,  2.63380051e-01,
          5.30252904e-02,  1.34121314e-01,  8.77678454e-01,
         -3.93621117e-01,  5.17267108e-01, -1.04674518e+00,
          4.00007218e-02],
        [ 3.46141875e-01, -2.21463859e-01,  1.51258543e-01,
         -8.30499381e-02, -1.51601970e-01,  6.03372991e-01,
         -2.98534900e-01,  3.31560105e-01,  4.47529763e-01,
          7.84211606e-02],
        [ 7.01330602e-02,  3.73776019e-01,  3.14476192e-02,
          8.66234973e-02,  4.37999755e-01,  2.95698792e-01,
         -2.07813993e-01,  4.18076575e-01, -1.23005152e+00,
          3.77705514e-01],
        [ 2.88788885e-01, -2.98465610e-01, -3.87324840e-01,
         -6.02497041e-01,  2.54160553e-01,  2.54337251e-01,
         -4.44469064e-01,  1.66325003e-01, -2.182069

In [61]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure6_2nd.h5')

In [62]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_1(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_55 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_37 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_56 (Dense)             (None, 5)                 30        
_________________________________________________________________
activation_38 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_57 (Dense)             (None, 1)                 6         
Total params: 56
Trainable params: 56
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 12ms/step - loss: 36.65

Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0121 - val_loss: 0.0081
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0106 - val_loss: 0.0066
Epoch 73/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0093 - val_loss: 0.0057
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0083 - val_loss: 0.0054
Epoch 75/1000
32/32 [==============================] - 0s 146us/step - loss: 0.0075 - val_loss: 0.0052
Epoch 76/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0070 - val_loss: 0.0049
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0067 - val_loss: 0.0046
Epoch 78/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0066 - val_loss: 0.0041
Epoch 79/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0066 - val_loss: 0.0036
Epoch 80/1000
32/32 [==============================] - 0s 125us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0036 - val_loss: 0.0013
Epoch 152/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0035 - val_loss: 0.0013
Epoch 153/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0035 - val_loss: 0.0013
Epoch 154/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0035 - val_loss: 0.0013
Epoch 155/1000
32/32 [==============================] - 0s 126us/step - loss: 0.0035 - val_loss: 0.0012
Epoch 156/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0035 - val_loss: 0.0012
Epoch 157/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0035 - val_loss: 0.0012
Epoch 158/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0035 - val_loss: 0.0012
Epoch 159/1000
32/32 [==============================] - 0s 146us/step - loss: 0.0034 - val_loss: 0.0012
Epoch 160/1000
32/32 [==============================] - 0s 155us

Epoch 229/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0027 - val_loss: 8.5480e-04
Epoch 230/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0027 - val_loss: 8.5057e-04
Epoch 231/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0027 - val_loss: 8.4638e-04
Epoch 232/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0027 - val_loss: 8.4223e-04
Epoch 233/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0026 - val_loss: 8.3815e-04
Epoch 234/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0026 - val_loss: 8.3408e-04
Epoch 235/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0026 - val_loss: 8.3005e-04
Epoch 236/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0026 - val_loss: 8.2606e-04
Epoch 237/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0026 - val_loss: 8.2209e-04
Epoch 238/1000
32/32 [======

Epoch 305/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0021 - val_loss: 5.9099e-04
Epoch 306/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0021 - val_loss: 5.8813e-04
Epoch 307/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0021 - val_loss: 5.8528e-04
Epoch 308/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0021 - val_loss: 5.8248e-04
Epoch 309/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0021 - val_loss: 5.7969e-04
Epoch 310/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0021 - val_loss: 5.7690e-04
Epoch 311/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0021 - val_loss: 5.7410e-04
Epoch 312/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0021 - val_loss: 5.7134e-04
Epoch 313/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0021 - val_loss: 5.6858e-04
Epoch 314/1000
32/32 [======

...

Epoch 757/1000
32/32 [==============================] - 0s 156us/step - loss: 6.5568e-04 - val_loss: 1.0081e-04
Epoch 758/1000
32/32 [==============================] - 0s 125us/step - loss: 6.5393e-04 - val_loss: 1.0054e-04
Epoch 759/1000
32/32 [==============================] - 0s 156us/step - loss: 6.5218e-04 - val_loss: 1.0027e-04
Epoch 760/1000
32/32 [==============================] - 0s 124us/step - loss: 6.5045e-04 - val_loss: 1.0000e-04
Epoch 761/1000
32/32 [==============================] - 0s 156us/step - loss: 6.4871e-04 - val_loss: 9.9740e-05
Epoch 762/1000
32/32 [==============================] - 0s 156us/step - loss: 6.4698e-04 - val_loss: 9.9486e-05
Epoch 763/1000
32/32 [==============================] - 0s 156us/step - loss: 6.4525e-04 - val_loss: 9.9219e-05
Epoch 764/1000
32/32 [==============================] - 0s 156us/step - loss: 6.4353e-04 - val_loss: 9.8960e-05
Epoch 765/1000
32/32 [==============================] - 0s 155us/step - loss: 6.4182e-04 - val_loss: 9.8

Epoch 831/1000
32/32 [==============================] - 0s 156us/step - loss: 5.3756e-04 - val_loss: 8.3491e-05
Epoch 832/1000
32/32 [==============================] - 0s 156us/step - loss: 5.3611e-04 - val_loss: 8.3281e-05
Epoch 833/1000
32/32 [==============================] - 0s 156us/step - loss: 5.3466e-04 - val_loss: 8.3075e-05
Epoch 834/1000
32/32 [==============================] - 0s 125us/step - loss: 5.3322e-04 - val_loss: 8.2870e-05
Epoch 835/1000
32/32 [==============================] - 0s 187us/step - loss: 5.3179e-04 - val_loss: 8.2671e-05
Epoch 836/1000
32/32 [==============================] - 0s 218us/step - loss: 5.3036e-04 - val_loss: 8.2468e-05
Epoch 837/1000
32/32 [==============================] - 0s 156us/step - loss: 5.2893e-04 - val_loss: 8.2267e-05
Epoch 838/1000
32/32 [==============================] - 0s 156us/step - loss: 5.2751e-04 - val_loss: 8.2063e-05
Epoch 839/1000
32/32 [==============================] - 0s 156us/step - loss: 5.2608e-04 - val_loss: 8.1

Epoch 905/1000
32/32 [==============================] - 0s 187us/step - loss: 4.4015e-04 - val_loss: 6.9814e-05
Epoch 906/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3896e-04 - val_loss: 6.9651e-05
Epoch 907/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3778e-04 - val_loss: 6.9486e-05
Epoch 908/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3660e-04 - val_loss: 6.9315e-05
Epoch 909/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3542e-04 - val_loss: 6.9151e-05
Epoch 910/1000
32/32 [==============================] - 0s 187us/step - loss: 4.3424e-04 - val_loss: 6.8986e-05
Epoch 911/1000
32/32 [==============================] - 0s 155us/step - loss: 4.3308e-04 - val_loss: 6.8820e-05
Epoch 912/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3190e-04 - val_loss: 6.8655e-05
Epoch 913/1000
32/32 [==============================] - 0s 156us/step - loss: 4.3074e-04 - val_loss: 6.8

Epoch 979/1000
32/32 [==============================] - 0s 187us/step - loss: 3.6060e-04 - val_loss: 5.8425e-05
Epoch 980/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5963e-04 - val_loss: 5.8284e-05
Epoch 981/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5867e-04 - val_loss: 5.8141e-05
Epoch 982/1000
32/32 [==============================] - 0s 187us/step - loss: 3.5771e-04 - val_loss: 5.8003e-05
Epoch 983/1000
32/32 [==============================] - 0s 155us/step - loss: 3.5676e-04 - val_loss: 5.7862e-05
Epoch 984/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5581e-04 - val_loss: 5.7720e-05
Epoch 985/1000
32/32 [==============================] - 0s 187us/step - loss: 3.5485e-04 - val_loss: 5.7581e-05
Epoch 986/1000
32/32 [==============================] - 0s 125us/step - loss: 3.5391e-04 - val_loss: 5.7440e-05
Epoch 987/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5296e-04 - val_loss: 5.7

[array([[ 0.89171654, -0.76205015,  0.08254873,  0.6707255 ,  0.2554909 ],
        [-0.48835367,  0.5092173 , -0.33163166,  0.36471644, -0.01312719],
        [ 0.66100895,  0.8473332 , -0.54665524,  1.4670547 ,  1.1479717 ]],
       dtype=float32),
 array([-0.51324785, -0.6671838 , -0.81470305,  0.6924536 ,  0.5144541 ],
       dtype=float32),
 array([[-0.47093776, -0.23187377,  0.01583179, -0.95371944,  0.71973705],
        [-0.9090844 , -0.17508113,  0.40705237, -0.72806317, -0.1065243 ],
        [-0.44984707, -0.30987573, -0.5420416 , -0.08325931,  0.06469244],
        [ 0.49054736, -0.17894533, -0.32699907,  0.78325397,  0.21815884],
        [-0.6837565 , -0.7079664 , -0.46730724,  0.42671806,  0.02811091]],
       dtype=float32),
 array([ 0.9575484 , -0.92973846, -0.8974282 ,  0.9691136 , -0.8821732 ],
       dtype=float32),
 array([[ 1.0957047 ],
        [-0.42902774],
        [-0.5003522 ],
        [ 0.98112893],
        [-0.5037892 ]], dtype=float32),
 array([0.9872191], dtype=

In [63]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure1_2nd.h5')

In [64]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_2(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_58 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_39 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_59 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_40 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_60 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 12ms/step - loss: 36.17

Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0037 - val_loss: 0.0067
Epoch 72/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0041 - val_loss: 0.0062
Epoch 73/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0042 - val_loss: 0.0058
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0041 - val_loss: 0.0054
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0037 - val_loss: 0.0051
Epoch 76/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0032 - val_loss: 0.0049
Epoch 77/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0027 - val_loss: 0.0046
Epoch 78/1000
32/32 [==============================] - 0s 218us/step - loss: 0.0023 - val_loss: 0.0044
Epoch 79/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0020 - val_loss: 0.0043
Epoch 80/1000
32/32 [==============================] - 0s 187us/step - lo

Epoch 148/1000
32/32 [==============================] - 0s 187us/step - loss: 5.3937e-04 - val_loss: 6.0002e-04
Epoch 149/1000
32/32 [==============================] - 0s 156us/step - loss: 5.3356e-04 - val_loss: 5.9241e-04
Epoch 150/1000
32/32 [==============================] - 0s 155us/step - loss: 5.2796e-04 - val_loss: 5.8483e-04
Epoch 151/1000
32/32 [==============================] - 0s 156us/step - loss: 5.2247e-04 - val_loss: 5.7735e-04
Epoch 152/1000
32/32 [==============================] - 0s 156us/step - loss: 5.1708e-04 - val_loss: 5.6958e-04
Epoch 153/1000
32/32 [==============================] - 0s 187us/step - loss: 5.1174e-04 - val_loss: 5.6128e-04
Epoch 154/1000
32/32 [==============================] - 0s 125us/step - loss: 5.0644e-04 - val_loss: 5.5239e-04
Epoch 155/1000
32/32 [==============================] - 0s 155us/step - loss: 5.0120e-04 - val_loss: 5.4333e-04
Epoch 156/1000
32/32 [==============================] - 0s 187us/step - loss: 4.9604e-04 - val_loss: 5.3

Epoch 222/1000
32/32 [==============================] - 0s 156us/step - loss: 3.0248e-04 - val_loss: 3.1530e-04
Epoch 223/1000
32/32 [==============================] - 0s 155us/step - loss: 3.0081e-04 - val_loss: 3.1390e-04
Epoch 224/1000
32/32 [==============================] - 0s 155us/step - loss: 2.9917e-04 - val_loss: 3.1254e-04
Epoch 225/1000
32/32 [==============================] - 0s 156us/step - loss: 2.9756e-04 - val_loss: 3.1122e-04
Epoch 226/1000
32/32 [==============================] - 0s 155us/step - loss: 2.9595e-04 - val_loss: 3.0995e-04
Epoch 227/1000
32/32 [==============================] - 0s 156us/step - loss: 2.9438e-04 - val_loss: 3.0869e-04
Epoch 228/1000
32/32 [==============================] - 0s 187us/step - loss: 2.9282e-04 - val_loss: 3.0749e-04
Epoch 229/1000
32/32 [==============================] - 0s 156us/step - loss: 2.9128e-04 - val_loss: 3.0631e-04
Epoch 230/1000
32/32 [==============================] - 0s 156us/step - loss: 2.8976e-04 - val_loss: 3.0

Epoch 296/1000
32/32 [==============================] - 0s 218us/step - loss: 2.1969e-04 - val_loss: 2.6003e-04
Epoch 297/1000
32/32 [==============================] - 0s 156us/step - loss: 2.1894e-04 - val_loss: 2.5958e-04
Epoch 298/1000
32/32 [==============================] - 0s 156us/step - loss: 2.1820e-04 - val_loss: 2.5914e-04
Epoch 299/1000
32/32 [==============================] - 0s 187us/step - loss: 2.1746e-04 - val_loss: 2.5870e-04
Epoch 300/1000
32/32 [==============================] - 0s 156us/step - loss: 2.1673e-04 - val_loss: 2.5827e-04
Epoch 301/1000
32/32 [==============================] - 0s 187us/step - loss: 2.1601e-04 - val_loss: 2.5785e-04
Epoch 302/1000
32/32 [==============================] - 0s 157us/step - loss: 2.1529e-04 - val_loss: 2.5742e-04
Epoch 303/1000
32/32 [==============================] - 0s 156us/step - loss: 2.1458e-04 - val_loss: 2.5699e-04
Epoch 304/1000
32/32 [==============================] - 0s 187us/step - loss: 2.1387e-04 - val_loss: 2.5

...

Epoch 740/1000
32/32 [==============================] - 0s 155us/step - loss: 9.0577e-05 - val_loss: 1.3808e-04
Epoch 741/1000
32/32 [==============================] - 0s 156us/step - loss: 9.0444e-05 - val_loss: 1.3788e-04
Epoch 742/1000
32/32 [==============================] - 0s 125us/step - loss: 9.0304e-05 - val_loss: 1.3768e-04
Epoch 743/1000
32/32 [==============================] - 0s 155us/step - loss: 9.0168e-05 - val_loss: 1.3748e-04
Epoch 744/1000
32/32 [==============================] - 0s 157us/step - loss: 9.0036e-05 - val_loss: 1.3727e-04
Epoch 745/1000
32/32 [==============================] - 0s 155us/step - loss: 8.9897e-05 - val_loss: 1.3707e-04
Epoch 746/1000
32/32 [==============================] - 0s 156us/step - loss: 8.9763e-05 - val_loss: 1.3686e-04
Epoch 747/1000
32/32 [==============================] - 0s 155us/step - loss: 8.9628e-05 - val_loss: 1.3667e-04
Epoch 748/1000
32/32 [==============================] - 0s 156us/step - loss: 8.9493e-05 - val_loss: 1.3

Epoch 814/1000
32/32 [==============================] - 0s 126us/step - loss: 8.1120e-05 - val_loss: 1.2370e-04
Epoch 815/1000
32/32 [==============================] - 0s 156us/step - loss: 8.0997e-05 - val_loss: 1.2352e-04
Epoch 816/1000
32/32 [==============================] - 0s 162us/step - loss: 8.0878e-05 - val_loss: 1.2333e-04
Epoch 817/1000
32/32 [==============================] - 0s 155us/step - loss: 8.0759e-05 - val_loss: 1.2314e-04
Epoch 818/1000
32/32 [==============================] - 0s 156us/step - loss: 8.0638e-05 - val_loss: 1.2295e-04
Epoch 819/1000
32/32 [==============================] - 0s 187us/step - loss: 8.0516e-05 - val_loss: 1.2277e-04
Epoch 820/1000
32/32 [==============================] - 0s 156us/step - loss: 8.0402e-05 - val_loss: 1.2259e-04
Epoch 821/1000
32/32 [==============================] - 0s 125us/step - loss: 8.0281e-05 - val_loss: 1.2240e-04
Epoch 822/1000
32/32 [==============================] - 0s 125us/step - loss: 8.0162e-05 - val_loss: 1.2

Epoch 888/1000
32/32 [==============================] - 0s 156us/step - loss: 7.2745e-05 - val_loss: 1.1048e-04
Epoch 889/1000
32/32 [==============================] - 0s 154us/step - loss: 7.2637e-05 - val_loss: 1.1030e-04
Epoch 890/1000
32/32 [==============================] - 0s 155us/step - loss: 7.2532e-05 - val_loss: 1.1013e-04
Epoch 891/1000
32/32 [==============================] - 0s 144us/step - loss: 7.2425e-05 - val_loss: 1.0997e-04
Epoch 892/1000
32/32 [==============================] - 0s 124us/step - loss: 7.2318e-05 - val_loss: 1.0979e-04
Epoch 893/1000
32/32 [==============================] - 0s 124us/step - loss: 7.2210e-05 - val_loss: 1.0963e-04
Epoch 894/1000
32/32 [==============================] - 0s 124us/step - loss: 7.2107e-05 - val_loss: 1.0945e-04
Epoch 895/1000
32/32 [==============================] - 0s 156us/step - loss: 7.1999e-05 - val_loss: 1.0928e-04
Epoch 896/1000
32/32 [==============================] - 0s 156us/step - loss: 7.1895e-05 - val_loss: 1.0

Epoch 962/1000
32/32 [==============================] - 0s 125us/step - loss: 6.5251e-05 - val_loss: 9.8323e-05
Epoch 963/1000
32/32 [==============================] - 0s 153us/step - loss: 6.5154e-05 - val_loss: 9.8166e-05
Epoch 964/1000
32/32 [==============================] - 0s 179us/step - loss: 6.5058e-05 - val_loss: 9.8009e-05
Epoch 965/1000
32/32 [==============================] - 0s 125us/step - loss: 6.4962e-05 - val_loss: 9.7843e-05
Epoch 966/1000
32/32 [==============================] - 0s 124us/step - loss: 6.4865e-05 - val_loss: 9.7701e-05
Epoch 967/1000
32/32 [==============================] - 0s 156us/step - loss: 6.4772e-05 - val_loss: 9.7544e-05
Epoch 968/1000
32/32 [==============================] - 0s 125us/step - loss: 6.4675e-05 - val_loss: 9.7384e-05
Epoch 969/1000
32/32 [==============================] - 0s 156us/step - loss: 6.4580e-05 - val_loss: 9.7228e-05
Epoch 970/1000
32/32 [==============================] - 0s 156us/step - loss: 6.4487e-05 - val_loss: 9.7

[array([[-0.06182549, -0.38713577,  0.06759677, -0.0458573 , -0.41453752],
        [ 1.2679622 , -0.39310175,  0.3822536 ,  1.3825084 , -0.3998088 ],
        [ 0.7199374 , -1.7513487 ,  1.7682974 ,  0.76659495, -1.4130423 ]],
       dtype=float32),
 array([ 0.33862007,  0.55749637, -0.47889024,  0.37576637,  0.28218088],
       dtype=float32),
 array([[ 0.43243444,  0.22042912, -0.22639693, -0.35370985,  0.7066437 ,
          0.10902827,  0.45573014, -0.27942678, -0.42391744, -0.19043551],
        [-0.4396294 , -0.0880951 , -0.4899491 ,  0.48285547,  0.39852217,
         -0.10564619, -0.68857837, -0.3855749 ,  0.17675224, -0.48042095],
        [ 0.0415375 ,  0.26459557, -0.02249081,  0.20196736, -0.23938161,
          0.23885371, -0.11045036,  0.459829  , -0.42828596,  0.23675881],
        [-0.5428275 , -0.04239545,  0.00960112, -0.494626  , -0.35884798,
         -0.22439824, -0.16406092, -0.39249632, -0.629626  , -0.72096425],
        [ 0.37810072, -0.33490768, -0.25408244,  0.4455541

In [65]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure2_2nd.h5')

In [66]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_3(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_61 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_41 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_62 (Dense)             (None, 15)                90        
_________________________________________________________________
activation_42 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_63 (Dense)             (None, 1)                 16        
Total params: 126
Trainable params: 126
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 14ms/step - loss: 35.

Epoch 71/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0141 - val_loss: 0.0144
Epoch 72/1000
32/32 [==============================] - 0s 157us/step - loss: 0.0151 - val_loss: 0.0170
Epoch 73/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0160 - val_loss: 0.0184
Epoch 74/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0166 - val_loss: 0.0185
Epoch 75/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0164 - val_loss: 0.0175
Epoch 76/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0156 - val_loss: 0.0156
Epoch 77/1000
32/32 [==============================] - 0s 178us/step - loss: 0.0143 - val_loss: 0.0130
Epoch 78/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0129 - val_loss: 0.0103
Epoch 79/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0116 - val_loss: 0.0078
Epoch 80/1000
32/32 [==============================] - 0s 156us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0046 - val_loss: 0.0025
Epoch 152/1000
32/32 [==============================] - 0s 170us/step - loss: 0.0046 - val_loss: 0.0024
Epoch 153/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0045 - val_loss: 0.0024
Epoch 154/1000
32/32 [==============================] - 0s 163us/step - loss: 0.0045 - val_loss: 0.0024
Epoch 155/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0044 - val_loss: 0.0024
Epoch 156/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0044 - val_loss: 0.0024
Epoch 157/1000
32/32 [==============================] - 0s 147us/step - loss: 0.0044 - val_loss: 0.0024
Epoch 158/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0043 - val_loss: 0.0024
Epoch 159/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0043 - val_loss: 0.0024
Epoch 160/1000
32/32 [==============================] - 0s 156us

Epoch 230/1000
32/32 [==============================] - 0s 130us/step - loss: 0.0028 - val_loss: 0.0015
Epoch 231/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0028 - val_loss: 0.0015
Epoch 232/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0027 - val_loss: 0.0014
Epoch 233/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0027 - val_loss: 0.0014
Epoch 234/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0027 - val_loss: 0.0014
Epoch 235/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0027 - val_loss: 0.0014
Epoch 236/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0027 - val_loss: 0.0014
Epoch 237/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0027 - val_loss: 0.0014
Epoch 238/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0027 - val_loss: 0.0014
Epoch 239/1000
32/32 [==============================] - 0s 176us

Epoch 309/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0020 - val_loss: 9.8662e-04
Epoch 310/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0020 - val_loss: 9.8242e-04
Epoch 311/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0020 - val_loss: 9.7821e-04
Epoch 312/1000
32/32 [==============================] - 0s 154us/step - loss: 0.0020 - val_loss: 9.7408e-04
Epoch 313/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0020 - val_loss: 9.6996e-04
Epoch 314/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0020 - val_loss: 9.6586e-04
Epoch 315/1000
32/32 [==============================] - 0s 151us/step - loss: 0.0020 - val_loss: 9.6182e-04
Epoch 316/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0020 - val_loss: 9.5778e-04
Epoch 317/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0020 - val_loss: 9.5380e-04
Epoch 318/1000
32/32 [======

...

Epoch 760/1000
32/32 [==============================] - 0s 124us/step - loss: 7.2163e-04 - val_loss: 3.6968e-04
Epoch 761/1000
32/32 [==============================] - 0s 156us/step - loss: 7.2038e-04 - val_loss: 3.6929e-04
Epoch 762/1000
32/32 [==============================] - 0s 187us/step - loss: 7.1913e-04 - val_loss: 3.6890e-04
Epoch 763/1000
32/32 [==============================] - 0s 156us/step - loss: 7.1788e-04 - val_loss: 3.6851e-04
Epoch 764/1000
32/32 [==============================] - 0s 188us/step - loss: 7.1664e-04 - val_loss: 3.6814e-04
Epoch 765/1000
32/32 [==============================] - 0s 156us/step - loss: 7.1541e-04 - val_loss: 3.6775e-04
Epoch 766/1000
32/32 [==============================] - 0s 147us/step - loss: 7.1417e-04 - val_loss: 3.6736e-04
Epoch 767/1000
32/32 [==============================] - 0s 155us/step - loss: 7.1294e-04 - val_loss: 3.6700e-04
Epoch 768/1000
32/32 [==============================] - 0s 151us/step - loss: 7.1173e-04 - val_loss: 3.6

Epoch 834/1000
32/32 [==============================] - 0s 156us/step - loss: 6.3700e-04 - val_loss: 3.4343e-04
Epoch 835/1000
32/32 [==============================] - 0s 156us/step - loss: 6.3596e-04 - val_loss: 3.4308e-04
Epoch 836/1000
32/32 [==============================] - 0s 156us/step - loss: 6.3492e-04 - val_loss: 3.4276e-04
Epoch 837/1000
32/32 [==============================] - 0s 156us/step - loss: 6.3388e-04 - val_loss: 3.4244e-04
Epoch 838/1000
32/32 [==============================] - 0s 156us/step - loss: 6.3284e-04 - val_loss: 3.4212e-04
Epoch 839/1000
32/32 [==============================] - 0s 155us/step - loss: 6.3181e-04 - val_loss: 3.4179e-04
Epoch 840/1000
32/32 [==============================] - 0s 156us/step - loss: 6.3078e-04 - val_loss: 3.4148e-04
Epoch 841/1000
32/32 [==============================] - 0s 165us/step - loss: 6.2975e-04 - val_loss: 3.4116e-04
Epoch 842/1000
32/32 [==============================] - 0s 187us/step - loss: 6.2872e-04 - val_loss: 3.4

Epoch 908/1000
32/32 [==============================] - 0s 156us/step - loss: 5.6587e-04 - val_loss: 3.2073e-04
Epoch 909/1000
32/32 [==============================] - 0s 156us/step - loss: 5.6500e-04 - val_loss: 3.2045e-04
Epoch 910/1000
32/32 [==============================] - 0s 187us/step - loss: 5.6412e-04 - val_loss: 3.2016e-04
Epoch 911/1000
32/32 [==============================] - 0s 187us/step - loss: 5.6323e-04 - val_loss: 3.1987e-04
Epoch 912/1000
32/32 [==============================] - 0s 156us/step - loss: 5.6235e-04 - val_loss: 3.1959e-04
Epoch 913/1000
32/32 [==============================] - 0s 125us/step - loss: 5.6148e-04 - val_loss: 3.1930e-04
Epoch 914/1000
32/32 [==============================] - 0s 187us/step - loss: 5.6061e-04 - val_loss: 3.1900e-04
Epoch 915/1000
32/32 [==============================] - 0s 156us/step - loss: 5.5973e-04 - val_loss: 3.1873e-04
Epoch 916/1000
32/32 [==============================] - 0s 187us/step - loss: 5.5887e-04 - val_loss: 3.1

Epoch 982/1000
32/32 [==============================] - 0s 157us/step - loss: 5.0546e-04 - val_loss: 3.0025e-04
Epoch 983/1000
32/32 [==============================] - 0s 156us/step - loss: 5.0471e-04 - val_loss: 2.9998e-04
Epoch 984/1000
32/32 [==============================] - 0s 155us/step - loss: 5.0396e-04 - val_loss: 2.9972e-04
Epoch 985/1000
32/32 [==============================] - 0s 155us/step - loss: 5.0322e-04 - val_loss: 2.9945e-04
Epoch 986/1000
32/32 [==============================] - 0s 156us/step - loss: 5.0246e-04 - val_loss: 2.9918e-04
Epoch 987/1000
32/32 [==============================] - 0s 155us/step - loss: 5.0172e-04 - val_loss: 2.9890e-04
Epoch 988/1000
32/32 [==============================] - 0s 149us/step - loss: 5.0097e-04 - val_loss: 2.9864e-04
Epoch 989/1000
32/32 [==============================] - 0s 125us/step - loss: 5.0023e-04 - val_loss: 2.9839e-04
Epoch 990/1000
32/32 [==============================] - 0s 124us/step - loss: 4.9948e-04 - val_loss: 2.9

[array([[ 0.47976074, -0.36353016, -0.01851191,  0.23863232,  0.03646738],
        [ 0.5874555 ,  0.64983755, -1.0630714 ,  0.5912907 ,  1.0239949 ],
        [ 1.5246214 ,  1.4286333 ,  0.75188553,  1.7786391 ,  0.8043896 ]],
       dtype=float32),
 array([ 0.08185434, -0.16567197, -0.6640233 ,  0.44877312, -0.48303416],
       dtype=float32),
 array([[ 0.30607936, -0.4592974 ,  0.08266341,  0.1851418 , -0.40322548,
         -0.03020122,  0.11569655,  0.47172412, -0.7517093 , -0.5368275 ,
         -0.2260384 ,  0.04775874, -0.29997924, -0.19782972, -0.02385323],
        [ 0.4642896 ,  0.36660776,  0.03975499, -0.46183714,  0.34478793,
         -0.23975322,  0.39313942,  0.32522342, -0.2305253 , -0.49788794,
         -0.2551308 ,  0.1320612 , -0.5311148 , -0.34472758, -0.30377263],
        [-0.5869909 ,  0.26495343, -0.5037383 , -0.6367936 , -0.4689473 ,
          0.0987145 , -0.23745602, -0.34936252, -0.33994085,  0.29732886,
         -0.25864533,  0.05623841, -0.07872384,  0.5858796 ,

In [67]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure3_2nd.h5')

In [68]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_4(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_64 (Dense)             (None, 10)                40        
_________________________________________________________________
activation_43 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_65 (Dense)             (None, 5)                 55        
_________________________________________________________________
activation_44 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_66 (Dense)             (None, 1)                 6         
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 14ms/step - loss: 37.

Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0115 - val_loss: 0.0036
Epoch 72/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0112 - val_loss: 0.0032
Epoch 73/1000
32/32 [==============================] - 0s 164us/step - loss: 0.0101 - val_loss: 0.0028
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0087 - val_loss: 0.0026
Epoch 75/1000
32/32 [==============================] - 0s 159us/step - loss: 0.0072 - val_loss: 0.0028
Epoch 76/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0062 - val_loss: 0.0036
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0058 - val_loss: 0.0049
Epoch 78/1000
32/32 [==============================] - 0s 134us/step - loss: 0.0059 - val_loss: 0.0063
Epoch 79/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0064 - val_loss: 0.0075
Epoch 80/1000
32/32 [==============================] - 0s 156us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0027 - val_loss: 0.0038
Epoch 152/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0027 - val_loss: 0.0038
Epoch 153/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0027 - val_loss: 0.0038
Epoch 154/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0026 - val_loss: 0.0037
Epoch 155/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0026 - val_loss: 0.0037
Epoch 156/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0026 - val_loss: 0.0037
Epoch 157/1000
32/32 [==============================] - 0s 186us/step - loss: 0.0026 - val_loss: 0.0036
Epoch 158/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0026 - val_loss: 0.0036
Epoch 159/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0025 - val_loss: 0.0036
Epoch 160/1000
32/32 [==============================] - 0s 187us

Epoch 230/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0017 - val_loss: 0.0025
Epoch 231/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0017 - val_loss: 0.0025
Epoch 232/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0017 - val_loss: 0.0025
Epoch 233/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0017 - val_loss: 0.0025
Epoch 234/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0017 - val_loss: 0.0025
Epoch 235/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0017 - val_loss: 0.0025
Epoch 236/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0017 - val_loss: 0.0024
Epoch 237/1000
32/32 [==============================] - 0s 182us/step - loss: 0.0016 - val_loss: 0.0024
Epoch 238/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0016 - val_loss: 0.0024
Epoch 239/1000
32/32 [==============================] - 0s 156us

Epoch 309/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0013 - val_loss: 0.0019
Epoch 310/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0012 - val_loss: 0.0019
Epoch 311/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0012 - val_loss: 0.0019
Epoch 312/1000
32/32 [==============================] - 0s 147us/step - loss: 0.0012 - val_loss: 0.0018
Epoch 313/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0012 - val_loss: 0.0018
Epoch 314/1000
32/32 [==============================] - 0s 151us/step - loss: 0.0012 - val_loss: 0.0018
Epoch 315/1000
32/32 [==============================] - 0s 153us/step - loss: 0.0012 - val_loss: 0.0018
Epoch 316/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0012 - val_loss: 0.0018
Epoch 317/1000
32/32 [==============================] - 0s 151us/step - loss: 0.0012 - val_loss: 0.0018
Epoch 318/1000
32/32 [==============================] - 0s 157us

...

Epoch 762/1000
32/32 [==============================] - 0s 165us/step - loss: 4.5847e-04 - val_loss: 6.7624e-04
Epoch 763/1000
32/32 [==============================] - 0s 156us/step - loss: 4.5761e-04 - val_loss: 6.7502e-04
Epoch 764/1000
32/32 [==============================] - 0s 156us/step - loss: 4.5675e-04 - val_loss: 6.7380e-04
Epoch 765/1000
32/32 [==============================] - 0s 141us/step - loss: 4.5589e-04 - val_loss: 6.7259e-04
Epoch 766/1000
32/32 [==============================] - 0s 159us/step - loss: 4.5506e-04 - val_loss: 6.7137e-04
Epoch 767/1000
32/32 [==============================] - 0s 187us/step - loss: 4.5420e-04 - val_loss: 6.7016e-04
Epoch 768/1000
32/32 [==============================] - 0s 156us/step - loss: 4.5335e-04 - val_loss: 6.6898e-04
Epoch 769/1000
32/32 [==============================] - 0s 125us/step - loss: 4.5251e-04 - val_loss: 6.6777e-04
Epoch 770/1000
32/32 [==============================] - 0s 156us/step - loss: 4.5167e-04 - val_loss: 6.6

Epoch 836/1000
32/32 [==============================] - 0s 156us/step - loss: 3.9979e-04 - val_loss: 5.9257e-04
Epoch 837/1000
32/32 [==============================] - 0s 187us/step - loss: 3.9906e-04 - val_loss: 5.9153e-04
Epoch 838/1000
32/32 [==============================] - 0s 156us/step - loss: 3.9832e-04 - val_loss: 5.9048e-04
Epoch 839/1000
32/32 [==============================] - 0s 134us/step - loss: 3.9760e-04 - val_loss: 5.8945e-04
Epoch 840/1000
32/32 [==============================] - 0s 125us/step - loss: 3.9687e-04 - val_loss: 5.8842e-04
Epoch 841/1000
32/32 [==============================] - 0s 156us/step - loss: 3.9614e-04 - val_loss: 5.8736e-04
Epoch 842/1000
32/32 [==============================] - 0s 156us/step - loss: 3.9542e-04 - val_loss: 5.8632e-04
Epoch 843/1000
32/32 [==============================] - 0s 167us/step - loss: 3.9469e-04 - val_loss: 5.8529e-04
Epoch 844/1000
32/32 [==============================] - 0s 131us/step - loss: 3.9397e-04 - val_loss: 5.8

Epoch 910/1000
32/32 [==============================] - 0s 187us/step - loss: 3.4945e-04 - val_loss: 5.2071e-04
Epoch 911/1000
32/32 [==============================] - 0s 156us/step - loss: 3.4882e-04 - val_loss: 5.1981e-04
Epoch 912/1000
32/32 [==============================] - 0s 155us/step - loss: 3.4819e-04 - val_loss: 5.1889e-04
Epoch 913/1000
32/32 [==============================] - 0s 165us/step - loss: 3.4757e-04 - val_loss: 5.1798e-04
Epoch 914/1000
32/32 [==============================] - 0s 157us/step - loss: 3.4694e-04 - val_loss: 5.1708e-04
Epoch 915/1000
32/32 [==============================] - 0s 156us/step - loss: 3.4631e-04 - val_loss: 5.1619e-04
Epoch 916/1000
32/32 [==============================] - 0s 184us/step - loss: 3.4568e-04 - val_loss: 5.1530e-04
Epoch 917/1000
32/32 [==============================] - 0s 131us/step - loss: 3.4507e-04 - val_loss: 5.1440e-04
Epoch 918/1000
32/32 [==============================] - 0s 148us/step - loss: 3.4444e-04 - val_loss: 5.1

Epoch 984/1000
32/32 [==============================] - 0s 156us/step - loss: 3.0606e-04 - val_loss: 4.5861e-04
Epoch 985/1000
32/32 [==============================] - 0s 166us/step - loss: 3.0552e-04 - val_loss: 4.5779e-04
Epoch 986/1000
32/32 [==============================] - 0s 134us/step - loss: 3.0497e-04 - val_loss: 4.5701e-04
Epoch 987/1000
32/32 [==============================] - 0s 155us/step - loss: 3.0443e-04 - val_loss: 4.5624e-04
Epoch 988/1000
32/32 [==============================] - 0s 155us/step - loss: 3.0389e-04 - val_loss: 4.5548e-04
Epoch 989/1000
32/32 [==============================] - 0s 124us/step - loss: 3.0335e-04 - val_loss: 4.5470e-04
Epoch 990/1000
32/32 [==============================] - 0s 186us/step - loss: 3.0281e-04 - val_loss: 4.5392e-04
Epoch 991/1000
32/32 [==============================] - 0s 169us/step - loss: 3.0227e-04 - val_loss: 4.5315e-04
Epoch 992/1000
32/32 [==============================] - 0s 155us/step - loss: 3.0173e-04 - val_loss: 4.5

[array([[ 0.09387104,  0.44794676, -0.9581307 , -0.80688125,  0.45144805,
          0.71762943,  0.27972683, -0.2861967 , -0.6753328 ,  0.6117131 ],
        [ 1.2731714 , -0.23780279,  0.41497785,  0.39323267,  0.35095212,
         -0.7965661 ,  0.29592818,  0.25799155, -0.13029987,  0.10059815],
        [ 0.70047224, -0.6196809 , -0.6312383 ,  0.9282479 ,  1.1063507 ,
          0.70490867, -0.8073384 ,  0.5214568 , -0.3223029 ,  1.7474324 ]],
       dtype=float32),
 array([-0.4336528 ,  0.6199729 ,  0.6150489 , -0.5336144 ,  0.5968196 ,
        -0.6498447 ,  0.28098714,  0.61180943,  0.55085194,  0.28109884],
       dtype=float32),
 array([[ 6.1483198e-01, -3.0546552e-01,  2.8894332e-01,  1.7758511e-01,
         -4.8411402e-01],
        [ 7.3918238e-02, -2.6886219e-01, -8.6621881e-01,  6.2503922e-01,
         -7.8445029e-01],
        [-1.0726971e-01,  4.7291529e-01, -5.1571685e-01, -2.6251176e-01,
         -3.6824557e-01],
        [ 2.3006614e-01, -6.5256077e-01,  7.1913427e-01, -4.29

In [69]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure4_2nd.h5')

In [70]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_5(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_67 (Dense)             (None, 10)                40        
_________________________________________________________________
activation_45 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_68 (Dense)             (None, 10)                110       
_________________________________________________________________
activation_46 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_69 (Dense)             (None, 1)                 11        
Total params: 161
Trainable params: 161
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 14ms/step - loss: 36.

Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0193 - val_loss: 0.0375
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0179 - val_loss: 0.0319
Epoch 73/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0154 - val_loss: 0.0261
Epoch 74/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0130 - val_loss: 0.0213
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0115 - val_loss: 0.0179
Epoch 76/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0111 - val_loss: 0.0158
Epoch 77/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0117 - val_loss: 0.0145
Epoch 78/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0125 - val_loss: 0.0139
Epoch 79/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0129 - val_loss: 0.0136
Epoch 80/1000
32/32 [==============================] - 0s 187us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 129us/step - loss: 0.0053 - val_loss: 0.0093
Epoch 152/1000
32/32 [==============================] - 0s 158us/step - loss: 0.0053 - val_loss: 0.0093
Epoch 153/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0053 - val_loss: 0.0092
Epoch 154/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0052 - val_loss: 0.0091
Epoch 155/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0052 - val_loss: 0.0091
Epoch 156/1000
32/32 [==============================] - 0s 143us/step - loss: 0.0052 - val_loss: 0.0090
Epoch 157/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0051 - val_loss: 0.0089
Epoch 158/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0051 - val_loss: 0.0088
Epoch 159/1000
32/32 [==============================] - 0s 152us/step - loss: 0.0051 - val_loss: 0.0087
Epoch 160/1000
32/32 [==============================] - 0s 157us

Epoch 230/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0034 - val_loss: 0.0048
Epoch 231/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0034 - val_loss: 0.0047
Epoch 232/1000
32/32 [==============================] - 0s 178us/step - loss: 0.0034 - val_loss: 0.0047
Epoch 233/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0034 - val_loss: 0.0047
Epoch 234/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0033 - val_loss: 0.0046
Epoch 235/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0033 - val_loss: 0.0046
Epoch 236/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0033 - val_loss: 0.0046
Epoch 237/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0033 - val_loss: 0.0046
Epoch 238/1000
32/32 [==============================] - 0s 184us/step - loss: 0.0033 - val_loss: 0.0045
Epoch 239/1000
32/32 [==============================] - 0s 156us

Epoch 309/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0024 - val_loss: 0.0029
Epoch 310/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0023 - val_loss: 0.0029
Epoch 311/1000
32/32 [==============================] - 0s 136us/step - loss: 0.0023 - val_loss: 0.0029
Epoch 312/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0023 - val_loss: 0.0029
Epoch 313/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0023 - val_loss: 0.0029
Epoch 314/1000
32/32 [==============================] - 0s 154us/step - loss: 0.0023 - val_loss: 0.0028
Epoch 315/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0023 - val_loss: 0.0028
Epoch 316/1000
32/32 [==============================] - 0s 135us/step - loss: 0.0023 - val_loss: 0.0028
Epoch 317/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0023 - val_loss: 0.0028
Epoch 318/1000
32/32 [==============================] - 0s 156us

...

Epoch 694/1000
32/32 [==============================] - 0s 148us/step - loss: 5.9727e-04 - val_loss: 6.3526e-04
Epoch 695/1000
32/32 [==============================] - 0s 156us/step - loss: 5.9546e-04 - val_loss: 6.3331e-04
Epoch 696/1000
32/32 [==============================] - 0s 187us/step - loss: 5.9366e-04 - val_loss: 6.3138e-04
Epoch 697/1000
32/32 [==============================] - 0s 156us/step - loss: 5.9187e-04 - val_loss: 6.2945e-04
Epoch 698/1000
32/32 [==============================] - 0s 186us/step - loss: 5.9009e-04 - val_loss: 6.2753e-04
Epoch 699/1000
32/32 [==============================] - 0s 187us/step - loss: 5.8831e-04 - val_loss: 6.2562e-04
Epoch 700/1000
32/32 [==============================] - 0s 155us/step - loss: 5.8654e-04 - val_loss: 6.2373e-04
Epoch 701/1000
32/32 [==============================] - 0s 155us/step - loss: 5.8479e-04 - val_loss: 6.2184e-04
Epoch 702/1000
32/32 [==============================] - 0s 165us/step - loss: 5.8301e-04 - val_loss: 6.1

Epoch 768/1000
32/32 [==============================] - 0s 156us/step - loss: 4.7969e-04 - val_loss: 5.1052e-04
Epoch 769/1000
32/32 [==============================] - 0s 156us/step - loss: 4.7830e-04 - val_loss: 5.0906e-04
Epoch 770/1000
32/32 [==============================] - 0s 156us/step - loss: 4.7692e-04 - val_loss: 5.0761e-04
Epoch 771/1000
32/32 [==============================] - 0s 168us/step - loss: 4.7554e-04 - val_loss: 5.0616e-04
Epoch 772/1000
32/32 [==============================] - 0s 156us/step - loss: 4.7416e-04 - val_loss: 5.0474e-04
Epoch 773/1000
32/32 [==============================] - 0s 155us/step - loss: 4.7280e-04 - val_loss: 5.0331e-04
Epoch 774/1000
32/32 [==============================] - 0s 149us/step - loss: 4.7143e-04 - val_loss: 5.0187e-04
Epoch 775/1000
32/32 [==============================] - 0s 156us/step - loss: 4.7006e-04 - val_loss: 5.0045e-04
Epoch 776/1000
32/32 [==============================] - 0s 156us/step - loss: 4.6871e-04 - val_loss: 4.9

Epoch 915/1000
32/32 [==============================] - 0s 187us/step - loss: 3.1745e-04 - val_loss: 3.4470e-04
Epoch 916/1000
32/32 [==============================] - 0s 155us/step - loss: 3.1659e-04 - val_loss: 3.4383e-04
Epoch 917/1000
32/32 [==============================] - 0s 156us/step - loss: 3.1572e-04 - val_loss: 3.4298e-04
Epoch 918/1000
32/32 [==============================] - 0s 177us/step - loss: 3.1486e-04 - val_loss: 3.4211e-04
Epoch 919/1000
32/32 [==============================] - 0s 156us/step - loss: 3.1400e-04 - val_loss: 3.4127e-04
Epoch 920/1000
32/32 [==============================] - 0s 156us/step - loss: 3.1315e-04 - val_loss: 3.4040e-04
Epoch 921/1000
32/32 [==============================] - 0s 187us/step - loss: 3.1230e-04 - val_loss: 3.3956e-04
Epoch 922/1000
32/32 [==============================] - 0s 125us/step - loss: 3.1145e-04 - val_loss: 3.3871e-04
Epoch 923/1000
32/32 [==============================] - 0s 154us/step - loss: 3.1060e-04 - val_loss: 3.3

Epoch 989/1000
32/32 [==============================] - 0s 156us/step - loss: 2.5994e-04 - val_loss: 2.8765e-04
Epoch 990/1000
32/32 [==============================] - 0s 165us/step - loss: 2.5924e-04 - val_loss: 2.8698e-04
Epoch 991/1000
32/32 [==============================] - 0s 192us/step - loss: 2.5855e-04 - val_loss: 2.8630e-04
Epoch 992/1000
32/32 [==============================] - 0s 156us/step - loss: 2.5786e-04 - val_loss: 2.8562e-04
Epoch 993/1000
32/32 [==============================] - 0s 152us/step - loss: 2.5717e-04 - val_loss: 2.8494e-04
Epoch 994/1000
32/32 [==============================] - 0s 186us/step - loss: 2.5648e-04 - val_loss: 2.8426e-04
Epoch 995/1000
32/32 [==============================] - 0s 156us/step - loss: 2.5580e-04 - val_loss: 2.8359e-04
Epoch 996/1000
32/32 [==============================] - 0s 155us/step - loss: 2.5512e-04 - val_loss: 2.8292e-04
Epoch 997/1000
32/32 [==============================] - 0s 156us/step - loss: 2.5444e-04 - val_loss: 2.8

[array([[ 0.30344215,  0.5981575 ,  1.3714898 , -0.3562178 , -0.31354195,
          0.13192108,  0.13221851,  0.9863693 , -0.65575457, -0.07532035],
        [-0.86586976,  0.05136219, -0.22377595,  0.59202933, -0.23341396,
          0.14532332, -0.7001754 ,  0.12755077,  0.29753608,  0.8739443 ],
        [ 0.1435279 , -0.17363562,  0.437401  ,  0.48312804, -0.2889677 ,
          0.6466802 ,  0.5057201 ,  0.89197576,  0.6346137 ,  0.27146336]],
       dtype=float32),
 array([-0.28273633,  0.4288475 , -0.28342763,  0.14944038,  0.4033937 ,
         0.45628664, -0.44824025,  0.30493417, -0.21832311, -0.11787035],
       dtype=float32),
 array([[-0.15507554,  0.15060177, -0.5361081 ,  0.11823204, -0.00429425,
          0.18165737, -0.38480863, -0.17095774,  0.05339982, -0.13141346],
        [-0.48483717,  0.56164473, -0.24717133, -0.21279953, -0.4196161 ,
         -0.46178466,  0.55522066,  0.15830465,  0.0506952 , -0.4591233 ],
        [ 0.17016777, -0.21019843,  0.5921142 , -0.08638865, 

In [71]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure5_2nd.h5')

In [72]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_6(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_70 (Dense)             (None, 10)                40        
_________________________________________________________________
activation_47 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_71 (Dense)             (None, 15)                165       
_________________________________________________________________
activation_48 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_72 (Dense)             (None, 1)                 16        
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 15ms/step - loss: 35.

Epoch 71/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0099 - val_loss: 0.0044
Epoch 72/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0084 - val_loss: 0.0022
Epoch 73/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0084 - val_loss: 0.0012
Epoch 74/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0094 - val_loss: 9.1714e-04
Epoch 75/1000
32/32 [==============================] - 0s 148us/step - loss: 0.0106 - val_loss: 9.8924e-04
Epoch 76/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0112 - val_loss: 0.0011
Epoch 77/1000
32/32 [==============================] - 0s 143us/step - loss: 0.0109 - val_loss: 0.0011
Epoch 78/1000
32/32 [==============================] - 0s 154us/step - loss: 0.0099 - val_loss: 0.0013
Epoch 79/1000
32/32 [==============================] - 0s 153us/step - loss: 0.0085 - val_loss: 0.0017
Epoch 80/1000
32/32 [==============================] - 0s 155us/s

Epoch 150/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0040 - val_loss: 0.0011
Epoch 151/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0039 - val_loss: 0.0011
Epoch 152/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0039 - val_loss: 0.0011
Epoch 153/1000
32/32 [==============================] - 0s 142us/step - loss: 0.0039 - val_loss: 0.0011
Epoch 154/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0039 - val_loss: 0.0011
Epoch 155/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0039 - val_loss: 0.0011
Epoch 156/1000
32/32 [==============================] - 0s 154us/step - loss: 0.0038 - val_loss: 0.0011
Epoch 157/1000
32/32 [==============================] - 0s 195us/step - loss: 0.0038 - val_loss: 0.0010
Epoch 158/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0038 - val_loss: 0.0010
Epoch 159/1000
32/32 [==============================] - 0s 156us

Epoch 227/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0027 - val_loss: 6.5593e-04
Epoch 228/1000
32/32 [==============================] - 0s 157us/step - loss: 0.0027 - val_loss: 6.5122e-04
Epoch 229/1000
32/32 [==============================] - 0s 200us/step - loss: 0.0027 - val_loss: 6.4657e-04
Epoch 230/1000
32/32 [==============================] - 0s 174us/step - loss: 0.0027 - val_loss: 6.4198e-04
Epoch 231/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0027 - val_loss: 6.3743e-04
Epoch 232/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0026 - val_loss: 6.3289e-04
Epoch 233/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0026 - val_loss: 6.2840e-04
Epoch 234/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0026 - val_loss: 6.2395e-04
Epoch 235/1000
32/32 [==============================] - 0s 148us/step - loss: 0.0026 - val_loss: 6.1951e-04
Epoch 236/1000
32/32 [======

Epoch 303/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0019 - val_loss: 3.7963e-04
Epoch 304/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0019 - val_loss: 3.7695e-04
Epoch 305/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0019 - val_loss: 3.7427e-04
Epoch 306/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0019 - val_loss: 3.7165e-04
Epoch 307/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0019 - val_loss: 3.6903e-04
Epoch 308/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0019 - val_loss: 3.6643e-04
Epoch 309/1000
32/32 [==============================] - 0s 195us/step - loss: 0.0018 - val_loss: 3.6385e-04
Epoch 310/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0018 - val_loss: 3.6128e-04
Epoch 311/1000
32/32 [==============================] - 0s 135us/step - loss: 0.0018 - val_loss: 3.5875e-04
Epoch 312/1000
32/32 [======

...

Epoch 751/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6607e-04 - val_loss: 6.6458e-05
Epoch 752/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6502e-04 - val_loss: 6.6396e-05
Epoch 753/1000
32/32 [==============================] - 0s 125us/step - loss: 2.6397e-04 - val_loss: 6.6335e-05
Epoch 754/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6293e-04 - val_loss: 6.6272e-05
Epoch 755/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6189e-04 - val_loss: 6.6215e-05
Epoch 756/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6086e-04 - val_loss: 6.6146e-05
Epoch 757/1000
32/32 [==============================] - 0s 186us/step - loss: 2.5983e-04 - val_loss: 6.6091e-05
Epoch 758/1000
32/32 [==============================] - 0s 156us/step - loss: 2.5881e-04 - val_loss: 6.6022e-05
Epoch 759/1000
32/32 [==============================] - 0s 186us/step - loss: 2.5779e-04 - val_loss: 6.5

Epoch 825/1000
32/32 [==============================] - 0s 218us/step - loss: 2.0029e-04 - val_loss: 6.2669e-05
Epoch 826/1000
32/32 [==============================] - 0s 218us/step - loss: 1.9954e-04 - val_loss: 6.2627e-05
Epoch 827/1000
32/32 [==============================] - 0s 218us/step - loss: 1.9881e-04 - val_loss: 6.2577e-05
Epoch 828/1000
32/32 [==============================] - 0s 218us/step - loss: 1.9808e-04 - val_loss: 6.2538e-05
Epoch 829/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9734e-04 - val_loss: 6.2492e-05
Epoch 830/1000
32/32 [==============================] - 0s 249us/step - loss: 1.9662e-04 - val_loss: 6.2455e-05
Epoch 831/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9589e-04 - val_loss: 6.2412e-05
Epoch 832/1000
32/32 [==============================] - 0s 156us/step - loss: 1.9518e-04 - val_loss: 6.2371e-05
Epoch 833/1000
32/32 [==============================] - 0s 187us/step - loss: 1.9446e-04 - val_loss: 6.2

Epoch 899/1000
32/32 [==============================] - 0s 156us/step - loss: 1.5383e-04 - val_loss: 5.9854e-05
Epoch 900/1000
32/32 [==============================] - 0s 155us/step - loss: 1.5330e-04 - val_loss: 5.9813e-05
Epoch 901/1000
32/32 [==============================] - 0s 187us/step - loss: 1.5278e-04 - val_loss: 5.9781e-05
Epoch 902/1000
32/32 [==============================] - 0s 187us/step - loss: 1.5226e-04 - val_loss: 5.9740e-05
Epoch 903/1000
32/32 [==============================] - 0s 218us/step - loss: 1.5174e-04 - val_loss: 5.9707e-05
Epoch 904/1000
32/32 [==============================] - 0s 187us/step - loss: 1.5122e-04 - val_loss: 5.9665e-05
Epoch 905/1000
32/32 [==============================] - 0s 187us/step - loss: 1.5071e-04 - val_loss: 5.9640e-05
Epoch 906/1000
32/32 [==============================] - 0s 156us/step - loss: 1.5020e-04 - val_loss: 5.9605e-05
Epoch 907/1000
32/32 [==============================] - 0s 218us/step - loss: 1.4969e-04 - val_loss: 5.9

Epoch 973/1000
32/32 [==============================] - 0s 155us/step - loss: 1.2077e-04 - val_loss: 5.7196e-05
Epoch 974/1000
32/32 [==============================] - 0s 186us/step - loss: 1.2039e-04 - val_loss: 5.7154e-05
Epoch 975/1000
32/32 [==============================] - 0s 155us/step - loss: 1.2002e-04 - val_loss: 5.7122e-05
Epoch 976/1000
32/32 [==============================] - 0s 155us/step - loss: 1.1965e-04 - val_loss: 5.7083e-05
Epoch 977/1000
32/32 [==============================] - 0s 156us/step - loss: 1.1927e-04 - val_loss: 5.7051e-05
Epoch 978/1000
32/32 [==============================] - 0s 157us/step - loss: 1.1891e-04 - val_loss: 5.7008e-05
Epoch 979/1000
32/32 [==============================] - 0s 187us/step - loss: 1.1854e-04 - val_loss: 5.6971e-05
Epoch 980/1000
32/32 [==============================] - 0s 187us/step - loss: 1.1817e-04 - val_loss: 5.6931e-05
Epoch 981/1000
32/32 [==============================] - 0s 157us/step - loss: 1.1781e-04 - val_loss: 5.6

[array([[-0.1186784 ,  0.5071984 ,  0.71971214,  0.19782898, -0.39259022,
         -0.3644355 ,  0.90937513,  0.3558004 ,  0.22596833,  0.52333874],
        [-0.10546404, -0.339198  , -0.45486286,  0.33948243, -0.37335497,
          0.2354284 ,  0.03002328,  0.23226644,  0.80701184, -0.6658903 ],
        [-1.4980038 , -0.9150436 , -0.33741596, -0.0348817 , -0.8857887 ,
          0.04428585,  1.5919839 ,  1.6355377 ,  0.8622473 ,  0.44304168]],
       dtype=float32),
 array([-0.3669674 ,  0.43546796,  0.2732511 ,  0.17991593, -0.28736356,
         0.4976804 , -0.15350744,  0.38114703, -0.31616494, -0.4709147 ],
       dtype=float32),
 array([[ 0.3071579 , -0.2548043 ,  0.41262117,  0.3132513 ,  0.3595088 ,
         -0.28894395,  0.18239592,  0.38625085,  0.39865914, -0.251236  ,
          0.07151511, -0.30121744, -0.06108657,  0.56440085, -0.22462454],
        [-0.18699262,  0.15194319,  0.37452093,  0.15611991, -0.5705591 ,
          0.20795155,  0.12751439,  0.38170597, -0.41096804,  

In [73]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure6_2nd.h5')